In [226]:
# Celda 0: [SETUP & PARÁMETROS GLOBALES]
# ------------------------------------------------------------
# Responsabilidad única:
# - Fijar parámetros globales (símbolo, timeframe, fechas, seed)
# - Ajustar variables de entorno para reproducibilidad (threads/BLAS)
# - Resolver rutas base y crear estructura de carpetas del proyecto
# - Forzar UTC, fijar seeds y PATH del proyecto
# - Guardar un "snapshot" de entorno (versiones / OS / env clave)
# - Imprimir verificaciones exhaustivas de todo lo anterior
# ------------------------------------------------------------
# NOTA: Esta celda es la ÚNICA que puede definir o modificar TWF_*
#       El resto de celdas SOLO leen del entorno o de archivos generados aquí.
# ------------------------------------------------------------

# ----(0) Variables de entorno para reproducibilidad (antes de imports científicos)----
import os
# Mono-hilo para BLAS/OMP (mejor determinismo). Si luego quieres velocidad, sube estos valores.
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")
os.environ.setdefault("VECLIB_MAXIMUM_THREADS", "1")
os.environ.setdefault("NUMEXPR_NUM_THREADS", "1")
# Polars: por defecto usa todos los núcleos. Si quieres restringir, define TWF_POLARS_THREADS (p.ej. "1" o "0" para auto).
if "TWF_POLARS_THREADS" in os.environ:
    os.environ["POLARS_MAX_THREADS"] = os.environ["TWF_POLARS_THREADS"]

# ----(1) Imports mínimos (después de fijar variables de entorno)----
import sys
import json
import time
import random
import platform
from pathlib import Path
from datetime import datetime

# Carga paulatina de libs científicas (manejar ausencias con mensajes claros)
try:
    import numpy as np
    NP_OK = True
except Exception as e:
    NP_OK = False
    print(f"⚠️  [AVISO] NumPy no disponible: {e}")

try:
    import polars as pl
    PL_OK = True
except Exception as e:
    PL_OK = False
    print(f"⚠️  [AVISO] Polars no disponible: {e}")

# ----(2) Utilidades de impresión y validación----
def _print_kv(k, v):
    print(f"   - {k:<28}: {v}")

def _ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)
    return p

def _is_valid_date(s: str) -> bool:
    try:
        datetime.fromisoformat(s)
        return True
    except Exception:
        return False

# ----(3) Parámetros globales (ÚNICA FUENTE DE VERDAD)----
# Si ya vienes con variables de entorno definidas desde fuera, las respetamos.
# Si no, fijamos valores por defecto SEGUROS y los propagamos a os.environ.
DEFAULTS = {
    "TWF_SYMBOL": "BTCUSDT",         # Activo (ejemplos: BTCUSDT, ETHUSDT, ES, AAPL)
    "TWF_TF": "15m",                  # Timeframe (ej.: 1m, 5m, 15m, 1h, 4h, 1d)
    "TWF_START_DATE": "2021-01-01",  # ISO-8601 YYYY-MM-DD
    "TWF_END_DATE": "2099-12-31",    # ISO-8601 YYYY-MM-DD (usar futuro amplio si harás filtros luego)
    "TWF_SEED_BASE": "42",           # Semilla base reproducible
    # Opcional: fuerza número de hilos de Polars (si lo deseas): os.environ["TWF_POLARS_THREADS"]="1"
}

print("🧭 Paso 3: Leyendo/estableciendo parámetros globales TWF_* ...")
for k, default_v in DEFAULTS.items():
    v = os.environ.get(k, default_v)
    os.environ[k] = str(v)
    _print_kv(k, os.environ[k])

# Validaciones de formato de fechas
assert _is_valid_date(os.environ["TWF_START_DATE"]), "Fecha inválida en TWF_START_DATE (use YYYY-MM-DD)"
assert _is_valid_date(os.environ["TWF_END_DATE"]), "Fecha inválida en TWF_END_DATE (use YYYY-MM-DD)"
dt_start = datetime.fromisoformat(os.environ["TWF_START_DATE"])
dt_end   = datetime.fromisoformat(os.environ["TWF_END_DATE"])
assert dt_start <= dt_end, "TWF_START_DATE debe ser <= TWF_END_DATE"

# ----(4) Zona horaria (UTC)----
print("🕒 Paso 4: Forzando UTC para consistencia ...")
os.environ["TZ"] = "UTC"
try:
    # En sistemas Unix, tzset aplica TZ inmediatamente
    import time as _time
    if hasattr(_time, "tzset"):
        _time.tzset()
        print("   ✓ TZ=UTC aplicado vía tzset()")
    else:
        print("   ℹ️  tzset() no disponible en este OS; se usará TZ=UTC en procesos hijos.")
except Exception as e:
    print(f"   ⚠️  No se pudo aplicar tzset(): {e}")

# ----(5) Raíz del proyecto y sys.path----
print("📁 Paso 5: Resolviendo rutas base y sys.path ...")
# Puedes sobreescribir la raíz con TWF_BASE si tu repo no coincide con Path.cwd()
PROJ_ROOT = Path(os.environ.get("TWF_BASE", Path.cwd())).resolve()
if str(PROJ_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJ_ROOT))
_print_kv("PROJ_ROOT", PROJ_ROOT)

# ----(6) Construcción de rutas por activo/timeframe----
print("🗂️  Paso 6: Creando estructura de carpetas por activo/timeframe ...")
SYMBOL = os.environ["TWF_SYMBOL"]
TF     = os.environ["TWF_TF"]

OUT_BASE = PROJ_ROOT / "outputs" / SYMBOL / TF
PATHS = {
    "base": OUT_BASE,
    "data_raw": OUT_BASE / "data_raw",
    "data_curated": OUT_BASE / "data_curated",
    "features": OUT_BASE / "features",
    "forward": OUT_BASE / "forward",
    "wf": OUT_BASE / "wf",
    "models": OUT_BASE / "models",
    "reports": OUT_BASE / "reports",
    "figures": OUT_BASE / "figures",
    "logs": OUT_BASE / "logs",
    "tmp": OUT_BASE / "tmp",
}

for name, p in PATHS.items():
    _ensure_dir(p)
    _print_kv(f"PATHS['{name}']", p)

# Guardamos un archivo de prueba de escritura/lectura para validar permisos.
print("📝 Paso 6.1: Validando permisos de escritura/lectura en outputs ...")
touch_file = PATHS["tmp"] / "_write_test.ok"
try:
    touch_file.write_text("ok")
    assert touch_file.read_text() == "ok"
    print(f"   ✓ OK escritura/lectura en {touch_file}")
    touch_file.unlink(missing_ok=True)
except Exception as e:
    raise RuntimeError(f"❌ Error de permisos en {PATHS['tmp']}: {e}")

# ----(7) Fijar semillas (reproducibilidad)----
print("🎲 Paso 7: Fijando semillas para reproducibilidad ...")
SEED_BASE = int(os.environ["TWF_SEED_BASE"])
random.seed(SEED_BASE)
os.environ["PYTHONHASHSEED"] = str(SEED_BASE)
if NP_OK:
    np.random.seed(SEED_BASE)
_print_kv("SEED_BASE", SEED_BASE)

# ----(8) Snapshot de entorno (versiones y sistema)----
print("🧾 Paso 8: Creando snapshot de entorno ...")
snapshot = {
    "created_utc": datetime.utcnow().isoformat(timespec="seconds") + "Z",
    "platform": {
        "python": sys.version.replace("\n", " "),
        "implementation": platform.python_implementation(),
        "system": platform.system(),
        "release": platform.release(),
        "machine": platform.machine(),
        "processor": platform.processor(),
    },
    "packages": {},
    "env": {
        "TWF_SYMBOL": os.environ["TWF_SYMBOL"],
        "TWF_TF": os.environ["TWF_TF"],
        "TWF_START_DATE": os.environ["TWF_START_DATE"],
        "TWF_END_DATE": os.environ["TWF_END_DATE"],
        "TWF_SEED_BASE": os.environ["TWF_SEED_BASE"],
        "POLARS_MAX_THREADS": os.environ.get("POLARS_MAX_THREADS", None),
        "OMP_NUM_THREADS": os.environ.get("OMP_NUM_THREADS", None),
        "OPENBLAS_NUM_THREADS": os.environ.get("OPENBLAS_NUM_THREADS", None),
        "MKL_NUM_THREADS": os.environ.get("MKL_NUM_THREADS", None),
    },
    "paths": {k: str(v) for k, v in PATHS.items()},
}

# Versiones de paquetes (si están disponibles)
if NP_OK:
    snapshot["packages"]["numpy"] = np.__version__
if PL_OK:
    snapshot["packages"]["polars"] = pl.__version__

# Persistimos snapshot
snapshot_path = PATHS["logs"] / "run_snapshot.json"
with snapshot_path.open("w", encoding="utf-8") as f:
    json.dump(snapshot, f, indent=2, ensure_ascii=False)

_print_kv("snapshot_path", snapshot_path)
print("   ✓ Snapshot escrito.")

# ----(9) Resumen final imprimible (para “ojo clínico”)----
print("\n✅ RESUMEN DE SETUP")
_print_kv("SYMBOL", SYMBOL)
_print_kv("TIMEFRAME", TF)
_print_kv("START_DATE", os.environ["TWF_START_DATE"])
_print_kv("END_DATE", os.environ["TWF_END_DATE"])
_print_kv("OUTPUT_BASE", PATHS["base"])
_print_kv("LOG_FILE", snapshot_path)
print("   ✓ Celda 0 finalizada sin errores.\n")

# ----(10) Contratos/Asserts para las siguientes celdas----
# Las próximas celdas SOLO deben leer TWF_* y PATHS desde disco/entorno. No modificar aquí.
assert all(k in os.environ for k in ("TWF_SYMBOL","TWF_TF","TWF_START_DATE","TWF_END_DATE","TWF_SEED_BASE")), \
    "Faltan variables TWF_* en entorno."
for needed in ("base","logs","tmp"):
    assert PATHS[needed].exists(), f"Ruta obligatoria no existe: PATHS['{needed}']"


🧭 Paso 3: Leyendo/estableciendo parámetros globales TWF_* ...
   - TWF_SYMBOL                  : BTCUSDT
   - TWF_TF                      : 15m
   - TWF_START_DATE              : 2021-01-01
   - TWF_END_DATE                : 2099-12-31
   - TWF_SEED_BASE               : 42
🕒 Paso 4: Forzando UTC para consistencia ...
   ℹ️  tzset() no disponible en este OS; se usará TZ=UTC en procesos hijos.
📁 Paso 5: Resolviendo rutas base y sys.path ...
   - PROJ_ROOT                   : C:\Quant\TWF
🗂️  Paso 6: Creando estructura de carpetas por activo/timeframe ...
   - PATHS['base']               : C:\Quant\TWF\outputs\BTCUSDT\15m
   - PATHS['data_raw']           : C:\Quant\TWF\outputs\BTCUSDT\15m\data_raw
   - PATHS['data_curated']       : C:\Quant\TWF\outputs\BTCUSDT\15m\data_curated
   - PATHS['features']           : C:\Quant\TWF\outputs\BTCUSDT\15m\features
   - PATHS['forward']            : C:\Quant\TWF\outputs\BTCUSDT\15m\forward
   - PATHS['wf']                 : C:\Quant\TWF\outputs\BTCUSD

C:\Users\PC\AppData\Local\Temp\ipykernel_40828\208170312.py:160: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "created_utc": datetime.utcnow().isoformat(timespec="seconds") + "Z",


In [227]:
# Celda 1: [CONFIGURACIÓN PERSISTENTE → config_{SYMBOL}_{TF}.json]
# -----------------------------------------------------------------------------
# Responsabilidad única:
# - Leer TWF_* del entorno (definidos en Celda 0).
# - Derivar parámetros estándar del pipeline (data/features/labeling/WF/compute).
# - Persistir un JSON de configuración único para este símbolo/timeframe.
# - Imprimir verificaciones exhaustivas del contenido y las rutas.
# -----------------------------------------------------------------------------

import os, json
from pathlib import Path
from datetime import datetime, timezone

# ----(A) Utilidades locales----
def _print_kv(k, v):
    print(f"   - {k:<28}: {v}")

def _ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True); return p

def _timeframe_to_ms(tf: str) -> int:
    """Convierte strings tipo '1m','5m','15m','1h','4h','1d' a milisegundos."""
    tf = tf.strip().lower()
    if tf.endswith("ms"):
        return int(tf[:-2])
    if tf.endswith("s"):
        return int(tf[:-1]) * 1_000
    if tf.endswith("m"):
        return int(tf[:-1]) * 60_000
    if tf.endswith("h"):
        return int(tf[:-1]) * 3_600_000
    if tf.endswith("d"):
        return int(tf[:-1]) * 86_400_000
    raise ValueError(f"Timeframe no reconocido: {tf}")

# ----(B) Contratos de entrada (no tocar TWF_* aquí)----
print("📜 Leyendo variables TWF_* del entorno ...")
needed = ("TWF_SYMBOL","TWF_TF","TWF_START_DATE","TWF_END_DATE","TWF_SEED_BASE")
assert all(k in os.environ for k in needed), "Faltan TWF_* en el entorno; ejecuta primero la Celda 0."
SYMBOL = os.environ["TWF_SYMBOL"]
TF     = os.environ["TWF_TF"]
START  = os.environ["TWF_START_DATE"]
END    = os.environ["TWF_END_DATE"]
SEED   = int(os.environ["TWF_SEED_BASE"])
for k in needed: _print_kv(k, os.environ[k])

# ----(C) Rutas base (derivadas de TWF_BASE o CWD; sin depender de variables en memoria)----
PROJ_ROOT = Path(os.environ.get("TWF_BASE", Path.cwd())).resolve()
OUT_BASE  = PROJ_ROOT / "outputs" / SYMBOL / TF
LOGS_DIR  = _ensure_dir(OUT_BASE / "logs")
_print_kv("PROJ_ROOT", PROJ_ROOT)
_print_kv("OUT_BASE", OUT_BASE)
_print_kv("LOGS_DIR", LOGS_DIR)

# ----(D) Derivados clave de configuración (valores razonables por defecto)----
print("🧮 Derivando parámetros de configuración ...")
interval_ms = _timeframe_to_ms(TF)
_print_kv("expected_interval_ms", interval_ms)

# Nota: Estos valores son seguros para arrancar y pueden refinarse luego.
#       Mantén listas cortas para validación rápida fin-a-fin; luego se amplían.
config = {
    "schema_version": 1,
    "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
    "run": {
        "symbol": SYMBOL,
        "timeframe": TF,
        "start_date": START,
        "end_date": END,
        "seed_base": SEED,
    },
    "data": {
        "source": "binance",                 # Puedes cambiar a 'csv', 'parquet', etc. en el futuro
        "market_hours": "24x7",
        "expected_interval_ms": interval_ms,
        "allow_gaps": False,                 # Si True, se loguean; si False, se consideran error aguas abajo
    },
    "features": {
        "ma": {
            "types": ["ema","wma"],          # Mantener corto para pruebas; luego agregar "hma","kama" si quieres
            "windows": [10, 20, 50],         # Grilla inicial corta; luego expandimos
            "price_col": "close",
        },
        "er": {
            "windows": [10, 20, 50],
            "thresholds": [0.2, 0.3, 0.4],   # Filtro de “tendencia limpia”
            "price_col": "close",
        },
        "slope": {
            "window": 10,
            "price_col": "close",
        }
    },
    "labeling": {
        "triple_barrier": {
            "horizon_bars": 48,              # ~2 días en TF=1h (ajustable)
            "up_mult": 2.0,                  # TP múltiplo relativo (placeholder)
            "dn_mult": 1.0,                  # SL múltiplo relativo (placeholder)
            "timeout_bars": 48               # Igual a horizon por defecto
        }
    },
    "walk_forward": {
        "train_bars": 2000,
        "test_bars": 500,
        "embargo_bars": 0,
        "top_k": 10,                         # Top K combinaciones post-clustering
        "min_trades": 50                     # Filtro mínimo de operaciones por combo
    },
    "compute": {
        "n_jobs": os.cpu_count() or 4,
        "polars_threads": os.environ.get("POLARS_MAX_THREADS", None),
    },
    "paths": {
        "base": str(OUT_BASE),
        # Rutas estándar consumidas por celdas posteriores
        "data_raw": str(OUT_BASE / "data_raw"),
        "data_curated": str(OUT_BASE / "data_curated"),
        "features": str(OUT_BASE / "features"),
        "forward": str(OUT_BASE / "forward"),
        "wf": str(OUT_BASE / "wf"),
        "models": str(OUT_BASE / "models"),
        "reports": str(OUT_BASE / "reports"),
        "figures": str(OUT_BASE / "figures"),
        "logs": str(OUT_BASE / "logs"),
        "tmp": str(OUT_BASE / "tmp"),
    }
}

# ----(E) Persistencia del config----
CONFIG_PATH = LOGS_DIR / f"config_{SYMBOL}_{TF}.json"
with CONFIG_PATH.open("w", encoding="utf-8") as f:
    json.dump(config, f, indent=2, ensure_ascii=False)

# ----(F) Impresiones de verificación (resumen claro)----
print("\n🧾 CONFIG RESUMEN")
_print_kv("config_path", CONFIG_PATH)
_print_kv("schema_version", config["schema_version"])
_print_kv("symbol", config["run"]["symbol"])
_print_kv("timeframe", config["run"]["timeframe"])
_print_kv("start_date", config["run"]["start_date"])
_print_kv("end_date", config["run"]["end_date"])
_print_kv("seed_base", config["run"]["seed_base"])
_print_kv("expected_interval_ms", config["data"]["expected_interval_ms"])
_print_kv("features.ma.types", ",".join(config["features"]["ma"]["types"]))
_print_kv("features.ma.windows", config["features"]["ma"]["windows"])
_print_kv("features.er.windows", config["features"]["er"]["windows"])
_print_kv("features.er.thresholds", config["features"]["er"]["thresholds"])
_print_kv("labeling.tb.horizon_bars", config["labeling"]["triple_barrier"]["horizon_bars"])
_print_kv("walk_forward.train_bars", config["walk_forward"]["train_bars"])
_print_kv("walk_forward.test_bars", config["walk_forward"]["test_bars"])
_print_kv("walk_forward.top_k", config["walk_forward"]["top_k"])
_print_kv("compute.n_jobs", config["compute"]["n_jobs"])

# ----(G) Contratos de salida----
assert CONFIG_PATH.exists() and CONFIG_PATH.stat().st_size > 0, "Config no se escribió correctamente."
print("   ✓ Configuración persistida correctamente.\n")


📜 Leyendo variables TWF_* del entorno ...
   - TWF_SYMBOL                  : BTCUSDT
   - TWF_TF                      : 15m
   - TWF_START_DATE              : 2021-01-01
   - TWF_END_DATE                : 2099-12-31
   - TWF_SEED_BASE               : 42
   - PROJ_ROOT                   : C:\Quant\TWF
   - OUT_BASE                    : C:\Quant\TWF\outputs\BTCUSDT\15m
   - LOGS_DIR                    : C:\Quant\TWF\outputs\BTCUSDT\15m\logs
🧮 Derivando parámetros de configuración ...
   - expected_interval_ms        : 900000

🧾 CONFIG RESUMEN
   - config_path                 : C:\Quant\TWF\outputs\BTCUSDT\15m\logs\config_BTCUSDT_15m.json
   - schema_version              : 1
   - symbol                      : BTCUSDT
   - timeframe                   : 15m
   - start_date                  : 2021-01-01
   - end_date                    : 2099-12-31
   - seed_base                   : 42
   - expected_interval_ms        : 900000
   - features.ma.types           : ema,wma
   - features.ma.windo

In [228]:
# Celda 2: [INGESTA & CURACIÓN DE OHLCV]
# -----------------------------------------------------------------------------
# Responsabilidad única:
# - Leer config_{SYMBOL}_{TF}.json
# - Descargar (o reaprovechar) OHLCV del origen indicado (por defecto: Binance)
# - Normalizar esquema, ordenar, deduplicar, validar gaps
# - Persistir Parquet curado y manifest JSON con métricas de cobertura
# - Imprimir verificaciones exhaustivas y progreso por "chunks"
# -----------------------------------------------------------------------------

import os, json, math, time, hashlib
from pathlib import Path
from datetime import datetime, timezone
from typing import List, Dict

# ----(A) Utilidades de impresión y tiempo----
def _print_kv(k, v):
    print(f"   - {k:<28}: {v}")

def _to_utc_ms(dt_iso: str) -> int:
    # Acepta 'YYYY-MM-DD' o ISO completo; retorna epoch ms en UTC.
    if len(dt_iso) == 10:
        dt = datetime.fromisoformat(dt_iso).replace(tzinfo=timezone.utc)
    else:
        dt = datetime.fromisoformat(dt_iso)
        if dt.tzinfo is None:
            dt = dt.replace(tzinfo=timezone.utc)
        else:
            dt = dt.astimezone(timezone.utc)
    return int(dt.timestamp() * 1000)

def _ms_to_iso(ms: int) -> str:
    return datetime.fromtimestamp(ms/1000, tz=timezone.utc).isoformat(timespec="seconds")

def _timeframe_to_ms(tf: str) -> int:
    tf = tf.strip().lower()
    if tf.endswith("ms"): return int(tf[:-2])
    if tf.endswith("s"):  return int(tf[:-1]) * 1_000
    if tf.endswith("m"):  return int(tf[:-1]) * 60_000
    if tf.endswith("h"):  return int(tf[:-1]) * 3_600_000
    if tf.endswith("d"):  return int(tf[:-1]) * 86_400_000
    raise ValueError(f"Timeframe no reconocido: {tf}")

# ----(B) Cargar configuración persistente (NO modificar TWF_* aquí)----
needed = ("TWF_SYMBOL","TWF_TF","TWF_START_DATE","TWF_END_DATE","TWF_SEED_BASE")
assert all(k in os.environ for k in needed), "Faltan TWF_*; ejecuta primero Celda 0 y luego Celda 1."
SYMBOL = os.environ["TWF_SYMBOL"]
TF     = os.environ["TWF_TF"]

PROJ_ROOT = Path(os.environ.get("TWF_BASE", Path.cwd())).resolve()
CONFIG_PATH = PROJ_ROOT / "outputs" / SYMBOL / TF / "logs" / f"config_{SYMBOL}_{TF}.json"
assert CONFIG_PATH.exists(), f"No existe el archivo de configuración: {CONFIG_PATH}"

print("📥 Leyendo configuración ...")
with CONFIG_PATH.open("r", encoding="utf-8") as f:
    CFG = json.load(f)

DATA_SOURCE = CFG["data"]["source"]
INTERVAL_MS = CFG["data"]["expected_interval_ms"]
START_ISO   = CFG["run"]["start_date"]
END_ISO     = CFG["run"]["end_date"]

PATHS = {k: Path(v) for k, v in CFG["paths"].items()}
for k_name in ("data_raw","data_curated","logs","tmp"):
    PATHS[k_name].mkdir(parents=True, exist_ok=True)

_print_kv("symbol", SYMBOL)
_print_kv("timeframe", TF)
_print_kv("data_source", DATA_SOURCE)
_print_kv("expected_interval_ms", INTERVAL_MS)
_print_kv("start_date", START_ISO)
_print_kv("end_date", END_ISO)

# ----(C) Determinar rango temporal en ms (clamp a "ahora")----
START_MS = _to_utc_ms(START_ISO)
# Clamp final a "ahora" (UTC), para no pedir velas futuras
NOW_MS   = int(datetime.now(timezone.utc).timestamp() * 1000)
END_MS_RAW = _to_utc_ms(END_ISO)
END_MS  = min(END_MS_RAW, NOW_MS)
_print_kv("start_ms", START_MS)
_print_kv("end_ms", END_MS)
_print_kv("now_ms", NOW_MS)

# ----(D) Paths de salida----
PARQUET_PATH = PATHS["data_curated"] / f"ohlc_{SYMBOL}_{TF}.parquet"
MANIFEST_PATH = PATHS["logs"] / f"manifest_ohlc_{SYMBOL}_{TF}.json"
_print_kv("parquet_out", PARQUET_PATH)
_print_kv("manifest_out", MANIFEST_PATH)

# ----(E) Intentar reutilizar si ya existe (idempotencia)----
REUSE_IF_EXISTS = True
if REUSE_IF_EXISTS and PARQUET_PATH.exists():
    print("♻️  Archivo Parquet ya existe. Se validará y reutilizará si pasa controles...")
else:
    print("🆕 No existe Parquet curado previo o REUSE_IF_EXISTS=False. Se descargará y generará de cero.")

# ----(F) Descarga desde Binance (HTTP REST) si es necesario----
rows: List[List] = []
if (not PARQUET_PATH.exists()) or (not REUSE_IF_EXISTS):
    assert DATA_SOURCE.lower() == "binance", "Por ahora esta celda solo implementa 'binance' como origen."

    try:
        import requests
    except Exception as e:
        raise RuntimeError(
            "Falta el paquete 'requests' para descargar de Binance. Instálalo con: pip install requests"
        ) from e

    session = requests.Session()
    endpoint = "https://api.binance.com/api/v3/klines"
    limit = 1000
    cursor = START_MS
    n_calls = 0

    print("🌐 Iniciando paginación de klines (Binance) ...")
    while cursor < END_MS:
        params = {
            "symbol": SYMBOL.upper(),
            "interval": TF.lower(),
            "startTime": cursor,
            "limit": limit
        }
        # Opcionalmente podríamos pasar "endTime", pero con startTime+limit avanzamos sin solapar.
        for attempt in range(5):
            try:
                resp = session.get(endpoint, params=params, timeout=30)
                n_calls += 1
                if resp.status_code == 200:
                    data = resp.json()
                    break
                elif resp.status_code in (418, 429):  # rate limited / banned
                    wait_s = 2 ** attempt
                    print(f"   ⚠️  Rate limit HTTP {resp.status_code}. Reintento en {wait_s}s ... (attempt {attempt+1}/5)")
                    time.sleep(wait_s)
                else:
                    raise RuntimeError(f"HTTP {resp.status_code}: {resp.text[:200]}")
            except Exception as ex:
                wait_s = 2 ** attempt
                print(f"   ⚠️  Error de red: {ex}. Reintento en {wait_s}s ... (attempt {attempt+1}/5)")
                time.sleep(wait_s)
        else:
            raise RuntimeError("❌ Falló la descarga tras múltiples reintentos.")

        if not data:
            # Si no hay datos, avanzamos una vela para evitar bucles infinitos
            cursor += INTERVAL_MS
            print(f"   ℹ️  Respuesta vacía, avanzando cursor a {_ms_to_iso(cursor)}")
            continue

        # 'data' es una lista de klines. Cada kline:
        # [ openTime, open, high, low, close, volume, closeTime, quoteVolume, trades, takerBase, takerQuote, ignore ]
        first_open_ms = data[0][0]
        last_open_ms  = data[-1][0]
        rows.extend(data)

        print(f"   ✓ Chunk recibido: {len(data):4d} velas | "
              f"{_ms_to_iso(first_open_ms)} → {_ms_to_iso(last_open_ms)} | "
              f"total acumulado: {len(rows)}")

        # Avanzar cursor a la siguiente vela tras el último openTime
        cursor = last_open_ms + INTERVAL_MS

    print(f"✅ Descarga finalizada. Llamadas HTTP: {n_calls}, velas totales: {len(rows)}")
else:
    print("🚫 Descarga omitida (reutilizando Parquet existente).")

# ----(G) Construcción de DataFrame y normalización de esquema----
import polars as pl

def _rows_to_df(data: List[List]) -> "pl.DataFrame":
    if not data:
        return pl.DataFrame(
            schema={
                "ts": pl.Datetime(time_unit="ms", time_zone="UTC"),
                "open": pl.Float64, "high": pl.Float64, "low": pl.Float64, "close": pl.Float64,
                "volume": pl.Float64,
                "close_time": pl.Datetime(time_unit="ms", time_zone="UTC"),
                "quote_volume": pl.Float64, "trades": pl.Int64,
                "taker_base": pl.Float64, "taker_quote": pl.Float64
            }
        )
    df = pl.DataFrame(data, orient="row").select([
        pl.col("column_0").alias("open_time_ms"),
        pl.col("column_1").cast(pl.Float64).alias("open"),
        pl.col("column_2").cast(pl.Float64).alias("high"),
        pl.col("column_3").cast(pl.Float64).alias("low"),
        pl.col("column_4").cast(pl.Float64).alias("close"),
        pl.col("column_5").cast(pl.Float64).alias("volume"),
        pl.col("column_6").alias("close_time_ms"),
        pl.col("column_7").cast(pl.Float64).alias("quote_volume"),
        pl.col("column_8").cast(pl.Int64).alias("trades"),
        pl.col("column_9").cast(pl.Float64).alias("taker_base"),
        pl.col("column_10").cast(pl.Float64).alias("taker_quote"),
    ]).with_columns(
        ts = pl.from_epoch(pl.col("open_time_ms")/1000).dt.replace_time_zone("UTC"),
        close_time = pl.from_epoch(pl.col("close_time_ms")/1000).dt.replace_time_zone("UTC"),
    ).drop(["open_time_ms","close_time_ms"]).select([
        "ts","open","high","low","close","volume","close_time","quote_volume","trades","taker_base","taker_quote"
    ])
    # Orden, deduplicación por ts
    df = df.sort("ts").unique(subset=["ts"], keep="last")
    return df

if (not PARQUET_PATH.exists()) or (not REUSE_IF_EXISTS):
    print("🧱 Construyendo DataFrame Polars ...")
    df = _rows_to_df(rows)
    print(f"   ✓ Filas tras normalizar: {df.height}, columnas: {df.width}")
else:
    print("📖 Leyendo DataFrame desde Parquet existente ...")
    df = pl.read_parquet(PARQUET_PATH)
    print(f"   ✓ Filas leídas: {df.height}, columnas: {df.width}")

# ----(H) Validaciones y métricas de cobertura----
print("🔎 Validando esquema y cobertura ...")
expected_cols = ["ts","open","high","low","close","volume","close_time","quote_volume","trades","taker_base","taker_quote"]
missing = [c for c in expected_cols if c not in df.columns]
assert not missing, f"Faltan columnas en OHLCV: {missing}"

if df.height > 0:
    # first_ms / last_ms usando expresiones Polars (evita objetos Python intermedios)
    bounds = df.select(
        pl.col("ts").dt.epoch("ms").min().alias("first_ms"),
        pl.col("ts").dt.epoch("ms").max().alias("last_ms")
    ).row(0)
    first_ts, last_ts = int(bounds[0]), int(bounds[1])

    # Cálculo de esperadas (bordes inclusivos)
    expected_bars = (last_ts - first_ts) // INTERVAL_MS + 1
    coverage = df.height / expected_bars if expected_bars > 0 else 0.0

    # Gaps con diffs en ms (también 100% en Polars)
    diffs_ms = df.select(pl.col("ts").dt.epoch("ms").diff().alias("dms")).get_column("dms").drop_nulls()
    gap_bars = (diffs_ms / INTERVAL_MS).round(0).cast(pl.Int64) - 1
    total_gaps = int((gap_bars > 0).sum())
    max_gap_bars = int(gap_bars.max()) if gap_bars.len() > 0 else 0

    print("🧮 Métricas:")
    _print_kv("first_ts", _ms_to_iso(first_ts))
    _print_kv("last_ts", _ms_to_iso(last_ts))
    _print_kv("rows", df.height)
    _print_kv("expected_bars", expected_bars)
    _print_kv("coverage_pct", f"{coverage*100:.4f}%")
    _print_kv("total_gaps", total_gaps)
    _print_kv("max_gap_bars", max_gap_bars)
else:
    first_ts = last_ts = expected_bars = total_gaps = max_gap_bars = 0
    coverage = 0.0
    print("   ℹ️  DataFrame vacío (sin filas).")

# ----(I) Persistencia a Parquet y manifest----
print("💾 Persistiendo Parquet curado ...")
df.write_parquet(PARQUET_PATH)
assert PARQUET_PATH.exists() and PARQUET_PATH.stat().st_size > 0, "No se pudo escribir el Parquet curado."

def _sha256_file(path: Path) -> str:
    h = hashlib.sha256()
    with path.open("rb") as f:
        for chunk in iter(lambda: f.read(1 << 20), b""):
            h.update(chunk)
    return h.hexdigest()

manifest = {
    "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
    "source": DATA_SOURCE,
    "symbol": SYMBOL,
    "timeframe": TF,
    "expected_interval_ms": INTERVAL_MS,
    "file": str(PARQUET_PATH),
    "sha256": _sha256_file(PARQUET_PATH),
    "rows": int(df.height),
    "first_ts": _ms_to_iso(first_ts) if df.height > 0 else None,
    "last_ts": _ms_to_iso(last_ts) if df.height > 0 else None,
    "expected_bars": int(expected_bars) if df.height > 0 else 0,
    "coverage_pct": coverage,
    "total_gaps": int(total_gaps),
    "max_gap_bars": int(max_gap_bars),
}

with MANIFEST_PATH.open("w", encoding="utf-8") as f:
    json.dump(manifest, f, indent=2, ensure_ascii=False)

print("🧾 Manifest escrito.")
_print_kv("manifest_path", MANIFEST_PATH)
_print_kv("sha256", manifest["sha256"])

# ----(J) Contratos de salida----
assert PARQUET_PATH.exists() and MANIFEST_PATH.exists(), "Faltan artefactos de salida."
print("✅ Celda 2 finalizada sin errores.\n")


📥 Leyendo configuración ...
   - symbol                      : BTCUSDT
   - timeframe                   : 15m
   - data_source                 : binance
   - expected_interval_ms        : 900000
   - start_date                  : 2021-01-01
   - end_date                    : 2099-12-31
   - start_ms                    : 1609459200000
   - end_ms                      : 1762264747969
   - now_ms                      : 1762264747969
   - parquet_out                 : C:\Quant\TWF\outputs\BTCUSDT\15m\data_curated\ohlc_BTCUSDT_15m.parquet
   - manifest_out                : C:\Quant\TWF\outputs\BTCUSDT\15m\logs\manifest_ohlc_BTCUSDT_15m.json
♻️  Archivo Parquet ya existe. Se validará y reutilizará si pasa controles...
🚫 Descarga omitida (reutilizando Parquet existente).
📖 Leyendo DataFrame desde Parquet existente ...
   ✓ Filas leídas: 169643, columnas: 11
🔎 Validando esquema y cobertura ...
🧮 Métricas:
   - first_ts                    : 2021-01-01T00:00:00+00:00
   - last_ts                

In [229]:
# Celda 3: [FEATURES → features_{SYMBOL}_{TF}.parquet]
# -----------------------------------------------------------------------------
# Responsabilidad única:
# - Leer config y OHLC curado.
# - Generar features: MA (ema/wma), ER (windows), slope.
# - Persistir Parquet de features con 'ts' como clave temporal.
# - Imprimir verificaciones exhaustivas (formas, NaNs, dtypes, fechas, muestras).
# -----------------------------------------------------------------------------

import os, json, math
from pathlib import Path
from datetime import timezone, datetime
import numpy as np
import polars as pl

# -------------------- Utils de impresión/validación --------------------
def _print_kv(k, v):
    print(f"   - {k:<30}: {v}")

def _assert_cols(df: pl.DataFrame, cols):
    missing = [c for c in cols if c not in df.columns]
    assert not missing, f"❌ Faltan columnas: {missing}"

def _epoch_ms(series_dt_tz: pl.Series) -> pl.Series:
    # Convierte Datetime TZ a epoch ms (Series Polars)
    return series_dt_tz.dt.epoch("ms")

# -------------------- A) Cargar config y paths -------------------------
print("📖 Leyendo configuración ...")
needed = ("TWF_SYMBOL","TWF_TF","TWF_START_DATE","TWF_END_DATE","TWF_SEED_BASE")
assert all(k in os.environ for k in needed), "Faltan TWF_*; ejecuta celdas 0 y 1 primero."
SYMBOL = os.environ["TWF_SYMBOL"]
TF     = os.environ["TWF_TF"]
PROJ_ROOT = Path(os.environ.get("TWF_BASE", Path.cwd())).resolve()
CONFIG_PATH = PROJ_ROOT / "outputs" / SYMBOL / TF / "logs" / f"config_{SYMBOL}_{TF}.json"
assert CONFIG_PATH.exists(), f"No existe el archivo de configuración: {CONFIG_PATH}"

with CONFIG_PATH.open("r", encoding="utf-8") as f:
    CFG = json.load(f)

PATHS = {k: Path(v) for k, v in CFG["paths"].items()}
OHLC_PARQ = PATHS["data_curated"] / f"ohlc_{SYMBOL}_{TF}.parquet"
FEAT_PARQ = PATHS["features"] / f"features_{SYMBOL}_{TF}.parquet"
_print_kv("OHLC_PARQ", OHLC_PARQ)
_print_kv("FEAT_PARQ", FEAT_PARQ)
assert OHLC_PARQ.exists(), f"No existe el OHLC curado: {OHLC_PARQ}"

# -------------------- B) Leer OHLC en Polars ---------------------------
print("📦 Cargando OHLC curado ...")
df = pl.read_parquet(OHLC_PARQ)
_print_kv("ohlc_rows", df.height)
_print_kv("ohlc_cols", df.width)
_assert_cols(df, ["ts","open","high","low","close","volume"])

# Fechas primeras/últimas en texto (100% Polars)
bounds = df.select(
    pl.col("ts").dt.epoch("ms").min().alias("first_ms"),
    pl.col("ts").dt.epoch("ms").max().alias("last_ms")
).row(0)
first_ms, last_ms = int(bounds[0]), int(bounds[1])
_print_kv("first_ts", datetime.fromtimestamp(first_ms/1000, tz=timezone.utc).isoformat(timespec="seconds"))
_print_kv("last_ts",  datetime.fromtimestamp(last_ms/1000, tz=timezone.utc).isoformat(timespec="seconds"))

print("🔒 Validaciones base de OHLC ...")
# Monotonicidad (no decreciente) y unicidad de ts (100% Polars)
dms = df.select(pl.col("ts").dt.epoch("ms").diff().alias("dms")).get_column("dms").drop_nulls()
is_sorted = bool((dms >= 0).all())
unique_ts = (df["ts"].n_unique() == df.height)
_print_kv("ts_sorted_nondec", is_sorted)
_print_kv("ts_unique", unique_ts)
assert is_sorted and unique_ts, "❌ 'ts' debe estar ordenado y sin duplicados."

# -------------------- C) Preparar arrays NumPy -------------------------
print("🧱 Preparando arrays NumPy ...")
ts_ms = _epoch_ms(df["ts"]).to_numpy()          # epoch ms
close = df["close"].to_numpy()                  # <- corregido: sin dtype posicional
open_  = df["open"].to_numpy()
high   = df["high"].to_numpy()
low    = df["low"].to_numpy()
n = close.size
_print_kv("n_samples", n)
assert n > 0, "❌ No hay datos para generar features."

# -------------------- D) Helpers NumPy para features -------------------
def ema_numpy(x: np.ndarray, span: int) -> np.ndarray:
    """
    EMA con warm-up clásico:
    - NaN en [0:span-1)
    - y[span-1] = media de x[0:span]
    - y[i] = alpha*x[i] + (1-alpha)*y[i-1] para i>=span
    """
    if span < 1:
        raise ValueError("span debe ser >=1")
    y = np.full_like(x, np.nan, dtype=np.float64)
    if x.size == 0:
        return y
    alpha = 2.0 / (span + 1.0)
    if x.size >= span:
        y[span-1] = np.nanmean(x[:span])
        for i in range(span, x.size):
            y[i] = alpha * x[i] + (1 - alpha) * y[i - 1]
    return y

def wma_numpy(x: np.ndarray, window: int) -> np.ndarray:
    """WMA lineal (pesos 1..window) con warm-up: NaN en [0:window-1)."""
    if window < 1:
        raise ValueError("window debe ser >=1")
    y = np.full_like(x, np.nan, dtype=np.float64)
    if x.size < window:
        return y
    w = np.arange(1, window + 1, dtype=np.float64)
    conv = np.convolve(x, w, mode="valid") / w.sum()
    y[window - 1:] = conv
    return y

def er_numpy(price: np.ndarray, window: int) -> np.ndarray:
    """Efficiency Ratio (Kaufman) con warm-up NaN en [0:window-1)."""
    if window < 2:
        raise ValueError("ER window debe ser >=2")
    y = np.full_like(price, np.nan, dtype=np.float64)
    N = price.size
    if N < window:
        return y
    # |P_t - P_{t-w+1}|
    net = np.abs(price[window - 1:] - price[:-window + 1])
    # ∑|ΔP| en ventana
    diff = np.abs(np.diff(price))
    csum = np.concatenate([[0.0], np.cumsum(diff)])
    vol = csum[(window - 1):] - csum[:-(window - 1)]
    er = np.divide(net, vol, out=np.zeros_like(net), where=vol > 0)
    y[window - 1:] = er
    return y

def slope_simple(price: np.ndarray, window: int) -> np.ndarray:
    """Pendiente simple: (P_t - P_{t-w+1}) / (w-1). Warm-up NaN."""
    if window < 2:
        raise ValueError("slope window debe ser >=2")
    y = np.full_like(price, np.nan, dtype=np.float64)
    if price.size >= window:
        y[window - 1:] = (price[window - 1:] - price[:-window + 1]) / (window - 1)
    return y

# -------------------- E) Construir features según config ----------------
print("🧮 Generando features ...")
ma_types    = CFG["features"]["ma"]["types"]
ma_windows  = CFG["features"]["ma"]["windows"]
er_windows  = CFG["features"]["er"]["windows"]
er_thrs     = CFG["features"]["er"]["thresholds"]  # referencia para celdas futuras
slope_win   = CFG["features"]["slope"]["window"]

_print_kv("ma_types", ma_types)
_print_kv("ma_windows", ma_windows)
_print_kv("er_windows", er_windows)
_print_kv("slope_window", slope_win)

feat_cols = {}
# MAs
for w in ma_windows:
    if "ema" in ma_types:
        print(f"   • EMA {w} ...")
        feat_cols[f"ema_{w}"] = ema_numpy(close, w)
    if "wma" in ma_types:
        print(f"   • WMA {w} ...")
        feat_cols[f"wma_{w}"] = wma_numpy(close, w)

# ER
for w in er_windows:
    print(f"   • ER {w} ...")
    feat_cols[f"er_{w}"] = er_numpy(close, w)

# Slope
print(f"   • SLOPE {slope_win} ...")
feat_cols[f"slope_{slope_win}"] = slope_simple(close, slope_win)

# -------------------- F) Armar DataFrame de features -------------------
print("🧱 Construyendo DataFrame de features ...")
feat_df = pl.DataFrame({"ts": pl.from_epoch(ts_ms/1000).dt.replace_time_zone("UTC")})
for name, arr in feat_cols.items():
    feat_df = feat_df.with_columns(pl.Series(name, arr.astype(np.float64)))

# Orden y unicidad por ts
feat_df = feat_df.sort("ts").unique(subset=["ts"], keep="last")

# -------------------- G) Chequeos de calidad ---------------------------
print("🔎 Chequeos de calidad de features ...")
_print_kv("features_rows", feat_df.height)
_print_kv("features_cols", feat_df.width)

# Rango temporal de features
b = feat_df.select(
    pl.col("ts").dt.epoch("ms").min().alias("first_ms"),
    pl.col("ts").dt.epoch("ms").max().alias("last_ms")
).row(0)
_print_kv("first_ts(features)", datetime.fromtimestamp(int(b[0])/1000, tz=timezone.utc).isoformat(timespec="seconds"))
_print_kv("last_ts(features)",  datetime.fromtimestamp(int(b[1])/1000, tz=timezone.utc).isoformat(timespec="seconds"))

# Dtypes resumen
dtype_map = {c: str(t) for c, t in feat_df.schema.items()}
_print_kv("dtype_ts", dtype_map.get("ts"))
numeric_cols = [c for c, t in feat_df.schema.items() if c != "ts"]  # todas las features son float64
_print_kv("num_cols_count", len(numeric_cols))
_print_kv("sample_num_cols", numeric_cols[:8])

# NaN por columna (solo floats)
if numeric_cols:
    nan_df = feat_df.select([pl.col(c).is_nan().sum().alias(c) for c in numeric_cols])
    nan_counts = {c: int(nan_df[0, c]) for c in numeric_cols}
    _print_kv("total_nan_cols>0", sum(1 for v in nan_counts.values() if v > 0))
    # Top 10 columnas con más NaN (por warm-up)
    top_nan = sorted(nan_counts.items(), key=lambda kv: kv[1], reverse=True)[:10]
    for k, v in top_nan:
        _print_kv(f"NaN in {k}", v)

# Muestras visuales (head/tail reducidas)
print("👀 Muestra HEAD:")
print(feat_df.head(3))
print("👀 Muestra TAIL:")
print(feat_df.tail(3))

# -------------------- H) Persistencia -------------------------------
print("💾 Guardando features ...")
feat_df.write_parquet(FEAT_PARQ)
assert FEAT_PARQ.exists() and FEAT_PARQ.stat().st_size > 0, "❌ No se pudo escribir features parquet."
_print_kv("features_parquet", FEAT_PARQ)
_print_kv("features_file_size(bytes)", FEAT_PARQ.stat().st_size)
_print_kv("features_file_size(MB)", round(FEAT_PARQ.stat().st_size/1024/1024, 3))

print("✅ Celda 3 finalizada sin errores.\n")


📖 Leyendo configuración ...
   - OHLC_PARQ                     : C:\Quant\TWF\outputs\BTCUSDT\15m\data_curated\ohlc_BTCUSDT_15m.parquet
   - FEAT_PARQ                     : C:\Quant\TWF\outputs\BTCUSDT\15m\features\features_BTCUSDT_15m.parquet
📦 Cargando OHLC curado ...
   - ohlc_rows                     : 169643
   - ohlc_cols                     : 11
   - first_ts                      : 2021-01-01T00:00:00+00:00
   - last_ts                       : 2025-11-03T20:00:00+00:00
🔒 Validaciones base de OHLC ...
   - ts_sorted_nondec              : True
   - ts_unique                     : True
🧱 Preparando arrays NumPy ...
   - n_samples                     : 169643
🧮 Generando features ...
   - ma_types                      : ['ema', 'wma']
   - ma_windows                    : [10, 20, 50]
   - er_windows                    : [10, 20, 50]
   - slope_window                  : 10
   • EMA 10 ...
   • WMA 10 ...
   • EMA 20 ...
   • WMA 20 ...
   • EMA 50 ...
   • WMA 50 ...
   • ER 10 ...
 

In [230]:
# Celda 4a: [JOIN & SANEO MÍNIMO → tmp/joined_{SYMBOL}_{TF}.parquet]
# -----------------------------------------------------------------------------
# Responsabilidad única:
# - Leer config, OHLC curado y features
# - Unir por 'ts' (inner join), mantener orden temporal
# - Validar: filas/columnas, monotonicidad, unicidad, NaNs en features, warm-up global
# - Persistir un Parquet temporal 'joined' para inspección de downstream
# -----------------------------------------------------------------------------

import os, json
from pathlib import Path
from datetime import datetime, timezone
import numpy as np
import polars as pl

def _print_kv(k, v):
    print(f"   - {k:<30}: {v}")

def _assert_cols(df: pl.DataFrame, cols):
    missing = [c for c in cols if c not in df.columns]
    assert not missing, f"❌ Faltan columnas: {missing}"

def _epoch_bounds_iso(df: pl.DataFrame, ts_col: str = "ts"):
    b = df.select(
        pl.col(ts_col).dt.epoch("ms").min().alias("first_ms"),
        pl.col(ts_col).dt.epoch("ms").max().alias("last_ms")
    ).row(0)
    first_ms, last_ms = int(b[0]), int(b[1])
    return (
        datetime.fromtimestamp(first_ms/1000, tz=timezone.utc).isoformat(timespec="seconds"),
        datetime.fromtimestamp(last_ms/1000, tz=timezone.utc).isoformat(timespec="seconds"),
    )

# -------------------- A) Cargar config y paths -------------------------
needed = ("TWF_SYMBOL","TWF_TF","TWF_START_DATE","TWF_END_DATE","TWF_SEED_BASE")
assert all(k in os.environ for k in needed), "Faltan TWF_*; ejecuta celdas 0 y 1 primero."
SYMBOL = os.environ["TWF_SYMBOL"]; TF = os.environ["TWF_TF"]
PROJ_ROOT   = Path(os.environ.get("TWF_BASE", Path.cwd())).resolve()
CONFIG_PATH = PROJ_ROOT / "outputs" / SYMBOL / TF / "logs" / f"config_{SYMBOL}_{TF}.json"
assert CONFIG_PATH.exists(), f"No existe el archivo de configuración: {CONFIG_PATH}"

print("📖 Leyendo configuración ...")
with CONFIG_PATH.open("r", encoding="utf-8") as f:
    CFG = json.load(f)

PATHS       = {k: Path(v) for k, v in CFG["paths"].items()}
OHLC_PARQ   = PATHS["data_curated"] / f"ohlc_{SYMBOL}_{TF}.parquet"
FEAT_PARQ   = PATHS["features"] / f"features_{SYMBOL}_{TF}.parquet"
JOINED_PARQ = PATHS["tmp"] / f"joined_{SYMBOL}_{TF}.parquet"

_print_kv("OHLC_PARQ", OHLC_PARQ)
_print_kv("FEAT_PARQ", FEAT_PARQ)
_print_kv("JOINED_PARQ", JOINED_PARQ)

assert OHLC_PARQ.exists(), f"No existe OHLC: {OHLC_PARQ}"
assert FEAT_PARQ.exists(), f"No existe features: {FEAT_PARQ}"

# -------------------- B) Leer DataFrames -------------------------------
print("📦 Cargando OHLC y FEATURES ...")
ohlc = pl.read_parquet(OHLC_PARQ)
feat = pl.read_parquet(FEAT_PARQ)

_print_kv("ohlc_rows", ohlc.height); _print_kv("ohlc_cols", ohlc.width)
_print_kv("feat_rows", feat.height); _print_kv("feat_cols", feat.width)
_assert_cols(ohlc, ["ts","open","high","low","close","volume"])
_assert_cols(feat, ["ts"])

# Rango temporal previo
f1, l1 = _epoch_bounds_iso(ohlc, "ts")
f2, l2 = _epoch_bounds_iso(feat, "ts")
_print_kv("ohlc.first_ts", f1); _print_kv("ohlc.last_ts", l1)
_print_kv("feat.first_ts", f2); _print_kv("feat.last_ts", l2)

# -------------------- C) Join por 'ts' (inner) ------------------------
print("🧩 Uniendo por 'ts' (inner join) ...")
joined = ohlc.join(feat, on="ts", how="inner", suffix="_f")

_print_kv("joined_rows", joined.height)
_print_kv("joined_cols", joined.width)
# Validaciones rápidas
dm = joined.select(pl.col("ts").dt.epoch("ms").diff().alias("dms")).get_column("dms").drop_nulls()
is_sorted = bool((dm >= 0).all())
unique_ts = (joined["ts"].n_unique() == joined.height)
_print_kv("ts_sorted_nondec", is_sorted); _print_kv("ts_unique", unique_ts)
assert is_sorted and unique_ts, "❌ El 'ts' del join debe quedar ordenado y único."

jf, jl = _epoch_bounds_iso(joined, "ts")
_print_kv("joined.first_ts", jf); _print_kv("joined.last_ts", jl)

# -------------------- D) Chequeo de NaNs en features -------------------
print("🔎 NaNs en columnas de features (warm-up esperado) ...")
feat_cols = [c for c in joined.columns if c not in ["ts","open","high","low","close","volume","close_time","quote_volume","trades","taker_base","taker_quote"]]
nan_sums = {}
if feat_cols:
    nan_df = joined.select([pl.col(c).is_nan().sum().alias(c) for c in feat_cols])
    nan_sums = {c: int(nan_df[0, c]) for c in feat_cols}
    total_nan_cols = sum(1 for v in nan_sums.values() if v > 0)
    _print_kv("feat_cols_count", len(feat_cols))
    _print_kv("feat_nan_cols>0", total_nan_cols)
    # Top 10
    top_nan = sorted(nan_sums.items(), key=lambda kv: kv[1], reverse=True)[:10]
    for k, v in top_nan:
        _print_kv(f"NaN in {k}", v)
else:
    print("   ℹ️  No se detectaron columnas de features (¿archivo vacío o columnas no esperadas?).")

# -------------------- E) Warm-up global (primer índice usable) ---------
print("🧊 Cálculo de warm-up global (primer índice donde TODAS las features son válidas) ...")
def _first_valid_index(series: pl.Series) -> int:
    # índice del primer valor válido (not null y not NaN); -1 si ninguno
    s = series
    ok = s.is_not_null() & (~s.is_nan())
    if ok.any():
        # np.argmax devuelve el primer True
        return int(np.argmax(ok.to_numpy()))
    return -1

if feat_cols:
    first_valid_idxs = {c: _first_valid_index(joined.get_column(c)) for c in feat_cols}
    # descartamos -1 (columnas totalmente NaN, no debería pasar aquí)
    filtered = [ix for ix in first_valid_idxs.values() if ix >= 0]
    warmup_global = max(filtered) if filtered else 0
    _print_kv("warmup_global_bars", warmup_global)
    if joined.height > 0:
        valid_ts = joined["ts"][warmup_global] if warmup_global < joined.height else None
        _print_kv("first_valid_ts_global", str(valid_ts) if valid_ts is not None else "None")
else:
    warmup_global = 0
    _print_kv("warmup_global_bars", warmup_global)

# -------------------- F) Muestras y dtypes -----------------------------
print("👀 Muestra HEAD (post-join):")
print(joined.head(3))
print("👀 Muestra TAIL (post-join):")
print(joined.tail(3))

dtype_map = {c: str(t) for c, t in joined.schema.items()}
_print_kv("dtype_ts", dtype_map.get("ts"))
_print_kv("numeric_cols_count", sum(1 for t in joined.schema.values() if "f64" in str(t)))

# -------------------- G) Persistencia ----------------------------------
print("💾 Guardando joined temporal ...")
JOINED_PARQ.parent.mkdir(parents=True, exist_ok=True)
joined.write_parquet(JOINED_PARQ)
assert JOINED_PARQ.exists() and JOINED_PARQ.stat().st_size > 0, "❌ No se pudo escribir joined parquet."
_print_kv("joined_parquet", JOINED_PARQ)
_print_kv("joined_size(MB)", round(JOINED_PARQ.stat().st_size/1024/1024, 3))

print("✅ Celda 4a finalizada sin errores.\n")


📖 Leyendo configuración ...
   - OHLC_PARQ                     : C:\Quant\TWF\outputs\BTCUSDT\15m\data_curated\ohlc_BTCUSDT_15m.parquet
   - FEAT_PARQ                     : C:\Quant\TWF\outputs\BTCUSDT\15m\features\features_BTCUSDT_15m.parquet
   - JOINED_PARQ                   : C:\Quant\TWF\outputs\BTCUSDT\15m\tmp\joined_BTCUSDT_15m.parquet
📦 Cargando OHLC y FEATURES ...
   - ohlc_rows                     : 169643
   - ohlc_cols                     : 11
   - feat_rows                     : 169643
   - feat_cols                     : 11
   - ohlc.first_ts                 : 2021-01-01T00:00:00+00:00
   - ohlc.last_ts                  : 2025-11-03T20:00:00+00:00
   - feat.first_ts                 : 2021-01-01T00:00:00+00:00
   - feat.last_ts                  : 2025-11-03T20:00:00+00:00
🧩 Uniendo por 'ts' (inner join) ...
   - joined_rows                   : 169643
   - joined_cols                   : 21
   - ts_sorted_nondec              : True
   - ts_unique                     : True


In [231]:
# Celda 4b: [DETECCIÓN DE ENTRADAS (ER ≥ UMBRAL) → tmp/entries_{SYMBOL}_{TF}.parquet]
# -----------------------------------------------------------------------------
# Responsabilidad única:
# - Leer config y el joined temporal de 4a.
# - Recalcular warm-up global (o usar el de 4a si lo hubiéramos persistido).
# - Para cada ventana ER y umbral, crear una columna booleana de "entrada".
# - Enmascarar entradas antes del warm-up global.
# - Persistir Parquet con ts, close y banderas de entrada por combo.
# - Imprimir verificaciones exhaustivas (conteos por combo, primeros/últimos ts, etc.).
# -----------------------------------------------------------------------------

import os, json
from pathlib import Path
from datetime import datetime, timezone
import numpy as np
import polars as pl

# -------------------- Utils --------------------
def _print_kv(k, v):
    print(f"   - {k:<30}: {v}")

def _assert_cols(df: pl.DataFrame, cols):
    missing = [c for c in cols if c not in df.columns]
    assert not missing, f"❌ Faltan columnas: {missing}"

def _epoch_bounds_iso(df: pl.DataFrame, ts_col: str = "ts"):
    b = df.select(
        pl.col(ts_col).dt.epoch("ms").min().alias("first_ms"),
        pl.col(ts_col).dt.epoch("ms").max().alias("last_ms")
    ).row(0)
    first_ms, last_ms = int(b[0]), int(b[1])
    return (
        datetime.fromtimestamp(first_ms/1000, tz=timezone.utc).isoformat(timespec="seconds"),
        datetime.fromtimestamp(last_ms/1000, tz=timezone.utc).isoformat(timespec="seconds"),
    )

def _first_valid_index(series: pl.Series) -> int:
    ok = series.is_not_null() & (~series.is_nan())
    if ok.any():
        return int(np.argmax(ok.to_numpy()))
    return -1

# -------------------- A) Cargar config y paths -------------------------
print("📖 Leyendo configuración ...")
need_env = ("TWF_SYMBOL","TWF_TF","TWF_START_DATE","TWF_END_DATE","TWF_SEED_BASE")
assert all(k in os.environ for k in need_env), "Faltan TWF_*; ejecuta celdas 0→4a primero."
SYMBOL = os.environ["TWF_SYMBOL"]; TF = os.environ["TWF_TF"]

PROJ_ROOT   = Path(os.environ.get("TWF_BASE", Path.cwd())).resolve()
CONFIG_PATH = PROJ_ROOT / "outputs" / SYMBOL / TF / "logs" / f"config_{SYMBOL}_{TF}.json"
assert CONFIG_PATH.exists(), f"No existe configuración: {CONFIG_PATH}"

with CONFIG_PATH.open("r", encoding="utf-8") as f:
    CFG = json.load(f)

PATHS        = {k: Path(v) for k, v in CFG["paths"].items()}
JOINED_PARQ  = PATHS["tmp"] / f"joined_{SYMBOL}_{TF}.parquet"
ENTRIES_PARQ = PATHS["tmp"] / f"entries_{SYMBOL}_{TF}.parquet"

_print_kv("JOINED_PARQ", JOINED_PARQ)
_print_kv("ENTRIES_PARQ", ENTRIES_PARQ)
assert JOINED_PARQ.exists(), f"No existe joined temporal: {JOINED_PARQ}"

# -------------------- B) Leer JOINED -------------------------
print("📦 Cargando joined temporal ...")
df = pl.read_parquet(JOINED_PARQ)
_print_kv("joined_rows", df.height)
_print_kv("joined_cols", df.width)
_assert_cols(df, ["ts","close"])

# Rango temporal para referencia
fj, lj = _epoch_bounds_iso(df, "ts")
_print_kv("joined.first_ts", fj); _print_kv("joined.last_ts", lj)

# -------------------- C) Recalcular warm-up global ---------------------
print("🧊 Recalculando warm-up global ...")
feature_cols = [c for c in df.columns if c.startswith(("ema_","wma_","er_","slope_"))]
_print_kv("feature_cols_count", len(feature_cols))
assert feature_cols, "❌ No se hallaron columnas de features en el joined."

first_valid_idxs = {c: _first_valid_index(df.get_column(c)) for c in feature_cols}
valid_ixs = [ix for ix in first_valid_idxs.values() if ix >= 0]
warmup_global = max(valid_ixs) if valid_ixs else 0
_print_kv("warmup_global_bars", warmup_global)
wts = df["ts"][warmup_global] if warmup_global < df.height else None
_print_kv("first_valid_ts_global", str(wts) if wts is not None else "None")

# -------------------- D) Config: ER windows y thresholds ----------------
print("🧮 Preparando combos de ER (ventana/umbral) ...")
er_windows = CFG["features"]["er"]["windows"]
er_thrs    = CFG["features"]["er"]["thresholds"]
_print_kv("er_windows", er_windows)
_print_kv("er_thresholds", er_thrs)

# Validar que existan columnas er_*
missing_er_cols = [f"er_{w}" for w in er_windows if f"er_{w}" not in df.columns]
assert not missing_er_cols, f"❌ Faltan columnas ER requeridas en joined: {missing_er_cols}"

# -------------------- E) Construcción de flags de entrada ----------------
print("🚦 Generando banderas de entrada (ER ≥ umbral) ...")
mask_after_warmup = pl.arange(0, df.height).cast(pl.Int64) >= warmup_global

entry_cols = []
work_df = df.select(["ts","close"] + [f"er_{w}" for w in er_windows])

for w in er_windows:
    er_col = f"er_{w}"
    for thr in er_thrs:
        # Normalizar nombre de columna (ej: 0.3 -> 030)
        thr_tag = str(thr).replace(".","p").replace(",","p")
        col_name = f"entry_er{w}_ge_{thr_tag}"
        entry_cols.append(col_name)
        # entrada = (er_w >= thr) & no NaN & despues del warmup
        work_df = work_df.with_columns(
            pl.when(
                (pl.col(er_col).is_not_null()) & (~pl.col(er_col).is_nan()) & (pl.col(er_col) >= pl.lit(thr)) & mask_after_warmup
            ).then(pl.lit(True)).otherwise(pl.lit(False)).alias(col_name)
        )
        print(f"   • {col_name} creado.")

# -------------------- F) Verificaciones de conteos ----------------------
print("🔎 Conteos de entradas por combo ...")
counts = {}
for c in entry_cols:
    counts[c] = int(work_df[c].sum())
# Top combos por número de señales
top = sorted(counts.items(), key=lambda kv: kv[1], reverse=True)[:10]
_print_kv("entry_cols_total", len(entry_cols))
for k, v in top:
    _print_kv(f"signals in {k}", v)

# Conteo por fila (cuántos combos disparan simultáneo)
work_df = work_df.with_columns(
    pl.sum_horizontal([pl.col(c).cast(pl.Int8) for c in entry_cols]).alias("entry_count_row")
)
_print_kv("rows_with_any_entry", int((work_df["entry_count_row"] > 0).sum()))
_print_kv("max_entries_same_row", int(work_df["entry_count_row"].max()))

# Rango temporal donde hubo al menos 1 entrada
if int((work_df["entry_count_row"] > 0).sum()) > 0:
    sub = work_df.filter(pl.col("entry_count_row") > 0)
    f3, l3 = _epoch_bounds_iso(sub, "ts")
    _print_kv("first_ts_with_entry", f3)
    _print_kv("last_ts_with_entry", l3)
else:
    print("   ℹ️  No hubo ninguna entrada con los umbrales actuales.")

# Muestras
print("👀 Muestra HEAD (entries):")
print(work_df.head(3).select(["ts","close","entry_count_row"] + entry_cols[:min(5, len(entry_cols))]))
print("👀 Muestra TAIL (entries):")
print(work_df.tail(3).select(["ts","close","entry_count_row"] + entry_cols[:min(5, len(entry_cols))]))

# -------------------- G) Persistencia ----------------------------------
print("💾 Guardando entries ...")
ENTRIES_PARQ.parent.mkdir(parents=True, exist_ok=True)
work_df.write_parquet(ENTRIES_PARQ)
assert ENTRIES_PARQ.exists() and ENTRIES_PARQ.stat().st_size > 0, "❌ No se pudo escribir entries parquet."
_print_kv("entries_parquet", ENTRIES_PARQ)
_print_kv("entries_size(MB)", round(ENTRIES_PARQ.stat().st_size/1024/1024, 3))

# Meta opcional: guardar lista de columnas de entrada
META_PATH = PATHS["logs"] / f"entries_meta_{SYMBOL}_{TF}.json"
meta = {
    "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
    "symbol": SYMBOL,
    "timeframe": TF,
    "er_windows": er_windows,
    "er_thresholds": er_thrs,
    "entry_columns": entry_cols,
    "warmup_global_bars": int(warmup_global),
    "first_valid_ts_global": str(wts) if wts is not None else None
}
with META_PATH.open("w", encoding="utf-8") as f:
    json.dump(meta, f, indent=2, ensure_ascii=False)
_print_kv("entries_meta", META_PATH)

print("✅ Celda 4b finalizada sin errores.\n")


📖 Leyendo configuración ...
   - JOINED_PARQ                   : C:\Quant\TWF\outputs\BTCUSDT\15m\tmp\joined_BTCUSDT_15m.parquet
   - ENTRIES_PARQ                  : C:\Quant\TWF\outputs\BTCUSDT\15m\tmp\entries_BTCUSDT_15m.parquet
📦 Cargando joined temporal ...
   - joined_rows                   : 169643
   - joined_cols                   : 21
   - joined.first_ts               : 2021-01-01T00:00:00+00:00
   - joined.last_ts                : 2025-11-03T20:00:00+00:00
🧊 Recalculando warm-up global ...
   - feature_cols_count            : 10
   - warmup_global_bars            : 49
   - first_valid_ts_global         : 2021-01-01 12:15:00+00:00
🧮 Preparando combos de ER (ventana/umbral) ...
   - er_windows                    : [10, 20, 50]
   - er_thresholds                 : [0.2, 0.3, 0.4]
🚦 Generando banderas de entrada (ER ≥ umbral) ...
   • entry_er10_ge_0p2 creado.
   • entry_er10_ge_0p3 creado.
   • entry_er10_ge_0p4 creado.
   • entry_er20_ge_0p2 creado.
   • entry_er20_ge_0p3 crea

In [232]:
# Celda 4c (fix definitivo y métricas estables): [TRIPLE-BARRIER / FORWARD LABELING → forward/forward_{SYMBOL}_{TF}.parquet]
# -------------------------------------------------------------------------------------
# - Estudio estadístico (NO backtesting): etiquetado triple-barrier para medir ventaja.
# - Lee config, joined (OHLC+features) y entries (señales).
# - Calcula ATR% como base de volatilidad.
# - Etiqueta cada señal con Triple-Barrier (1=TP, -1=SL, 0=timeout).
# - Persiste un Parquet por-señal con métricas forward.
# - Compatibilidad Polars "legacy": sin Series/DataFrame.take(); se usa epoch(us)+NumPy.
# - FIX: usar DataFrame.group_by(...).agg(...) + Expr.sum() + pl.len().
# -------------------------------------------------------------------------------------

import os, json, time
from pathlib import Path
from datetime import datetime, timezone
import numpy as np
import polars as pl

TS_DTYPE = pl.Datetime(time_unit="us", time_zone="UTC")

# -------------------- Utils --------------------
def _print_kv(k, v):
    print(f"   - {k:<30}: {v}")

def _assert_cols(df: pl.DataFrame, cols):
    missing = [c for c in cols if c not in df.columns]
    assert not missing, f"❌ Faltan columnas: {missing}"

def _epoch_bounds_iso(df: pl.DataFrame, ts_col: str = "ts"):
    b = df.select(
        pl.col(ts_col).dt.epoch("ms").min().alias("first_ms"),
        pl.col(ts_col).dt.epoch("ms").max().alias("last_ms")
    ).row(0)
    first_ms, last_ms = int(b[0]), int(b[1])
    return (
        datetime.fromtimestamp(first_ms/1000, tz=timezone.utc).isoformat(timespec="seconds"),
        datetime.fromtimestamp(last_ms/1000, tz=timezone.utc).isoformat(timespec="seconds"),
    )

def _first_valid_index(series: pl.Series) -> int:
    ok = series.is_not_null() & (~series.is_nan())
    if ok.any():
        return int(np.argmax(ok.to_numpy()))
    return -1

# -------------------- A) Cargar config y paths -------------------------
print("📖 Leyendo configuración ...")
need_env = ("TWF_SYMBOL","TWF_TF","TWF_START_DATE","TWF_END_DATE","TWF_SEED_BASE")
assert all(k in os.environ for k in need_env), "Faltan TWF_*; ejecuta celdas 0→4b primero."
SYMBOL = os.environ["TWF_SYMBOL"]; TF = os.environ["TWF_TF"]

PROJ_ROOT   = Path(os.environ.get("TWF_BASE", Path.cwd())).resolve()
CONFIG_PATH = PROJ_ROOT / "outputs" / SYMBOL / TF / "logs" / f"config_{SYMBOL}_{TF}.json"
assert CONFIG_PATH.exists(), f"No existe configuración: {CONFIG_PATH}"

with CONFIG_PATH.open("r", encoding="utf-8") as f:
    CFG = json.load(f)

PATHS         = {k: Path(v) for k, v in CFG["paths"].items()}
JOINED_PARQ   = PATHS["tmp"] / f"joined_{SYMBOL}_{TF}.parquet"
ENTRIES_PARQ  = PATHS["tmp"] / f"entries_{SYMBOL}_{TF}.parquet"
FORWARD_PARQ  = PATHS["forward"] / f"forward_{SYMBOL}_{TF}.parquet"
META_PATH     = PATHS["logs"]    / f"forward_meta_{SYMBOL}_{TF}.json"

_print_kv("JOINED_PARQ", JOINED_PARQ)
_print_kv("ENTRIES_PARQ", ENTRIES_PARQ)
_print_kv("FORWARD_PARQ", FORWARD_PARQ)
assert JOINED_PARQ.exists(), f"No existe joined temporal: {JOINED_PARQ}"
assert ENTRIES_PARQ.exists(), f"No existe entries: {ENTRIES_PARQ}"

# -------------------- B) Leer DataFrames base --------------------------
print("📦 Cargando joined y entries ...")
joined  = pl.read_parquet(JOINED_PARQ)
entries = pl.read_parquet(ENTRIES_PARQ)

_assert_cols(joined, ["ts","open","high","low","close"])
_assert_cols(entries, ["ts","close"])

# Sincronizar por ts (seguridad)
print("🔗 Alineando por 'ts' ...")
base = joined.select(["ts","open","high","low","close"])
entries = entries.join(base, on="ts", how="inner", suffix="_j")
# Solo 'close' colisiona → 'close_j'
_assert_cols(entries, ["ts","close","open","high","low","close_j"])

fj, lj = _epoch_bounds_iso(entries, "ts")
_print_kv("entries.first_ts", fj); _print_kv("entries.last_ts", lj)
_print_kv("entries_rows", entries.height)

# -------------------- C) Parámetros Triple-Barrier ---------------------
print("⚙️ Parámetros Triple-Barrier ...")
TB_CFG = CFG["labeling"]["triple_barrier"]
HORIZON_BARS = int(TB_CFG.get("horizon_bars", 48))
UP_MULT      = float(TB_CFG.get("up_mult", 2.0))
DN_MULT      = float(TB_CFG.get("dn_mult", 1.0))
TIMEOUT_BARS = int(TB_CFG.get("timeout_bars", HORIZON_BARS))
ATR_WINDOW   = int(TB_CFG.get("atr_window", 14))   # local

_print_kv("horizon_bars", HORIZON_BARS)
_print_kv("timeout_bars", TIMEOUT_BARS)
_print_kv("up_mult", UP_MULT)
_print_kv("dn_mult", DN_MULT)
_print_kv("atr_window(local)", ATR_WINDOW)

# -------------------- D) Calcular ATR% (volatilidad base) --------------
print("📈 Calculando ATR% ...")
hi = entries["high"].to_numpy()
lo = entries["low"].to_numpy()
cl = entries["close_j"].to_numpy()

prev_close = np.roll(cl, 1); prev_close[0] = cl[0]
tr = np.maximum.reduce([hi - lo, np.abs(hi - prev_close), np.abs(lo - prev_close)])

atr = np.full_like(tr, np.nan, dtype=np.float64)
if tr.size >= ATR_WINDOW:
    csum = np.concatenate([[0.0], np.cumsum(tr, dtype=np.float64)])
    atr_win = (csum[ATR_WINDOW:] - csum[:-ATR_WINDOW]) / ATR_WINDOW
    atr[ATR_WINDOW-1:] = atr_win

vol_pct = np.divide(atr, cl, out=np.full_like(atr, np.nan), where=cl != 0.0)

entries = entries.with_columns(
    pl.Series("atr", atr),
    pl.Series("vol_pct", vol_pct)
)

first_valid_atr = _first_valid_index(entries["atr"])
_print_kv("first_valid_atr_idx", first_valid_atr)
if 0 <= first_valid_atr < entries.height:
    _print_kv("first_valid_atr_ts", str(entries["ts"][first_valid_atr]))

# -------------------- E) Arrays, epoch(us) y columnas de señales -------
print("🧱 Preparando arrays para escaneo forward ...")
close_arr = entries["close_j"].to_numpy()
high_arr  = entries["high"].to_numpy()
low_arr   = entries["low"].to_numpy()
vol_arr   = entries["vol_pct"].to_numpy()

# Epoch(us) como NumPy para resolver ts/event_ts por índices sin .take()
ts_us_arr = entries.select(pl.col("ts").dt.epoch("us")).to_series().to_numpy()

# Señales: solo columnas entry_er*
entry_cols = [c for c in entries.columns if c.startswith("entry_er")]
_print_kv("entry_cols_count", len(entry_cols))
_print_kv("entry_cols_sample", entry_cols[:8])
assert entry_cols, "❌ No se hallaron columnas entry_er* en entries."

# -------------------- F) Función de etiquetado por señales -------------
def label_signals_for_column(col_name: str) -> pl.DataFrame:
    """
    Estudio estadístico: recorre índices True de `col_name`, calcula barreras up/dn
    con base en ATR%, y etiqueta (TP/SL/timeout) dentro del horizonte.
    """
    mask = entries[col_name].to_numpy().astype(bool)
    idxs = np.where(mask)[0]
    m = idxs.size
    if m == 0:
        return pl.DataFrame({
            "ts": pl.Series([], dtype=TS_DTYPE),
            "entry_col": pl.Series([], dtype=pl.Utf8),
            "entry_idx": pl.Series([], dtype=pl.Int64),
            "label": pl.Series([], dtype=pl.Int8),
            "bars_to_event": pl.Series([], dtype=pl.Int32),
            "event_ts": pl.Series([], dtype=TS_DTYPE),
            "entry_price": pl.Series([], dtype=pl.Float64),
            "event_price": pl.Series([], dtype=pl.Float64),
            "ret_event": pl.Series([], dtype=pl.Float64),
            "up_price": pl.Series([], dtype=pl.Float64),
            "dn_price": pl.Series([], dtype=pl.Float64),
        })

    out_entry_idx, out_event_idx = [], []
    out_label, out_bars = [], []
    out_entry_px, out_evt_px, out_ret = [], [], []
    out_up, out_dn = [], []
    out_col = []

    N = entries.height
    H = HORIZON_BARS
    T = TIMEOUT_BARS

    t0 = time.time(); last_report = t0

    for k, i in enumerate(idxs):
        start = i + 1
        if start >= N:
            continue
        end_h = min(i + H, N - 1)
        end_t = min(i + T, N - 1)

        entry_px = close_arr[i]
        v = vol_arr[i]
        if not (np.isfinite(entry_px) and np.isfinite(v)):
            continue

        up_px = entry_px * (1.0 + UP_MULT * v)
        dn_px = entry_px * (1.0 - DN_MULT * v)

        hi_seg = high_arr[start:end_h+1]
        lo_seg = low_arr[start:end_h+1]

        hit_up = np.where(hi_seg >= up_px)[0]
        hit_dn = np.where(lo_seg <= dn_px)[0]

        if hit_up.size == 0 and hit_dn.size == 0:
            evt_idx = min(i + T, end_t)
            label = 0
            evt_px = close_arr[evt_idx]
        else:
            fu = hit_up[0] if hit_up.size > 0 else None
            fd = hit_dn[0] if hit_dn.size > 0 else None
            if fu is not None and fd is not None:
                if fu < fd:
                    label = 1; evt_idx = start + fu; evt_px = up_px
                elif fd < fu:
                    label = -1; evt_idx = start + fd; evt_px = dn_px
                else:
                    label = 1; evt_idx = start + fu; evt_px = up_px
            elif fu is not None:
                label = 1; evt_idx = start + fu; evt_px = up_px
            else:
                label = -1; evt_idx = start + fd; evt_px = dn_px

        ret = (evt_px / entry_px) - 1.0 if (np.isfinite(evt_px) and entry_px != 0) else np.nan

        out_entry_idx.append(int(i))
        out_event_idx.append(int(evt_idx))
        out_label.append(int(label))
        out_bars.append(int(evt_idx - i))
        out_entry_px.append(float(entry_px))
        out_evt_px.append(float(evt_px))
        out_ret.append(float(ret))
        out_up.append(float(up_px))
        out_dn.append(float(dn_px))
        out_col.append(col_name)

        if k % 10000 == 0:
            now = time.time()
            if now - last_report > 0.5:
                print(f"   • {col_name}: procesadas {k}/{m} señales ...")
                last_report = now

    # ---- Construcción ts/event_ts desde epoch(us) por índices ----
    entry_idx_np = np.array(out_entry_idx, dtype=np.int64)
    event_idx_np = np.array(out_event_idx, dtype=np.int64)

    ts_us       = ts_us_arr[entry_idx_np]
    event_ts_us = ts_us_arr[event_idx_np]

    s_ts       = pl.Series("ts", ts_us, dtype=pl.Int64).cast(TS_DTYPE)
    s_event_ts = pl.Series("event_ts", event_ts_us, dtype=pl.Int64).cast(TS_DTYPE)

    df_out = pl.DataFrame({
        "ts": s_ts,
        "entry_col": pl.Series("entry_col", out_col, dtype=pl.Utf8),
        "entry_idx": pl.Series("entry_idx", out_entry_idx, dtype=pl.Int64),
        "label": pl.Series("label", out_label, dtype=pl.Int8),
        "bars_to_event": pl.Series("bars_to_event", out_bars, dtype=pl.Int32),
        "event_ts": s_event_ts,
        "entry_price": pl.Series("entry_price", out_entry_px, dtype=pl.Float64),
        "event_price": pl.Series("event_price", out_evt_px, dtype=pl.Float64),
        "ret_event": pl.Series("ret_event", out_ret, dtype=pl.Float64),
        "up_price": pl.Series("up_price", out_up, dtype=pl.Float64),
        "dn_price": pl.Series("dn_price", out_dn, dtype=pl.Float64),
    })
    return df_out

# -------------------- G) Ejecutar etiquetado para todas las columnas ----
print("🏷️ Etiquetando señales (todas las columnas entry_er*) ...")
per_col = []
for col in entry_cols:
    print(f"→ Procesando {col} ...")
    df_lab = label_signals_for_column(col)
    _print_kv("signals_labeled", df_lab.height)
    per_col.append(df_lab)

# Concatenar y ordenar por ts
if per_col:
    forward_df = pl.concat(per_col, how="vertical_relaxed").sort(["ts","entry_col"])
else:
    forward_df = pl.DataFrame({
        "ts": pl.Series([], dtype=TS_DTYPE),
        "entry_col": pl.Series([], dtype=pl.Utf8),
        "entry_idx": pl.Series([], dtype=pl.Int64),
        "label": pl.Series([], dtype=pl.Int8),
        "bars_to_event": pl.Series([], dtype=pl.Int32),
        "event_ts": pl.Series([], dtype=TS_DTYPE),
        "entry_price": pl.Series([], dtype=pl.Float64),
        "event_price": pl.Series([], dtype=pl.Float64),
        "ret_event": pl.Series([], dtype=pl.Float64),
        "up_price": pl.Series([], dtype=pl.Float64),
        "dn_price": pl.Series([], dtype=pl.Float64),
    })

_print_kv("forward_rows_total", forward_df.height)
_print_kv("forward_cols", forward_df.width)

# -------------------- H) Resumen estadístico por columna ----------------
print("📊 Resumen de etiquetas por columna ...")
if forward_df.height > 0:
    group = (
        forward_df
        .group_by("entry_col")
        .agg(
            pl.len().alias("signals"),
            (pl.col("label") == 1).cast(pl.Int32).sum().alias("tp"),
            (pl.col("label") == -1).cast(pl.Int32).sum().alias("sl"),
            (pl.col("label") == 0).cast(pl.Int32).sum().alias("timeout"),
            pl.col("ret_event").mean().alias("ret_mean"),
            pl.col("ret_event").median().alias("ret_median"),
            pl.col("bars_to_event").mean().alias("bars_mean"),
            pl.col("bars_to_event").median().alias("bars_median"),
        )
        .sort("entry_col")
    )
    print(group)
else:
    print("   ℹ️  No hay señales etiquetadas (¿no hubo entries válidas o ATR muy temprano?).")

# -------------------- I) Persistencia -----------------------------------
print("💾 Guardando forward ...")
FORWARD_PARQ.parent.mkdir(parents=True, exist_ok=True)
forward_df.write_parquet(FORWARD_PARQ)
assert FORWARD_PARQ.exists() and FORWARD_PARQ.stat().st_size > 0, "❌ No se pudo escribir forward parquet."
_print_kv("forward_parquet", FORWARD_PARQ)
_print_kv("forward_size(MB)", round(FORWARD_PARQ.stat().st_size/1024/1024, 3))

# Meta (solo estadística descriptiva del etiquetado)
meta = {
    "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
    "symbol": SYMBOL,
    "timeframe": TF,
    "horizon_bars": HORIZON_BARS,
    "timeout_bars": TIMEOUT_BARS,
    "up_mult": UP_MULT,
    "dn_mult": DN_MULT,
    "atr_window_local": ATR_WINDOW,
    "entry_cols": entry_cols,
}
with META_PATH.open("w", encoding="utf-8") as f:
    json.dump(meta, f, indent=2, ensure_ascii=False)
_print_kv("forward_meta", META_PATH)

print("✅ Celda 4c finalizada sin errores.\n")
 

📖 Leyendo configuración ...
   - JOINED_PARQ                   : C:\Quant\TWF\outputs\BTCUSDT\15m\tmp\joined_BTCUSDT_15m.parquet
   - ENTRIES_PARQ                  : C:\Quant\TWF\outputs\BTCUSDT\15m\tmp\entries_BTCUSDT_15m.parquet
   - FORWARD_PARQ                  : C:\Quant\TWF\outputs\BTCUSDT\15m\forward\forward_BTCUSDT_15m.parquet
📦 Cargando joined y entries ...
🔗 Alineando por 'ts' ...
   - entries.first_ts              : 2021-01-01T00:00:00+00:00
   - entries.last_ts               : 2025-11-03T20:00:00+00:00
   - entries_rows                  : 169643
⚙️ Parámetros Triple-Barrier ...
   - horizon_bars                  : 48
   - timeout_bars                  : 48
   - up_mult                       : 2.0
   - dn_mult                       : 1.0
   - atr_window(local)             : 14
📈 Calculando ATR% ...
   - first_valid_atr_idx           : 13
   - first_valid_atr_ts            : 2021-01-01 03:15:00+00:00
🧱 Preparando arrays para escaneo forward ...
   - entry_cols_count          

In [233]:
# Celda 4d (dedup v2): [DEDUP/QUALITY GATE + META → forward/forward_{SYMBOL}_{TF}.parquet + logs/forward_meta_*.json]
# ---------------------------------------------------------------------------------------------------------------
# - Entrada: forward_{SYMBOL}_{TF}.parquet (producido por 04c).
# - Objetivo: asegurar unicidad por (ts, entry_col), remover duplicados exactos y escribir meta con dedup_completed.
# - Salida: forward_{SYMBOL}_{TF}.parquet (sobrescrito limpio) + logs/forward_meta_{SYMBOL}_{TF}.json
# - Gates: si el archivo no existe → assert; imprime conteos before/after.
# ---------------------------------------------------------------------------------------------------------------

import os, json, hashlib
from pathlib import Path
from datetime import datetime, timezone
import polars as pl

def _print_kv(k, v): print(f"   - {k:<30}: {v}")
def _hash_file(p: Path, algo="sha256") -> str:
    h = hashlib.new(algo)
    with p.open("rb") as f:
        for chunk in iter(lambda: f.read(1 << 20), b""):
            h.update(chunk)
    return h.hexdigest()

print("📖 Leyendo configuración ...")
need_env = ("TWF_SYMBOL","TWF_TF","TWF_START_DATE","TWF_END_DATE","TWF_SEED_BASE")
assert all(k in os.environ for k in need_env), "Faltan TWF_*; ejecuta celdas previas."
SYMBOL = os.environ["TWF_SYMBOL"]; TF = os.environ["TWF_TF"]

PROJ_ROOT   = Path(os.environ.get("TWF_BASE", Path.cwd())).resolve()
CONFIG_PATH = PROJ_ROOT / "outputs" / SYMBOL / TF / "logs" / f"config_{SYMBOL}_{TF}.json"
assert CONFIG_PATH.exists(), f"No existe configuración: {CONFIG_PATH}"
CFG = json.loads(CONFIG_PATH.read_text(encoding="utf-8"))

PATHS         = {k: Path(v) for k, v in CFG["paths"].items()}
FORWARD_PARQ  = PATHS["forward"] / f"forward_{SYMBOL}_{TF}.parquet"
FORWARD_META  = PATHS["logs"]    / f"forward_meta_{SYMBOL}_{TF}.json"
assert FORWARD_PARQ.exists(), f"No existe forward: {FORWARD_PARQ}"

print("📦 Cargando forward (pre-dedup) ...")
fwd = pl.read_parquet(FORWARD_PARQ)
for c in ["ts","entry_col","label","ret_event","bars_to_event"]:
    assert c in fwd.columns, f"❌ Falta columna en forward: {c}"
_print_kv("rows_before", fwd.height)

# 1) Dedup exacto por columnas completas (si hay) y por clave (ts, entry_col)
cols_key = [c for c in ["ts","entry_col"] if c in fwd.columns]
fwd = fwd.unique(keep="first")  # dedup total row
if cols_key:
    fwd = fwd.unique(subset=cols_key, keep="first")

_print_kv("rows_after", fwd.height)

# 2) Persistimos limpio
print("💾 Sobrescribiendo forward limpio ...")
fwd.write_parquet(FORWARD_PARQ)
file_hash = _hash_file(FORWARD_PARQ)

# 3) META con bandera obligatoria para Celda 5
meta = {
    "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
    "symbol": SYMBOL,
    "timeframe": TF,
    "rows": int(fwd.height),
    "dedup_completed": True,
    "dedup_method": "unique((ts,entry_col), keep=first)",
    "forward_sha256": file_hash,
    "version": "4d_v2"
}
FORWARD_META.parent.mkdir(parents=True, exist_ok=True)
FORWARD_META.write_text(json.dumps(meta, indent=2, ensure_ascii=False), encoding="utf-8")
_print_kv("FORWARD_META", FORWARD_META)

print("✅ Celda 4d finalizada sin errores.\n")


📖 Leyendo configuración ...
📦 Cargando forward (pre-dedup) ...
   - rows_before                   : 447948
   - rows_after                    : 447948
💾 Sobrescribiendo forward limpio ...
   - FORWARD_META                  : C:\Quant\TWF\outputs\BTCUSDT\15m\logs\forward_meta_BTCUSDT_15m.json
✅ Celda 4d finalizada sin errores.



In [234]:
# Celda 5 (fix v3 c/ gate dedup): [KPIs ESTADÍSTICOS POR COMBO → reports/kpi_*.parquet/json]
# ------------------------------------------------------------------------------------------------------
# - Igual a tu 5 actual, pero añadimos el gate: exige logs/forward_meta_* con "dedup_completed": true.
# - Sin cambios en expresiones; mismos KPIs; compat legacy (**0.5, sin pl.sqrt).
# ------------------------------------------------------------------------------------------------------

import os, json
from pathlib import Path
from datetime import datetime, timezone
import polars as pl

def _print_kv(k, v): 
    print(f"   - {k:<30}: {v}")

def _safe_pos(expr: pl.Expr, floor: float) -> pl.Expr:
    return pl.when(expr < pl.lit(floor)).then(pl.lit(floor)).otherwise(expr)

def _clamp01(expr: pl.Expr) -> pl.Expr:
    return pl.when(expr < 0.0).then(0.0).otherwise(
           pl.when(expr > 1.0).then(1.0).otherwise(expr))

print("📖 Leyendo configuración ...")
need_env = ("TWF_SYMBOL","TWF_TF","TWF_START_DATE","TWF_END_DATE","TWF_SEED_BASE")
assert all(k in os.environ for k in need_env), "Faltan TWF_*; ejecuta celdas previas."
SYMBOL = os.environ["TWF_SYMBOL"]; TF = os.environ["TWF_TF"]

PROJ_ROOT   = Path(os.environ.get("TWF_BASE", Path.cwd())).resolve()
CONFIG_PATH = PROJ_ROOT / "outputs" / SYMBOL / TF / "logs" / f"config_{SYMBOL}_{TF}.json"
CFG = json.loads(CONFIG_PATH.read_text(encoding="utf-8"))
PATHS        = {k: Path(v) for k, v in CFG["paths"].items()}
FORWARD_PARQ = PATHS["forward"] / f"forward_{SYMBOL}_{TF}.parquet"
FORWARD_META = PATHS["logs"]    / f"forward_meta_{SYMBOL}_{TF}.json"
REPORTS_DIR  = PATHS["reports"]; REPORTS_DIR.mkdir(parents=True, exist_ok=True)

# Gate dedup
assert FORWARD_META.exists(), f"❌ Falta META de dedup: {FORWARD_META}. Ejecuta Celda 4d."
meta = json.loads(FORWARD_META.read_text(encoding="utf-8"))
assert meta.get("dedup_completed", False) is True, "❌ forward no está deduplicado (4d)."

KPI_PARQ      = REPORTS_DIR / f"kpi_{SYMBOL}_{TF}.parquet"
KPI_JSON      = REPORTS_DIR / f"kpi_{SYMBOL}_{TF}.json"
KPI_YEAR_PARQ = REPORTS_DIR / f"kpi_by_year_{SYMBOL}_{TF}.parquet"

min_trades_cfg = int((CFG.get("reporting", {}) or {}).get("min_trades", 1000))
_print_kv("FORWARD_PARQ", FORWARD_PARQ)
_print_kv("KPI_PARQ", KPI_PARQ)
_print_kv("KPI_YEAR_PARQ", KPI_YEAR_PARQ)
_print_kv("min_trades_cfg", min_trades_cfg)
assert FORWARD_PARQ.exists(), f"No existe forward: {FORWARD_PARQ}"

print("📦 Cargando forward (por-señal) ...")
fwd = pl.read_parquet(FORWARD_PARQ)
_print_kv("forward_rows", fwd.height); _print_kv("forward_cols", fwd.width)
for col in ["entry_col","label","ret_event","bars_to_event","ts"]:
    assert col in fwd.columns, f"❌ Falta columna en forward: {col}"

print("🧮 Agregando KPIs por 'entry_col' ...")
tp_expr  = (pl.col("label") == 1).cast(pl.Int32)
sl_expr  = (pl.col("label") == -1).cast(pl.Int32)
to_expr  = (pl.col("label") == 0).cast(pl.Int32)

sum_tp_ret = pl.when(pl.col("label")==1).then(pl.col("ret_event")).otherwise(0.0).sum()
sum_sl_abs = pl.when(pl.col("label")==-1).then(-pl.col("ret_event")).otherwise(0.0).sum()

quantiles = [
    pl.col("ret_event").quantile(0.05).alias("ret_q05"),
    pl.col("ret_event").quantile(0.25).alias("ret_q25"),
    pl.col("ret_event").quantile(0.50).alias("ret_q50"),
    pl.col("ret_event").quantile(0.75).alias("ret_q75"),
    pl.col("ret_event").quantile(0.95).alias("ret_q95"),
]

kpi_base = (
    fwd.group_by("entry_col").agg(
        pl.len().alias("n_signals"),
        tp_expr.sum().alias("tp"),
        sl_expr.sum().alias("sl"),
        to_expr.sum().alias("timeout"),
        pl.col("ret_event").mean().alias("ret_mean"),
        pl.col("ret_event").median().alias("ret_median"),
        pl.col("ret_event").std().alias("ret_std"),
        *quantiles,
        pl.col("bars_to_event").mean().alias("bars_mean"),
        pl.col("bars_to_event").median().alias("bars_median"),
        pl.when(pl.col("label")==1).then(pl.col("ret_event")).otherwise(None).mean().alias("ret_mean_tp"),
        pl.when(pl.col("label")==-1).then(pl.col("ret_event")).otherwise(None).mean().alias("ret_mean_sl"),
        pl.when(pl.col("label")==0).then(pl.col("ret_event")).otherwise(None).mean().alias("ret_mean_timeout"),
        sum_tp_ret.alias("sum_ret_tp"),
        sum_sl_abs.alias("sum_abs_ret_sl"),
    )
)

den_tp_sl = _safe_pos((pl.col("tp") + pl.col("sl")).cast(pl.Float64()), 1.0)
den_pf    = _safe_pos(pl.col("sum_abs_ret_sl"), 1e-12)

kpi = (
    kpi_base
    .with_columns(
        (pl.col("tp") / pl.col("n_signals")).alias("hit_rate_tp_incl_to"),
        (pl.col("tp").cast(pl.Float64()) / den_tp_sl).alias("hit_rate_tp_excl_to"),
        (pl.col("sum_ret_tp") / den_pf).alias("profit_factor_stat"),
        (pl.col("ret_std") / (pl.col("n_signals").cast(pl.Float64()) ** 0.5)).alias("ret_se"),
    )
    .with_columns(
        (pl.col("ret_mean") - 1.96 * pl.col("ret_se")).alias("ret_mean_ci95_lo"),
        (pl.col("ret_mean") + 1.96 * pl.col("ret_se")).alias("ret_mean_ci95_hi"),
    )
)

p   = (pl.col("tp").cast(pl.Float64()) / den_tp_sl)
var = p * (1.0 - p) / den_tp_sl
ci_lo = _clamp01(p - 1.96 * (var ** 0.5))
ci_hi = _clamp01(p + 1.96 * (var ** 0.5))

kpi = kpi.with_columns(
    ci_lo.alias("hit_rate_excl_to_ci95_lo"),
    ci_hi.alias("hit_rate_excl_to_ci95_hi"),
).sort("entry_col")

print("🗓️  Estabilidad muestral por año (ret_mean y n) ...")
kpi_year = (
    fwd.with_columns(pl.col("ts").dt.year().alias("year"))
       .group_by(["entry_col","year"])
       .agg(
            pl.len().alias("n_signals"),
            pl.col("ret_event").mean().alias("ret_mean"),
            pl.col("ret_event").median().alias("ret_median"),
       )
       .sort(["entry_col","year"])
)

print("💾 Guardando reportes ...")
kpi.write_parquet(KPI_PARQ)
kpi_year.write_parquet(KPI_YEAR_PARQ)
with KPI_JSON.open("w", encoding="utf-8") as f:
    json.dump({
        "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
        "symbol": SYMBOL, "timeframe": TF,
        "min_trades_cfg": int(min_trades_cfg),
        "rows": kpi.to_dicts()
    }, f, indent=2, ensure_ascii=False)

print("🏁 Rankings (filtro min_trades) ...")
kpi_f = kpi.filter(pl.col("n_signals") >= min_trades_cfg)
if kpi_f.height == 0:
    print(f"   ℹ️ Ningún combo alcanza min_trades={min_trades_cfg}.")
else:
    print("   ▶ Top 5 por ret_mean:"); print(kpi_f.sort("ret_mean", descending=True).head(5))
    print("   ▶ Top 5 por profit_factor_stat:"); print(kpi_f.sort("profit_factor_stat", descending=True).head(5))
    print("   ▶ Top 5 por hit_rate_tp_excl_to:"); print(kpi_f.sort("hit_rate_tp_excl_to", descending=True).head(5))

print("✅ Celda 5 finalizada sin errores.\n")


📖 Leyendo configuración ...
   - FORWARD_PARQ                  : C:\Quant\TWF\outputs\BTCUSDT\15m\forward\forward_BTCUSDT_15m.parquet
   - KPI_PARQ                      : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\kpi_BTCUSDT_15m.parquet
   - KPI_YEAR_PARQ                 : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\kpi_by_year_BTCUSDT_15m.parquet
   - min_trades_cfg                : 1000
📦 Cargando forward (por-señal) ...


   - forward_rows                  : 447948
   - forward_cols                  : 11
🧮 Agregando KPIs por 'entry_col' ...
🗓️  Estabilidad muestral por año (ret_mean y n) ...
💾 Guardando reportes ...
🏁 Rankings (filtro min_trades) ...
   ▶ Top 5 por ret_mean:
shape: (5, 28)
┌─────────────┬───────────┬───────┬───────┬───┬─────────────┬────────────┬────────────┬────────────┐
│ entry_col   ┆ n_signals ┆ tp    ┆ sl    ┆ … ┆ ret_mean_ci ┆ ret_mean_c ┆ hit_rate_e ┆ hit_rate_e │
│ ---         ┆ ---       ┆ ---   ┆ ---   ┆   ┆ 95_lo       ┆ i95_hi     ┆ xcl_to_ci9 ┆ xcl_to_ci9 │
│ str         ┆ u32       ┆ i32   ┆ i32   ┆   ┆ ---         ┆ ---        ┆ 5_lo       ┆ 5_hi       │
│             ┆           ┆       ┆       ┆   ┆ f64         ┆ f64        ┆ ---        ┆ ---        │
│             ┆           ┆       ┆       ┆   ┆             ┆            ┆ f64        ┆ f64        │
╞═════════════╪═══════════╪═══════╪═══════╪═══╪═════════════╪════════════╪════════════╪════════════╡
│ entry_er10_ ┆ 7763

In [235]:
# Celda 5b (QA FORMAL v1.4 FIX FINAL): [PRUEBAS UNITARIAS Y SANIDAD → reports/qa_results_* + run_log_records.csv]
# ---------------------------------------------------------------------------------------------------
# Ubicación: NUEVA. Colócala *después* de la Celda 5 y *antes* de la Celda 6.
# Entradas:
#   • outputs/{SYMBOL}/{TF}/logs/config_{SYMBOL}_{TF}.json
#   • outputs/{SYMBOL}/{TF}/forward/forward_{SYMBOL}_{TF}.parquet
# Salidas:
#   • outputs/{SYMBOL}/{TF}/reports/qa_results_{SYMBOL}_{TF}.json
#   • outputs/{SYMBOL}/{TF}/reports/run_log_records.csv   (append)
# Objetivo:
#   • Validaciones formales del artefacto 'forward' y labeling (contrato mínimo + checks de sanidad).
#   • FIX v1.4: evita DuplicateError usando unique(subset=...) para contar pares únicos (entry_col, ts).
# ---------------------------------------------------------------------------------------------------

import os, json, math, csv, hashlib
from pathlib import Path
from datetime import datetime, timezone
import polars as pl
import numpy as np

def _print_kv(k, v): print(f"   - {k:<34}: {v}")

def _status(ok: bool | None) -> str:
    if ok is True: return "ok"
    if ok is False: return "fail"
    return "skip"

def _md5_file(path: Path) -> str | None:
    try:
        h = hashlib.md5()
        with path.open("rb") as fh:
            for chunk in iter(lambda: fh.read(8192), b""):
                h.update(chunk)
        return h.hexdigest()
    except Exception:
        return None

def _ensure_ts_datetime(df: pl.DataFrame) -> tuple[pl.DataFrame, bool, str]:
    """
    Valida/coacciona 'ts' a Datetime sin depender de pl.datatypes.is_datetime.
    1) Probar operación dt.year; 2) strptime; 3) cast.
    """
    dtype_str = str(df.schema.get("ts"))
    try:
        _ = df.select(pl.col("ts").dt.year).height
        return df, True, dtype_str
    except Exception:
        pass
    try:
        df = df.with_columns(pl.col("ts").str.strptime(pl.Datetime, strict=False, fmt=None))
        _ = df.select(pl.col("ts").dt.year).height
        return df, True, "coerced:strptime"
    except Exception:
        pass
    try:
        df = df.with_columns(pl.col("ts").cast(pl.Datetime))
        _ = df.select(pl.col("ts").dt.year).height
        return df, True, "coerced:cast"
    except Exception:
        return df, False, dtype_str

print("📖 Leyendo configuración (Celda 5b QA FORMAL v1.4 FIX) ...")
need_env = ("TWF_SYMBOL","TWF_TF","TWF_START_DATE","TWF_END_DATE","TWF_SEED_BASE")
assert all(k in os.environ for k in need_env), "❌ Faltan TWF_*; ejecuta celdas previas."
SYMBOL = os.environ["TWF_SYMBOL"]; TF = os.environ["TWF_TF"]

PROJ_ROOT   = Path(os.environ.get("TWF_BASE", Path.cwd())).resolve()
CONFIG_PATH = PROJ_ROOT / "outputs" / SYMBOL / TF / "logs" / f"config_{SYMBOL}_{TF}.json"
assert CONFIG_PATH.exists(), f"❌ No existe configuración: {CONFIG_PATH}"

CFG = json.loads(CONFIG_PATH.read_text(encoding="utf-8"))
PATHS = {k: Path(v) for k, v in CFG["paths"].items()}
PATHS["reports"].mkdir(parents=True, exist_ok=True)

FWD_PARQ   = PATHS["forward"] / f"forward_{SYMBOL}_{TF}.parquet"
QA_JSON    = PATHS["reports"] / f"qa_results_{SYMBOL}_{TF}.json"
RUNLOG_CSV = PATHS["reports"] / "run_log_records.csv"

_print_kv("FWD_PARQ", FWD_PARQ)
assert FWD_PARQ.exists(), f"❌ Falta {FWD_PARQ}"

print("📦 Cargando forward ...")
fwd = pl.read_parquet(FWD_PARQ)
_print_kv("forward_shape", f"{fwd.height}x{fwd.width}")

# -------------------------------- Checks --------------------------------
checks = []
def add_check(name: str, ok: bool | None, detail: str = "", extras: dict | None = None):
    row = {"name": name, "status": _status(ok), "detail": detail}
    if extras: row.update(extras)
    checks.append(row)

# 1) Esquema mínimo
required_cols = ["ts","entry_col","label","ret_event"]
missing = [c for c in required_cols if c not in fwd.columns]
add_check("schema_min_required", len(missing)==0, f"missing={missing}")

# 2) Tipos/NA básicos + coerción segura de ts a Datetime
if set(required_cols).issubset(set(fwd.columns)):
    fwd, ts_ok, ts_dtype = _ensure_ts_datetime(fwd)
    add_check("dtype_ts_is_datetime", ts_ok, f"dtype_detected={ts_dtype}")
    # Dominio de label
    try:
        label_set = set(fwd.get_column("label").unique().to_list())
        labels_ok = label_set.issubset({-1,0,1})
    except Exception:
        label_set, labels_ok = set(), False
    add_check("label_domain_-1_0_1", labels_ok, f"seen={sorted(list(label_set)) if label_set else 'NA'}")
    # NAs en columnas clave
    na_counts = {}
    na_ok = True
    for c in required_cols:
        n_na = int(fwd.select(pl.col(c).is_null().sum().alias("__sum__")).item())
        na_counts[c] = n_na
        if n_na > 0: na_ok = False
    add_check("no_nulls_in_required", na_ok, f"nulls={na_counts}")

# 3) Coherencia de conteos
if set(required_cols).issubset(set(fwd.columns)):
    n  = fwd.height
    tp = int(fwd.select(((pl.col("label")==1).cast(pl.Int32)).sum().alias("__tp__")).item())
    sl = int(fwd.select(((pl.col("label")==-1).cast(pl.Int32)).sum().alias("__sl__")).item())
    to = int(fwd.select(((pl.col("label")==0).cast(pl.Int32)).sum().alias("__to__")).item())
    add_check("counts_match_total", (tp + sl + to) == n, f"n={n}, tp={tp}, sl={sl}, timeout={to}")

# 4) Duplicados y monotonicidad de ts por entry_col (FIX DuplicateError)
if {"entry_col","ts"}.issubset(fwd.columns):
    n_total  = fwd.height
    # Conteo de pares únicos (entry_col, ts) sin .alias() duplicados:
    n_unique = fwd.unique(subset=["entry_col","ts"]).height
    dups = n_total - n_unique
    add_check("no_duplicate_entry_ts", dups==0, f"duplicates={dups}")

    # Monotonicidad no decreciente por entry_col (vectorizado con shift)
    sorted_df = fwd.select("entry_col","ts").sort(["entry_col","ts"]).with_columns(
        prev_entry = pl.col("entry_col").shift(1),
        prev_ts    = pl.col("ts").shift(1),
    )
    violations = int(sorted_df.select(
        ((pl.col("entry_col")==pl.col("prev_entry")) & (pl.col("ts") < pl.col("prev_ts"))).sum().alias("__viol__")
    ).item())
    add_check("ts_monotonic_non_decreasing_by_entry", violations==0, f"violations={violations}")

# 5) “Doble cruce” (si hay flags/tiempos). Regla: doble cruce → -1.
tp_flag_cols = [c for c in fwd.columns if c.lower() in {"tp_hit","hit_tp","tp_reached"}]
sl_flag_cols = [c for c in fwd.columns if c.lower() in {"sl_hit","hit_sl","sl_reached"}]
tp_time_cols = [c for c in fwd.columns if c.lower() in {"tp_ts","tp_time","tp_idx"}]
sl_time_cols = [c for c in fwd.columns if c.lower() in {"sl_ts","sl_time","sl_idx"}]

if tp_flag_cols and sl_flag_cols:
    tp_flag = pl.col(tp_flag_cols[0]).cast(pl.Boolean).fill_null(False)
    sl_flag = pl.col(sl_flag_cols[0]).cast(pl.Boolean).fill_null(False)
    both = fwd.filter(tp_flag & sl_flag)
    if both.height == 0:
        add_check("double_cross_flags_detected", True, "0 rows con ambos flags")
    else:
        bad = both.filter(pl.col("label") != -1).height
        info = "(con tiempos)" if (tp_time_cols and sl_time_cols) else "(sin tiempos)"
        add_check("double_cross_labeled_minus_one", bad==0,
                  f"{info} double_cross_rows={both.height}, wrong_label_rows={bad}")
else:
    add_check("double_cross_check", None, "skip: no hay flags tp/sl")

# 6) Sanidad de ret_event y (si existen) mfe/mae
if "ret_event" in fwd.columns:
    finite = int(fwd.select(pl.col("ret_event").is_finite().cast(pl.Int32).sum().alias("__sum__")).item())
    add_check("ret_event_all_finite", finite == fwd.height, f"finite={finite}/{fwd.height}")
    ret = fwd.get_column("ret_event").fill_null(0.0).to_numpy()
    if ret.size > 0:
        p999 = float(np.nanpercentile(np.abs(ret), 99.9))
        std  = float(np.nanstd(ret))
        thr  = max(p999, 10.0*std) if np.isfinite(std) and std>0 else p999
        bad  = int(np.sum(np.abs(ret) > (thr if np.isfinite(thr) and thr>0 else 1e9)))
        add_check("ret_event_no_extreme_outliers", bad==0, f"outliers>{thr:.6g} = {bad}")
    else:
        add_check("ret_event_no_extreme_outliers", None, "skip: sin datos")
else:
    add_check("ret_event_present", False, "ret_event no existe en forward")

if {"mfe","mae"}.issubset(fwd.columns):
    mfef = int(fwd.select(pl.col("mfe").is_finite().cast(pl.Int32).sum().alias("__sum__")).item())
    maef = int(fwd.select(pl.col("mae").is_finite().cast(pl.Int32).sum().alias("__sum__")).item())
    add_check("mfe_mae_all_finite", (mfef==fwd.height and maef==fwd.height),
              f"mfe_finite={mfef}/{fwd.height}, mae_finite={maef}/{fwd.height}")
    if {"label","ret_event"}.issubset(fwd.columns):
        dfc = fwd.select(
            pl.when(pl.col("label")==1).then(pl.col("mfe") >= pl.col("ret_event").abs()).otherwise(True).alias("__mfe_ok__"),
            pl.when(pl.col("label")==-1).then(pl.col("mae") >= pl.col("ret_event").abs()).otherwise(True).alias("__mae_ok__"),
        )
        mfe_ok = bool(dfc.select(pl.col("__mfe_ok__").all()).item())
        mae_ok = bool(dfc.select(pl.col("__mae_ok__").all()).item())
        add_check("mfe_coherence_when_tp", mfe_ok, "mfe >= |ret_event| si label=1")
        add_check("mae_coherence_when_sl", mae_ok, "mae >= |ret_event| si label=-1")
else:
    add_check("mfe_mae_checks", None, "skip: no hay columnas mfe/mae")

# 7) MD5 bit-a-bit del parquet de forward
md5 = _md5_file(FWD_PARQ)
add_check("forward_md5_computed", md5 is not None, f"md5={md5}")

# ---------------------------- Guardado de resultados ----------------------------
created_utc = datetime.now(timezone.utc).isoformat(timespec="seconds")
summary = {
    "created_utc": created_utc,
    "symbol": SYMBOL,
    "timeframe": TF,
    "forward_shape": {"rows": fwd.height, "cols": fwd.width},
    "forward_md5": md5,
    "checks": checks,
    "all_ok": all(c["status"] in ("ok","skip") for c in checks),
}

QA_JSON.write_text(json.dumps(summary, indent=2, ensure_ascii=False), encoding="utf-8")
_print_kv("QA_JSON", QA_JSON)

# Append a run_log_records.csv
RUNLOG_CSV.parent.mkdir(parents=True, exist_ok=True)
header = ["created_utc","symbol","tf","cell","rows","block_method","B","comment"]
append_header = not RUNLOG_CSV.exists()
with RUNLOG_CSV.open("a", newline="", encoding="utf-8") as fh:
    w = csv.writer(fh)
    if append_header:
        w.writerow(header)
    w.writerow([
        created_utc, SYMBOL, TF, "5b_QA_formal_v1_4_fix",
        fwd.height, "", "", f"QA formal; all_ok={summary['all_ok']}"
    ])

print("✅ Celda 5b (QA FORMAL v1.4 FIX) finalizada.")
 

📖 Leyendo configuración (Celda 5b QA FORMAL v1.4 FIX) ...
   - FWD_PARQ                          : C:\Quant\TWF\outputs\BTCUSDT\15m\forward\forward_BTCUSDT_15m.parquet
📦 Cargando forward ...
   - forward_shape                     : 447948x11
   - QA_JSON                           : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\qa_results_BTCUSDT_15m.json
✅ Celda 5b (QA FORMAL v1.4 FIX) finalizada.


In [236]:
# Celda 6 (states_base v2): [ESTADOS/MEDIDAS BASE → reports/states_{SYMBOL}_{TF}.parquet]
# -------------------------------------------------------------------------------------------------
# - Estudio estadístico (NO backtest): cortes por tiempo y régimen del mercado del *entry*.
# - Lee forward_{SYMBOL}_{TF}.parquet y genera métricas por estado y entry_col.
# - Estados SIEMPRE: year, month, dow(0=Mon), hour(0..23), session(ASIA/EU/US), vol_bin(LOW/MID/HIGH/ULTRA).
# - Estados OPCIONALES (sólo si hay serie de precio para ventana): 
#       trend_er (ER de Kaufman sobre precio base), density (IQR de retornos, proxy de congestión).
#   * Ambos se calculan con ventana configurable y se discretizan en bins LOW/MID/HIGH.
# - Reconstruye vol_pct_est = (up_price/entry_price - 1) / up_mult (usado en 4c) para vol_bin.
# - Compatibilidad Polars "legacy": sin clip_min() ni pl.sqrt(); usa denominadores seguros y **0.5.
# - No rompe Celda 8: ésta detecta automáticamente los 'state_type' presentes en states_*.parquet.
# -------------------------------------------------------------------------------------------------

import os, json
from pathlib import Path
from datetime import datetime, timezone
import polars as pl

def _print_kv(k, v):
    print(f"   - {k:<30}: {v}")

def _safe_pos(expr: pl.Expr, floor: float) -> pl.Expr:
    # asegura expr >= floor (evita división por 0)
    return pl.when(expr < pl.lit(floor)).then(pl.lit(floor)).otherwise(expr)

# -------------------- A) Config y paths --------------------
print("📖 Leyendo configuración ...")
need_env = ("TWF_SYMBOL","TWF_TF","TWF_START_DATE","TWF_END_DATE","TWF_SEED_BASE")
assert all(k in os.environ for k in need_env), "Faltan TWF_*; ejecuta celdas previas."
SYMBOL = os.environ["TWF_SYMBOL"]; TF = os.environ["TWF_TF"]

PROJ_ROOT   = Path(os.environ.get("TWF_BASE", Path.cwd())).resolve()
CONFIG_PATH = PROJ_ROOT / "outputs" / SYMBOL / TF / "logs" / f"config_{SYMBOL}_{TF}.json"
assert CONFIG_PATH.exists(), f"No existe configuración: {CONFIG_PATH}"

with CONFIG_PATH.open("r", encoding="utf-8") as f:
    CFG = json.load(f)

PATHS         = {k: Path(v) for k, v in CFG["paths"].items()}
FORWARD_PARQ  = PATHS["forward"] / f"forward_{SYMBOL}_{TF}.parquet"
REPORTS_DIR   = PATHS["reports"]
STATES_PARQ   = REPORTS_DIR / f"states_{SYMBOL}_{TF}.parquet"
STATES_META   = REPORTS_DIR / f"states_meta_{SYMBOL}_{TF}.json"

# Parámetros opcionales para nuevos estados
STATES_CFG    = (CFG.get("states", {}) or {})
ER_WINDOW     = int(STATES_CFG.get("er_window", 100))     # ventana para ER de Kaufman
DENS_WINDOW   = int(STATES_CFG.get("density_window", 100))# ventana para IQR de retornos
USE_CLOSE_PREF= bool(STATES_CFG.get("prefer_close", True))

_print_kv("FORWARD_PARQ", FORWARD_PARQ)
_print_kv("STATES_PARQ", STATES_PARQ)
_print_kv("ER_WINDOW", ER_WINDOW)
_print_kv("DENS_WINDOW", DENS_WINDOW)
assert FORWARD_PARQ.exists(), f"No existe forward: {FORWARD_PARQ}"
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

# -------------------- B) Cargar forward -------------------
print("📦 Cargando forward ...")
fwd = pl.read_parquet(FORWARD_PARQ)
_print_kv("forward_rows", fwd.height)
_print_kv("forward_cols", fwd.width)

req_cols = ["ts","entry_col","label","ret_event","bars_to_event","entry_price","up_price"]
missing = [c for c in req_cols if c not in fwd.columns]
assert not missing, f"❌ Faltan columnas en forward: {missing}"

# -------------------- C) Derivar estados -------------------
print("🧭 Derivando estados (tiempo y régimen) ...")
UP_MULT = float(CFG.get("labeling", {}).get("triple_barrier", {}).get("up_mult", 2.0))

# Tiempos básicos
fwd = fwd.with_columns([
    pl.col("ts").dt.year().alias("year"),
    pl.col("ts").dt.month().alias("month"),
    pl.col("ts").dt.weekday().alias("dow"),   # 0=Mon ... 6=Sun
    pl.col("ts").dt.hour().alias("hour"),
])

# Sesión por hora UTC (bandas simples, sin DST):
#   ASIA:   [00..07], EUROPE: [08..15], US: [16..23]
session_expr = (
    pl.when((pl.col("hour") >= 0) & (pl.col("hour") <= 7)).then(pl.lit("ASIA"))
     .when((pl.col("hour") >= 8) & (pl.col("hour") <= 15)).then(pl.lit("EUROPE"))
     .otherwise(pl.lit("US"))
)
fwd = fwd.with_columns(session_expr.alias("session"))

# Volatilidad estimada al entry (reconstruida desde up_mult)
vol_pct_est = ((pl.col("up_price") / pl.col("entry_price")) - 1.0) / pl.lit(UP_MULT)
fwd = fwd.with_columns(vol_pct_est.cast(pl.Float64()).alias("vol_pct_est"))

# Cortes por cuartiles de vol_pct_est (global)
qdf = fwd.select([
    pl.col("vol_pct_est").quantile(0.25).alias("q25"),
    pl.col("vol_pct_est").quantile(0.50).alias("q50"),
    pl.col("vol_pct_est").quantile(0.75).alias("q75"),
])
q25, q50, q75 = qdf.row(0)
vol_bin_expr = pl.when(pl.col("vol_pct_est").is_null()).then(pl.lit("MISSING")).otherwise(
    pl.when(pl.col("vol_pct_est") < pl.lit(q25)).then(pl.lit("LOW"))
     .when(pl.col("vol_pct_est") < pl.lit(q50)).then(pl.lit("MID"))
     .when(pl.col("vol_pct_est") < pl.lit(q75)).then(pl.lit("HIGH"))
     .otherwise(pl.lit("ULTRA"))
)
fwd = fwd.with_columns(vol_bin_expr.alias("vol_bin"))
_print_kv("q25_vol", q25); _print_kv("q50_vol", q50); _print_kv("q75_vol", q75)

# --------- C2) Estados adicionales opcionales: ER y DENSITY ----------
# Precio base para ventanas: preferimos 'close' si existe; si no, usamos 'entry_price'.
price_col = None
if USE_CLOSE_PREF and "close" in fwd.columns:
    price_col = "close"
elif "entry_price" in fwd.columns:
    price_col = "entry_price"

added_optional_states = []
if price_col is not None and fwd.height >= max(ER_WINDOW, DENS_WINDOW) + 2:
    _print_kv("price_series_col", price_col)

    # Serie de log-precio y retornos 1-bar
    fwd = fwd.with_columns([
        pl.col(price_col).cast(pl.Float64()).alias("__px__"),
    ])
    fwd = fwd.with_columns([
        pl.col("__px__").log().diff().alias("__ret1__"),
    ])

    # --- ER de Kaufman (rolling) ---
    # ER_t = |px_t - px_{t-N}| / sum_{i=t-N+1..t} |px_i - px_{i-1}|
    num_er = pl.col("__px__").diff(n=ER_WINDOW).abs()
    den_er = pl.col("__px__").diff().abs().rolling_sum(window_size=ER_WINDOW, min_periods=ER_WINDOW)
    er_val = (num_er / _safe_pos(den_er, 1e-12)).alias("__er__")
    fwd = fwd.with_columns(er_val)

    # Bins para ER (terciles robustos)
    er_q = fwd.select([
        pl.col("__er__").quantile(0.33).alias("er_q33"),
        pl.col("__er__").quantile(0.66).alias("er_q66"),
    ])
    er_q33, er_q66 = er_q.row(0)
    fwd = fwd.with_columns(
        pl.when(pl.col("__er__").is_null()).then(pl.lit("MISSING")).otherwise(
            pl.when(pl.col("__er__") < pl.lit(er_q33)).then(pl.lit("LOW"))
             .when(pl.col("__er__") < pl.lit(er_q66)).then(pl.lit("MID"))
             .otherwise(pl.lit("HIGH"))
        ).alias("trend_er")
    )
    added_optional_states.append("trend_er")
    _print_kv("er_q33", er_q33); _print_kv("er_q66", er_q66)

    # --- DENSITY (IQR de retornos en ventana, proxy de congestión) ---
    # IQR = Q75(ret1) - Q25(ret1) sobre ventana; discretizado LOW/MID/HIGH
    # Nota: rolling_quantile requiere Polars >=0.20; si no está, se omite automáticamente.
    try:
        rq75 = pl.col("__ret1__").rolling_quantile(0.75, window_size=DENS_WINDOW, min_periods=DENS_WINDOW)
        rq25 = pl.col("__ret1__").rolling_quantile(0.25, window_size=DENS_WINDOW, min_periods=DENS_WINDOW)
        dens_val = (rq75 - rq25).alias("__dens__")
        fwd = fwd.with_columns(dens_val)

        dens_q = fwd.select([
            pl.col("__dens__").quantile(0.33).alias("dens_q33"),
            pl.col("__dens__").quantile(0.66).alias("dens_q66"),
        ])
        dens_q33, dens_q66 = dens_q.row(0)
        fwd = fwd.with_columns(
            pl.when(pl.col("__dens__").is_null()).then(pl.lit("MISSING")).otherwise(
                pl.when(pl.col("__dens__") < pl.lit(dens_q33)).then(pl.lit("LOW"))
                 .when(pl.col("__dens__") < pl.lit(dens_q66)).then(pl.lit("MID"))
                 .otherwise(pl.lit("HIGH"))
            ).alias("density")
        )
        added_optional_states.append("density")
        _print_kv("dens_q33", dens_q33); _print_kv("dens_q66", dens_q66)
    except Exception as e:
        _print_kv("density_skip_warn", f"omitido (rolling_quantile no disponible): {e}")

else:
    _print_kv("price_series_col", "NO DISPONIBLE — se omiten trend_er y density")

# -------------------- D) Agregadores reutilizables -----------
tp_expr  = (pl.col("label") == 1).cast(pl.Int32)
sl_expr  = (pl.col("label") == -1).cast(pl.Int32)
to_expr  = (pl.col("label") == 0).cast(pl.Int32)

def _agg_by_state(df: pl.DataFrame, state_col: str, state_name: str) -> pl.DataFrame:
    base = (
        df.group_by(["entry_col", state_col])
          .agg(
              pl.len().alias("n_signals"),
              tp_expr.sum().alias("tp"),
              sl_expr.sum().alias("sl"),
              to_expr.sum().alias("timeout"),
              pl.col("ret_event").mean().alias("ret_mean"),
              pl.col("ret_event").median().alias("ret_median"),
              pl.col("ret_event").std().alias("ret_std"),
              pl.col("ret_event").quantile(0.05).alias("ret_q05"),
              pl.col("ret_event").quantile(0.25).alias("ret_q25"),
              pl.col("ret_event").quantile(0.50).alias("ret_q50"),
              pl.col("ret_event").quantile(0.75).alias("ret_q75"),
              pl.col("ret_event").quantile(0.95).alias("ret_q95"),
              pl.col("bars_to_event").mean().alias("bars_mean"),
              pl.col("bars_to_event").median().alias("bars_median"),
          )
          .with_columns([
              (pl.col("tp") / pl.col("n_signals")).alias("hit_rate_tp_incl_to"),
              (pl.col("tp").cast(pl.Float64()) /
               _safe_pos((pl.col("tp")+pl.col("sl")).cast(pl.Float64()), 1.0)
              ).alias("hit_rate_tp_excl_to"),
              pl.lit(state_name).alias("state_type"),
              pl.col(state_col).cast(pl.Utf8).alias("state_value"),
          ])
          .drop(state_col)
    )
    return base

# -------------------- E) Cálculo por cada estado --------------
print("📊 Agregando por estados ...")
states_list = []

# Estados base siempre presentes
for sc, sn in [("year","year"), ("month","month"), ("dow","dow"), ("hour","hour"),
               ("session","session"), ("vol_bin","vol_bin")]:
    print(f"   • Estado: {sn}")
    states_list.append(_agg_by_state(fwd, sc, sn))

# Estados opcionales (si fueron generados)
if "trend_er" in fwd.columns:
    print("   • Estado: trend_er")
    states_list.append(_agg_by_state(fwd, "trend_er", "trend_er"))
if "density" in fwd.columns:
    print("   • Estado: density")
    states_list.append(_agg_by_state(fwd, "density", "density"))

states_df = pl.concat(states_list, how="vertical_relaxed").sort(["entry_col","state_type","state_value"])
_print_kv("states_rows", states_df.height)
_print_kv("states_cols", states_df.width)
print("👀 Sample HEAD:")
print(states_df.head(8))

# -------------------- F) Persistencia --------------------------
print("💾 Guardando estados ...")
states_df.write_parquet(STATES_PARQ)
with STATES_META.open("w", encoding="utf-8") as f:
    json.dump({
        "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
        "symbol": SYMBOL,
        "timeframe": TF,
        "up_mult": UP_MULT,
        "vol_quantiles": {"q25": float(q25), "q50": float(q50), "q75": float(q75)},
        "state_types": ["year","month","dow","hour","session","vol_bin"] + added_optional_states,
        "price_series_col": price_col if price_col is not None else "N/A",
        "er_window": ER_WINDOW,
        "density_window": DENS_WINDOW,
        "source": str(FORWARD_PARQ),
        "output": str(STATES_PARQ),
    }, f, indent=2, ensure_ascii=False)

_print_kv("STATES_PARQ", STATES_PARQ)
_print_kv("STATES_META", STATES_META)
print("✅ Celda 6 finalizada sin errores.\n")
 

📖 Leyendo configuración ...
   - FORWARD_PARQ                  : C:\Quant\TWF\outputs\BTCUSDT\15m\forward\forward_BTCUSDT_15m.parquet
   - STATES_PARQ                   : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\states_BTCUSDT_15m.parquet
   - ER_WINDOW                     : 100
   - DENS_WINDOW                   : 100
📦 Cargando forward ...
   - forward_rows                  : 447948
   - forward_cols                  : 11
🧭 Derivando estados (tiempo y régimen) ...
   - q25_vol                       : 0.0024798316741538873
   - q50_vol                       : 0.003859300154248846
   - q75_vol                       : 0.005715393011875403
   - price_series_col              : entry_price
   - er_q33                        : 0.004691534416456593
   - er_q66                        : 0.012106045792361658


C:\Users\PC\AppData\Local\Temp\ipykernel_40828\3044282886.py:132: DeprecationWarning: the argument `min_periods` for `Expr.rolling_sum` is deprecated. It was renamed to `min_samples` in version 1.21.0.
  den_er = pl.col("__px__").diff().abs().rolling_sum(window_size=ER_WINDOW, min_periods=ER_WINDOW)
C:\Users\PC\AppData\Local\Temp\ipykernel_40828\3044282886.py:156: DeprecationWarning: the argument `min_periods` for `Expr.rolling_quantile` is deprecated. It was renamed to `min_samples` in version 1.21.0.
  rq75 = pl.col("__ret1__").rolling_quantile(0.75, window_size=DENS_WINDOW, min_periods=DENS_WINDOW)
C:\Users\PC\AppData\Local\Temp\ipykernel_40828\3044282886.py:157: DeprecationWarning: the argument `min_periods` for `Expr.rolling_quantile` is deprecated. It was renamed to `min_samples` in version 1.21.0.
  rq25 = pl.col("__ret1__").rolling_quantile(0.25, window_size=DENS_WINDOW, min_periods=DENS_WINDOW)


   - dens_q33                      : 1.0275171382464166
   - dens_q66                      : 1.137051666878742
📊 Agregando por estados ...
   • Estado: year
   • Estado: month
   • Estado: dow
   • Estado: hour
   • Estado: session
   • Estado: vol_bin
   • Estado: trend_er
   • Estado: density
   - states_rows                   : 567
   - states_cols                   : 19
👀 Sample HEAD:
shape: (8, 19)
┌─────────────┬───────────┬───────┬───────┬───┬─────────────┬────────────┬────────────┬────────────┐
│ entry_col   ┆ n_signals ┆ tp    ┆ sl    ┆ … ┆ hit_rate_tp ┆ hit_rate_t ┆ state_type ┆ state_valu │
│ ---         ┆ ---       ┆ ---   ┆ ---   ┆   ┆ _incl_to    ┆ p_excl_to  ┆ ---        ┆ e          │
│ str         ┆ u32       ┆ i32   ┆ i32   ┆   ┆ ---         ┆ ---        ┆ str        ┆ ---        │
│             ┆           ┆       ┆       ┆   ┆ f64         ┆ f64        ┆            ┆ str        │
╞═════════════╪═══════════╪═══════╪═══════╪═══╪═════════════╪════════════╪════════════╪═

In [237]:
# Celda 7a (wf_windows v1): [VENTANAS WALK-FORWARD (SIN SOLAPE) → logs/wf_windows_{SYMBOL}_{TF}.json]
# ----------------------------------------------------------------------------------------------------
# - Estudio estadístico (NO backtest): solo define ventanas TRAIN/TEST sin solape.
# - Lee config + forward_{SYMBOL}_{TF}.parquet, respeta TWF_START_DATE/TWF_END_DATE.
# - Usa CFG["wf"] si existe; si no, defaults: train_days=365, test_days=30, step_days=test_days.
# - Persiste wf_windows_{SYMBOL}_{TF}.json con lista de ventanas y conteos muestrales (train/test).
# - Compatibilidad "legacy": Polars básico, nada exótico.
# ----------------------------------------------------------------------------------------------------

import os, json
from pathlib import Path
from datetime import datetime, timezone, timedelta
import polars as pl

def _print_kv(k, v):
    print(f"   - {k:<30}: {v}")

def _to_utc(dt: datetime) -> datetime:
    # Asegura timezone UTC
    return dt if dt.tzinfo is not None else dt.replace(tzinfo=timezone.utc)

def _parse_date_env(key: str) -> datetime:
    s = os.environ[key]
    # Permite 'YYYY-MM-DD' o ISO completo; si es solo fecha, asume 00:00 UTC
    try:
        dt = datetime.fromisoformat(s)
    except Exception:
        # Fallback robusto (solo por si hay formato raro)
        y, m, d = map(int, s.split("-"))
        dt = datetime(y, m, d)
    dt = _to_utc(dt)
    if dt.hour == 0 and dt.minute == 0 and dt.second == 0:
        # Mantener como límite inferior/superior según el uso
        pass
    return dt

# -------------------- A) Config y paths --------------------
print("📖 Leyendo configuración ...")
need_env = ("TWF_SYMBOL","TWF_TF","TWF_START_DATE","TWF_END_DATE","TWF_SEED_BASE")
assert all(k in os.environ for k in need_env), "Faltan TWF_*; ejecuta celdas previas."
SYMBOL = os.environ["TWF_SYMBOL"]; TF = os.environ["TWF_TF"]

PROJ_ROOT   = Path(os.environ.get("TWF_BASE", Path.cwd())).resolve()
CONFIG_PATH = PROJ_ROOT / "outputs" / SYMBOL / TF / "logs" / f"config_{SYMBOL}_{TF}.json"
assert CONFIG_PATH.exists(), f"No existe configuración: {CONFIG_PATH}"

with CONFIG_PATH.open("r", encoding="utf-8") as f:
    CFG = json.load(f)

PATHS        = {k: Path(v) for k, v in CFG["paths"].items()}
FORWARD_PARQ = PATHS["forward"] / f"forward_{SYMBOL}_{TF}.parquet"
LOGS_DIR     = PATHS["logs"]
LOGS_DIR.mkdir(parents=True, exist_ok=True)

WF_JSON = LOGS_DIR / f"wf_windows_{SYMBOL}_{TF}.json"

_print_kv("FORWARD_PARQ", FORWARD_PARQ)
_print_kv("WF_JSON", WF_JSON)
assert FORWARD_PARQ.exists(), f"No existe forward: {FORWARD_PARQ}"

# -------------------- B) Cargar forward y rango disponible -------------
print("📦 Cargando forward ...")
fwd = pl.read_parquet(FORWARD_PARQ).select(["ts"])
first_ms, last_ms = fwd.select(
    pl.col("ts").dt.epoch("ms").min().alias("first_ms"),
    pl.col("ts").dt.epoch("ms").max().alias("last_ms")
).row(0)
first_ts = datetime.fromtimestamp(int(first_ms)/1000, tz=timezone.utc)
last_ts  = datetime.fromtimestamp(int(last_ms)/1000, tz=timezone.utc)
_print_kv("forward.first_ts", first_ts.isoformat(timespec="seconds"))
_print_kv("forward.last_ts",  last_ts.isoformat(timespec="seconds"))
_print_kv("forward_rows", fwd.height)

# -------------------- C) WF config (CFG['wf'] o defaults) --------------
WF_CFG = CFG.get("wf", {})
train_days = int(WF_CFG.get("train_days", 365))
test_days  = int(WF_CFG.get("test_days", 30))
step_days  = int(WF_CFG.get("step_days", test_days))  # sin solape por defecto
min_train_signals = int(WF_CFG.get("min_train_signals", 0))  # solo informativo en 7a
min_test_signals  = int(WF_CFG.get("min_test_signals", 0))   # (el filtro real se aplica en 7b si lo decides)

_print_kv("train_days", train_days)
_print_kv("test_days",  test_days)
_print_kv("step_days",  step_days)
_print_kv("min_train_signals", min_train_signals)
_print_kv("min_test_signals",  min_test_signals)

# Rango WF acotado por entorno + datos
env_start = _parse_date_env("TWF_START_DATE")
env_end   = _parse_date_env("TWF_END_DATE")
wf_start  = max(env_start, first_ts)
wf_end    = min(env_end,   last_ts)
_print_kv("wf_start(used)", wf_start.isoformat(timespec="seconds"))
_print_kv("wf_end(used)",   wf_end.isoformat(timespec="seconds"))

# -------------------- D) Construcción de ventanas ----------------------
print("🧱 Construyendo ventanas TRAIN/TEST (sin solape) ...")
ts_series = fwd["ts"]  # para conteos rápidos

windows = []
cursor = wf_start
one_us = timedelta(microseconds=1)  # para inclusividad sobre límites

while True:
    train_start = cursor
    train_end   = train_start + timedelta(days=train_days) - one_us
    test_start  = train_end + one_us
    test_end    = test_start + timedelta(days=test_days) - one_us

    if test_end > wf_end:
        break

    # Conteos (muestrales, no filtros estrictos aquí)
    n_train = fwd.filter((pl.col("ts") >= train_start) & (pl.col("ts") <= train_end)).height
    n_test  = fwd.filter((pl.col("ts") >= test_start)  & (pl.col("ts") <= test_end)).height

    windows.append({
        "id":       f"wf_{len(windows):03d}",
        "train":    {"start": train_start.isoformat(timespec="seconds"),
                     "end":   train_end.isoformat(timespec="seconds"),
                     "n": int(n_train)},
        "test":     {"start": test_start.isoformat(timespec="seconds"),
                     "end":   test_end.isoformat(timespec="seconds"),
                     "n": int(n_test)},
    })

    cursor = test_start + timedelta(days=step_days)

_print_kv("wf_windows_count", len(windows))
if windows:
    print("👀 Muestra de ventanas:")
    print(json.dumps(windows[:2] + windows[-2:], indent=2))

# -------------------- E) Persistencia ----------------------------------
print("💾 Guardando wf_windows ...")
with WF_JSON.open("w", encoding="utf-8") as f:
    json.dump({
        "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
        "symbol": SYMBOL,
        "timeframe": TF,
        "config_used": {
            "train_days": train_days,
            "test_days": test_days,
            "step_days": step_days,
            "min_train_signals": min_train_signals,
            "min_test_signals": min_test_signals,
            "wf_start": wf_start.isoformat(timespec="seconds"),
            "wf_end":   wf_end.isoformat(timespec="seconds"),
        },
        "windows": windows
    }, f, indent=2, ensure_ascii=False)

_print_kv("WF_JSON", WF_JSON)
print("✅ Celda 7a finalizada sin errores.\n")


📖 Leyendo configuración ...
   - FORWARD_PARQ                  : C:\Quant\TWF\outputs\BTCUSDT\15m\forward\forward_BTCUSDT_15m.parquet
   - WF_JSON                       : C:\Quant\TWF\outputs\BTCUSDT\15m\logs\wf_windows_BTCUSDT_15m.json
📦 Cargando forward ...
   - forward.first_ts              : 2021-01-01T12:15:00+00:00
   - forward.last_ts               : 2025-11-03T19:45:00+00:00
   - forward_rows                  : 447948
   - train_days                    : 365
   - test_days                     : 30
   - step_days                     : 30
   - min_train_signals             : 0
   - min_test_signals              : 0
   - wf_start(used)                : 2021-01-01T12:15:00+00:00
   - wf_end(used)                  : 2025-11-03T19:45:00+00:00
🧱 Construyendo ventanas TRAIN/TEST (sin solape) ...
   - wf_windows_count              : 4
👀 Muestra de ventanas:
[
  {
    "id": "wf_000",
    "train": {
      "start": "2021-01-01T12:15:00+00:00",
      "end": "2022-01-01T12:14:59+00:00",
    

In [238]:
# Celda 7b (v3 FINAL, TZ-UTC + RANK FIX): [SCORING EN TRAIN + N_min + desempates → wf_train_scores_*]
# ---------------------------------------------------------------------------------------------------
# Ubicación: REEMPLAZA la Celda 7b actual (después de 7a y antes de 7c).
# Entradas:
#   • outputs/{SYMBOL}/{TF}/logs/config_{SYMBOL}_{TF}.json
#   • outputs/{SYMBOL}/{TF}/logs/wf_windows_{SYMBOL}_{TF}.json
#   • outputs/{SYMBOL}/{TF}/forward/forward_{SYMBOL}_{TF}.parquet
# Salidas:
#   • outputs/{SYMBOL}/{TF}/reports/wf_train_scores_{SYMBOL}_{TF}.parquet
#   • outputs/{SYMBOL}/{TF}/reports/wf_train_scores_{SYMBOL}_{TF}.json
# Objetivo:
#   • Por (wf_id, entry_col) en TRAIN: KPIs + L_b_H_train, n_eff_hit_train y Score_train estable:
#       - support_excl_to = tp+sl ; p_good = tp/(tp+sl)
#       - EV (ret_mean), ret_std, profit_factor_stat_train
#       - L_b_H_train por ACF (dos lags en banda ±1.96/√n_pairs)
#       - n_eff_hit_train = max(1, n_pairs / L_b_H_train)
#       - Score_train := EV_train × (2·p_good−1) × log1p(n_eff_hit_train)
#   • Aplicar N_min por soporte (GOOD y BAD) y ordenar con desempates normados.
#   • FIX: ranking sin GroupBy.apply (compat con tu Polars).
# ---------------------------------------------------------------------------------------------------

import os, json, math
from pathlib import Path
from datetime import datetime, timezone
import polars as pl
import numpy as np

def _print_kv(k, v): print(f"   - {k:<34}: {v}")

def _z_acf_band(n_pairs: int) -> float:
    n_pairs = max(1, int(n_pairs))
    return 1.96 / math.sqrt(n_pairs)

def _acf(x: np.ndarray, max_lag: int) -> np.ndarray:
    x = np.asarray(x, dtype=float)
    x = x - np.nanmean(x)
    x[np.isnan(x)] = 0.0
    n = len(x)
    if n == 0:
        return np.zeros(max_lag+1)
    denom = float(np.dot(x, x))
    if denom <= 0:
        return np.zeros(max_lag+1)
    out = np.empty(max_lag+1, dtype=float)
    for k in range(0, max_lag+1):
        out[k] = float(np.dot(x[:n-k], x[k:]) / denom)
    return out

def _estimate_block_len_train(labels_vec: np.ndarray) -> int:
    """L_b_H_train: primer H con 2 lags consecutivos en banda."""
    if labels_vec.size == 0:
        return 1
    y = np.zeros_like(labels_vec, dtype=float)
    y[labels_vec == 1]  = 1.0
    y[labels_vec == -1] = -1.0
    n_pairs = int(np.sum(labels_vec != 0))
    band = _z_acf_band(n_pairs)
    max_lag = max(10, min(200, int(len(y) * 0.2)))
    r = _acf(y, max_lag)
    for H in range(1, max_lag-1):
        if abs(r[H]) <= band and abs(r[H+1]) <= band:
            return max(1, H)
    return max(1, int(np.ceil(np.sqrt(max(n_pairs, 1)))))  # fallback conservador

def _score_train_fn(ev: float | None, p_good: float | None, n_eff: int | float | None) -> float | None:
    if ev is None or p_good is None or n_eff is None:
        return None
    try:
        return float(ev) * (2.0*float(p_good) - 1.0) * math.log1p(max(1.0, float(n_eff)))
    except Exception:
        return None

def _ensure_ts_datetime_utc(df: pl.DataFrame) -> pl.DataFrame:
    """Asegura ts como Datetime[UTC]."""
    # Garantizar Datetime
    try:
        _ = df.select(pl.col("ts").dt.year).height
    except Exception:
        try:
            df = df.with_columns(pl.col("ts").str.strptime(pl.Datetime, strict=False, fmt=None))
        except Exception:
            df = df.with_columns(pl.col("ts").cast(pl.Datetime))
    # Forzar UTC
    try:
        df = df.with_columns(pl.col("ts").dt.replace_time_zone("UTC"))
    except Exception:
        try:
            df = df.with_columns(pl.col("ts").dt.convert_time_zone("UTC"))
        except Exception:
            pass
    return df

print("📖 Leyendo configuración (Celda 7b v3 FINAL TZ+RANK) ...")
need_env = ("TWF_SYMBOL","TWF_TF","TWF_START_DATE","TWF_END_DATE","TWF_SEED_BASE")
assert all(k in os.environ for k in need_env), "❌ Faltan TWF_*; ejecuta celdas previas."
SYMBOL = os.environ["TWF_SYMBOL"]; TF = os.environ["TWF_TF"]

# N_min (ajustables por env)
NMIN_GOOD = int(os.environ.get("TWF_NMIN_GOOD", "50"))
NMIN_BAD  = int(os.environ.get("TWF_NMIN_BAD",  "50"))

PROJ_ROOT   = Path(os.environ.get("TWF_BASE", Path.cwd())).resolve()
CONFIG_PATH = PROJ_ROOT / "outputs" / SYMBOL / TF / "logs" / f"config_{SYMBOL}_{TF}.json"
WF_JSON     = PROJ_ROOT / "outputs" / SYMBOL / TF / "logs" / f"wf_windows_{SYMBOL}_{TF}.json"
FWD_PARQ    = PROJ_ROOT / "outputs" / SYMBOL / TF / "forward" / f"forward_{SYMBOL}_{TF}.parquet"
OUT_PARQ    = PROJ_ROOT / "outputs" / SYMBOL / TF / "reports" / f"wf_train_scores_{SYMBOL}_{TF}.parquet"
OUT_JSON    = PROJ_ROOT / "outputs" / SYMBOL / TF / "reports" / f"wf_train_scores_{SYMBOL}_{TF}.json"

assert CONFIG_PATH.exists(), f"❌ No existe configuración: {CONFIG_PATH}"
assert WF_JSON.exists(),     f"❌ No existe ventanas WF: {WF_JSON}"
assert FWD_PARQ.exists(),    f"❌ Falta forward: {FWD_PARQ}"

print("📦 Cargando forward y ventanas ...")
CFG = json.loads(CONFIG_PATH.read_text(encoding="utf-8"))
PATHS = {k: Path(v) for k, v in CFG["paths"].items()}
PATHS["reports"].mkdir(parents=True, exist_ok=True)

fwd = pl.read_parquet(FWD_PARQ)
for c in ["ts","entry_col","label","ret_event"]:
    assert c in fwd.columns, f"❌ Falta columna en forward: {c}"
fwd = _ensure_ts_datetime_utc(fwd)
_print_kv("fwd_shape", f"{fwd.height}x{fwd.width}")
_print_kv("ts_dtype", str(fwd.schema.get("ts")))

windows = json.loads(WF_JSON.read_text(encoding="utf-8")).get("windows", [])
assert len(windows) > 0, "❌ wf_windows vacío."
wf_map = {}
for idx, w in enumerate(windows):
    wf_key = str(w.get("wf_id", f"wf_{idx:03d}"))  # STRING
    t0 = datetime.fromisoformat(w["train"]["start"].replace("Z","+00:00")).astimezone(timezone.utc)
    t1 = datetime.fromisoformat(w["train"]["end"].replace("Z","+00:00")).astimezone(timezone.utc)
    wf_map[wf_key] = (t0, t1)
_print_kv("train_windows", len(wf_map))

# ----------------------------- Cálculo por ventana/entry_col -----------------------------
print("🧮 Scoring en TRAIN por (wf_id, entry_col) ...")
rows = []
for wf_id, (t0, t1) in wf_map.items():
    sub_wf = fwd.filter((pl.col("ts") >= pl.lit(t0)) & (pl.col("ts") <= pl.lit(t1)))
    if sub_wf.is_empty():
        continue
    for entry in sub_wf.get_column("entry_col").unique().to_list():
        g = sub_wf.filter(pl.col("entry_col") == entry).select(["ts","label","ret_event"]).sort("ts")

        n  = g.height
        tp = int(g.select(((pl.col("label")==1).cast(pl.Int32)).sum().alias("__tp__")).item()) if n>0 else 0
        sl = int(g.select(((pl.col("label")==-1).cast(pl.Int32)).sum().alias("__sl__")).item()) if n>0 else 0
        to = int(g.select(((pl.col("label")==0).cast(pl.Int32)).sum().alias("__to__")).item()) if n>0 else 0

        n_pairs = tp + sl
        p_good  = (tp / n_pairs) if n_pairs > 0 else None
        ev      = float(g.select(pl.col("ret_event").mean()).item()) if n>0 else None
        ev_std  = float(g.select(pl.col("ret_event").std()).item()) if n>0 else None

        if n>0:
            pos_sum = float(g.filter(pl.col("ret_event")>0).select(pl.col("ret_event").sum()).item() or 0.0)
            neg_sum = float(g.filter(pl.col("ret_event")<0).select(pl.col("ret_event").sum()).item() or 0.0)
            pf_train = (pos_sum / abs(neg_sum)) if abs(neg_sum) > 1e-12 else None
        else:
            pf_train = None

        labels_np = g.get_column("label").to_numpy() if n>0 else np.array([], dtype=int)
        L_b_H = _estimate_block_len_train(labels_np)
        n_eff = max(1, int(round(n_pairs / max(1, L_b_H))))

        score = _score_train_fn(ev, p_good, n_eff)

        rows.append({
            "wf_id": wf_id,                 # STRING
            "entry_col": entry,
            "n_signals": n,
            "tp": tp, "sl": sl, "timeout": to,
            "support_excl_to": n_pairs,
            "hit_rate_tp_excl_to": (tp / n_pairs) if n_pairs>0 else None,
            "ret_mean": ev,
            "ret_std": ev_std,
            "profit_factor_stat": pf_train,
            "L_b_H_train": L_b_H,
            "n_eff_hit_train": n_eff,
            "score_train": score,
            "meets_Nmin": (tp >= NMIN_GOOD and sl >= NMIN_BAD)
        })

df = pl.DataFrame(rows) if rows else pl.DataFrame({"wf_id": pl.Series([], dtype=pl.Utf8)})
_print_kv("train_scores_rows", df.height)
_print_kv("train_scores_cols", df.width)

# ----------------------------- Ranking con desempates (sin GroupBy.apply) ----------------
ranked_parts = []
if df.height > 0:
    wf_ids = df.get_column("wf_id").unique().to_list()
    for wfid in wf_ids:
        part = (
            df.filter(pl.col("wf_id")==wfid)
              .with_columns(
                  p_good = pl.when(pl.col("support_excl_to")>0)
                              .then(pl.col("tp")/pl.col("support_excl_to"))
                              .otherwise(None)
              )
              .sort(
                  by=[
                      pl.col("score_train").fill_null(-1e308),
                      pl.col("p_good").fill_null(-1.0),
                      pl.col("ret_mean").fill_null(-1e308),
                      pl.col("L_b_H_train").fill_null(1e9),
                      pl.col("entry_col")
                  ],
                  descending=[True, True, True, False, False]
              )
              .with_row_count("rank_in_window_train", offset=1)
        )
        ranked_parts.append(part)

ranked = pl.concat(ranked_parts) if ranked_parts else df
_print_kv("ranked_rows", ranked.height)

# ----------------------------- Guardado -----------------------------
REPORTS_DIR = PROJ_ROOT / "outputs" / SYMBOL / TF / "reports"
REPORTS_DIR.mkdir(parents=True, exist_ok=True)

ranked.write_parquet(OUT_PARQ)
(Path(OUT_JSON)).write_text(json.dumps({
    "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
    "symbol": SYMBOL, "timeframe": TF,
    "NMIN_GOOD": NMIN_GOOD, "NMIN_BAD": NMIN_BAD,
    "block_method": "nonoverlap",
    "acf_band": "±1.96/sqrt(n_pairs)",
    "score_train_definition": "EV_train * (2*p_good-1) * log1p(n_eff_hit_train)",
    "tie_breakers": ["score_train desc","p_good desc","ret_mean desc","L_b_H_train asc","entry_col asc"],
    "rows_head": ranked.head(50).to_dicts()
}, indent=2, ensure_ascii=False), encoding="utf-8")

_print_kv("OUT_PARQ", OUT_PARQ)
_print_kv("OUT_JSON", OUT_JSON)
print("✅ Celda 7b (v3 FINAL, TZ-UTC + RANK FIX) completada.\n")


📖 Leyendo configuración (Celda 7b v3 FINAL TZ+RANK) ...
📦 Cargando forward y ventanas ...
   - fwd_shape                         : 447948x11
   - ts_dtype                          : Datetime(time_unit='us', time_zone='UTC')
   - train_windows                     : 4
🧮 Scoring en TRAIN por (wf_id, entry_col) ...
   - train_scores_rows                 : 36
   - train_scores_cols                 : 15
   - ranked_rows                       : 36
   - OUT_PARQ                          : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_train_scores_BTCUSDT_15m.parquet
   - OUT_JSON                          : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_train_scores_BTCUSDT_15m.json
✅ Celda 7b (v3 FINAL, TZ-UTC + RANK FIX) completada.



C:\Users\PC\AppData\Local\Temp\ipykernel_40828\287493429.py:210: DeprecationWarning: `DataFrame.with_row_count` is deprecated; use `with_row_index` instead. Note that the default column name has changed from 'row_nr' to 'index'.
  .with_row_count("rank_in_window_train", offset=1)


In [239]:
# Celda 7c (v3 FINAL): [SELECCIÓN & DEDUP Greedy( Jaccard ≥ 0.80 ) → wf_selected_*]
# -----------------------------------------------------------------------------------
# Ubicación: REEMPLAZA la Celda 7c actual (después de 7b y antes de 7d).
# Entradas:
#   • outputs/{SYMBOL}/{TF}/reports/wf_train_scores_{SYMBOL}_{TF}.parquet   (7b v3 FINAL)
#   • outputs/{SYMBOL}/{TF}/forward/forward_{SYMBOL}_{TF}.parquet
#   • outputs/{SYMBOL}/{TF}/logs/wf_windows_{SYMBOL}_{TF}.json
# Salidas:
#   • outputs/{SYMBOL}/{TF}/reports/wf_selected_{SYMBOL}_{TF}.parquet
#   • outputs/{SYMBOL}/{TF}/reports/wf_selected_{SYMBOL}_{TF}.json  (metadatos + resumen)
#   • outputs/{SYMBOL}/{TF}/reports/run_log_records.csv              (append)
# Contrato del plan:
#   • Ranking por ventana: score_train desc → p_good desc → EV desc → L_b_H_train asc → entry asc.
#   • Gate N_min: usar meets_Nmin==True para elegibles.
#   • Deduplicación Greedy Step-Down por Jaccard≥0.80 sobre el conjunto de señales TRAIN (tp|sl).
#   • cluster_id por ventana; cluster_size = tamaño del cluster del representante.
#   • wf_id tratado como STRING (compatible con “wf_003”).
# -----------------------------------------------------------------------------------

import os, json, csv, math
from pathlib import Path
from datetime import datetime, timezone
import polars as pl
import numpy as np

def _print_kv(k, v): print(f"   - {k:<34}: {v}")

def _ensure_ts_datetime_utc(df: pl.DataFrame) -> pl.DataFrame:
    try:
        _ = df.select(pl.col("ts").dt.year).height
    except Exception:
        try:
            df = df.with_columns(pl.col("ts").str.strptime(pl.Datetime, strict=False, fmt=None))
        except Exception:
            df = df.with_columns(pl.col("ts").cast(pl.Datetime))
    try:
        df = df.with_columns(pl.col("ts").dt.replace_time_zone("UTC"))
    except Exception:
        try:
            df = df.with_columns(pl.col("ts").dt.convert_time_zone("UTC"))
        except Exception:
            pass
    return df

print("📖 Leyendo configuración (Celda 7c v3 FINAL) ...")
need_env = ("TWF_SYMBOL","TWF_TF","TWF_START_DATE","TWF_END_DATE","TWF_SEED_BASE")
assert all(k in os.environ for k in need_env), "❌ Faltan TWF_*; ejecuta celdas previas."
SYMBOL = os.environ["TWF_SYMBOL"]; TF = os.environ["TWF_TF"]

# Umbral Jaccard configurable (default 0.80)
J_THR = float(os.environ.get("TWF_JACCARD_THR", "0.80"))

PROJ_ROOT   = Path(os.environ.get("TWF_BASE", Path.cwd())).resolve()
CONFIG_PATH = PROJ_ROOT / "outputs" / SYMBOL / TF / "logs" / f"config_{SYMBOL}_{TF}.json"
WF_JSON     = PROJ_ROOT / "outputs" / SYMBOL / TF / "logs" / f"wf_windows_{SYMBOL}_{TF}.json"
FWD_PARQ    = PROJ_ROOT / "outputs" / SYMBOL / TF / "forward" / f"forward_{SYMBOL}_{TF}.parquet"
SCORES_PARQ = PROJ_ROOT / "outputs" / SYMBOL / TF / "reports" / f"wf_train_scores_{SYMBOL}_{TF}.parquet"
SEL_PARQ    = PROJ_ROOT / "outputs" / SYMBOL / TF / "reports" / f"wf_selected_{SYMBOL}_{TF}.parquet"
SEL_JSON    = PROJ_ROOT / "outputs" / SYMBOL / TF / "reports" / f"wf_selected_{SYMBOL}_{TF}.json"
RUNLOG_CSV  = PROJ_ROOT / "outputs" / SYMBOL / TF / "reports" / "run_log_records.csv"

assert CONFIG_PATH.exists(), f"❌ No existe configuración: {CONFIG_PATH}"
assert WF_JSON.exists(),     f"❌ No existe ventanas WF: {WF_JSON}"
assert FWD_PARQ.exists(),    f"❌ Falta forward: {FWD_PARQ}"
assert SCORES_PARQ.exists(), f"❌ Falta scores 7b: {SCORES_PARQ}"

CFG = json.loads(CONFIG_PATH.read_text(encoding="utf-8"))
PATHS = {k: Path(v) for k, v in CFG["paths"].items()}
PATHS["reports"].mkdir(parents=True, exist_ok=True)

print("📦 Cargando artefactos ...")
scores = pl.read_parquet(SCORES_PARQ)
fwd    = pl.read_parquet(FWD_PARQ).select(["ts","entry_col","label"])
fwd    = _ensure_ts_datetime_utc(fwd)

# wf_id como STRING
if scores.get_column("wf_id").dtype != pl.Utf8:
    scores = scores.with_columns(pl.col("wf_id").cast(pl.Utf8))

windows = json.loads(WF_JSON.read_text(encoding="utf-8")).get("windows", [])
assert windows, "❌ wf_windows vacío."
wf_map = {}
for idx, w in enumerate(windows):
    wf_key = str(w.get("wf_id", f"wf_{idx:03d}"))
    t0 = datetime.fromisoformat(w["train"]["start"].replace("Z","+00:00")).astimezone(timezone.utc)
    t1 = datetime.fromisoformat(w["train"]["end"].replace("Z","+00:00")).astimezone(timezone.utc)
    wf_map[wf_key] = (t0, t1)

_print_kv("scores_shape", f"{scores.height}x{scores.width}")
_print_kv("fwd_shape",    f"{fwd.height}x{fwd.width}")
_print_kv("windows_count", len(wf_map))

# Campos esperados en scores
need_cols = {"wf_id","entry_col","score_train","ret_mean","L_b_H_train",
             "tp","sl","support_excl_to","meets_Nmin"}
missing = [c for c in need_cols if c not in scores.columns]
assert not missing, f"❌ Falta columnas en wf_train_scores: {missing}"

# p_good para desempate (si no viene)
if "p_good" not in scores.columns:
    scores = scores.with_columns(
        pl.when(pl.col("support_excl_to")>0)
          .then(pl.col("tp")/pl.col("support_excl_to"))
          .otherwise(None)
          .alias("p_good")
    )

# Orden canónico de ranking en TRAIN
def _sort_train(df: pl.DataFrame) -> pl.DataFrame:
    return df.sort(
        by=[
            pl.col("score_train").fill_null(-1e308),
            pl.col("p_good").fill_null(-1.0),
            pl.col("ret_mean").fill_null(-1e308),
            pl.col("L_b_H_train").fill_null(1e9),
            pl.col("entry_col")
        ],
        descending=[True, True, True, False, False]
    )

# Precalcular, por ventana TRAIN, los sets de timestamps (tp|sl) por entry_col
print("🧮 Construyendo conjuntos TRAIN por ventana (tp|sl) ...")
train_sets: dict[str, dict[str, set]] = {}
for wf_id, (t0, t1) in wf_map.items():
    sub = (
        fwd.filter( (pl.col("ts")>=pl.lit(t0)) & (pl.col("ts")<=pl.lit(t1)) & (pl.col("label")!=0) )
           .select(["entry_col","ts"])
           .unique()  # asegurar (entry, ts) únicos
    )
    by_entry = {}
    if sub.height > 0:
        for rec in sub.group_by("entry_col").agg(pl.col("ts").alias("__ts__")).to_dicts():
            by_entry[rec["entry_col"]] = set(rec["__ts__"])
    train_sets[wf_id] = by_entry

def _jaccard(A: set, B: set) -> float:
    if not A and not B:
        return 0.0
    inter = len(A & B)
    union = len(A | B)
    return inter/union if union>0 else 0.0

# Greedy Step-Down con Jaccard ≥ J_THR (por ventana)
print("🧪 Selección + dedup Greedy (Jaccard ≥ %.2f) ..." % J_THR)
kept_rows = []
drop_rows = []
clusters  = []  # (wf_id, cluster_head_entry, members)
for wf_id in scores.get_column("wf_id").unique().to_list():
    cand = scores.filter( (pl.col("wf_id")==wf_id) & (pl.col("meets_Nmin")==True) )
    if cand.is_empty():
        continue
    cand = _sort_train(cand)
    kept = []          # lista de entry_col elegidos (orden ranking)
    cluster_members = {}  # head -> set(members)
    sets_map = train_sets.get(wf_id, {})

    for rec in cand.to_dicts():
        e = rec["entry_col"]
        e_set = sets_map.get(e, set())
        drop_to_head = None
        # comparar contra representantes ya kept
        for head in kept:
            h_set = sets_map.get(head, set())
            jac = _jaccard(e_set, h_set)
            if jac >= J_THR:
                drop_to_head = head
                break
        if drop_to_head is None:
            kept.append(e)
            cluster_members.setdefault(e, set([e]))
            kept_rows.append({**rec})
        else:
            drop_rows.append({**rec, "_dropped_to": drop_to_head})

    # cluster sizes
    for head in kept:
        # agregar a su cluster los que cayeron a ese head
        members = {head}
        for r in drop_rows:
            if r.get("_dropped_to")==head and r["wf_id"]==wf_id:
                members.add(r["entry_col"])
        clusters.append((wf_id, head, members))

# Construir DataFrame final de seleccionados con cluster_id/size y rank_in_window
sel_records = []
for wf_id in scores.get_column("wf_id").unique().to_list():
    # map head->members para wf
    cl_heads = [(h, m) for (w,h,m) in clusters if w==wf_id]
    head_to_size = {h: len(m) for (h,m) in cl_heads}

    kept_df = pl.DataFrame([r for r in kept_rows if r["wf_id"]==wf_id]) if kept_rows else pl.DataFrame()
    if kept_df.height==0:
        continue
    # rank_in_window = posición entre kept (orden ya es de cand ordenado; reordenamos por la misma regla)
    kept_df = _sort_train(kept_df).with_row_index("rank_in_window", offset=1)

    # asignar cluster_id/size (head = entry_col)
    kept_df = kept_df.with_columns(
        pl.col("entry_col").map_elements(lambda x: f"{wf_id}::cluster::{x}").alias("cluster_id"),
        pl.col("entry_col").map_elements(lambda x: head_to_size.get(x, 1)).alias("cluster_size")
    )

    sel_records.append(kept_df.select(["wf_id","entry_col","rank_in_window","cluster_id","cluster_size"]))

sel_df = pl.concat(sel_records) if sel_records else pl.DataFrame(
    {"wf_id": pl.Series([], dtype=pl.Utf8),
     "entry_col": pl.Series([], dtype=pl.Utf8),
     "rank_in_window": pl.Series([], dtype=pl.Int64),
     "cluster_id": pl.Series([], dtype=pl.Utf8),
     "cluster_size": pl.Series([], dtype=pl.Int64)}
)

_print_kv("selected_rows", sel_df.height)
_print_kv("selected_cols", sel_df.width)

# Guardado
sel_df.write_parquet(SEL_PARQ)

meta = {
    "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
    "symbol": SYMBOL, "timeframe": TF,
    "dedup_method": "greedy_stepdown",
    "jaccard_thr": J_THR,
    "rank_order": ["score_train desc","p_good desc","ret_mean desc","L_b_H_train asc","entry_col asc"],
    "wf_id_type": "string",
    "notes": "Jaccard sobre TRAIN (tp|sl) por (entry_col). Se guardan sólo representantes (kept).",
    "summary": {
        "n_windows": len(wf_map),
        "n_kept_total": int(sel_df.height),
        "n_dropped_total": int(len(drop_rows))
    },
    "dropped_examples_head": drop_rows[:20]
}
SEL_JSON.write_text(json.dumps(meta, indent=2, ensure_ascii=False), encoding="utf-8")
_print_kv("SEL_PARQ", SEL_PARQ)
_print_kv("SEL_JSON", SEL_JSON)

# run_log_records append
RUNLOG_CSV.parent.mkdir(parents=True, exist_ok=True)
header = ["created_utc","symbol","tf","cell","rows","block_method","B","comment"]
append_header = not RUNLOG_CSV.exists()
with RUNLOG_CSV.open("a", newline="", encoding="utf-8") as fh:
    w = csv.writer(fh)
    if append_header: w.writerow(header)
    w.writerow([
        datetime.now(timezone.utc).isoformat(timespec="seconds"),
        SYMBOL, TF, "7c_v3_final_greedy_0p80",
        sel_df.height, "", "",
        f"Greedy step-down; J={J_THR:.2f}; kept={sel_df.height}; dropped={len(drop_rows)}"
    ])

print("✅ Celda 7c (v3 FINAL) completada.\n")


📖 Leyendo configuración (Celda 7c v3 FINAL) ...
📦 Cargando artefactos ...


   - scores_shape                      : 36x17
   - fwd_shape                         : 447948x3
   - windows_count                     : 4
🧮 Construyendo conjuntos TRAIN por ventana (tp|sl) ...
🧪 Selección + dedup Greedy (Jaccard ≥ 0.80) ...
   - selected_rows                     : 36
   - selected_cols                     : 5
   - SEL_PARQ                          : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_selected_BTCUSDT_15m.parquet
   - SEL_JSON                          : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_selected_BTCUSDT_15m.json
✅ Celda 7c (v3 FINAL) completada.



In [240]:
# Celda 7c_bis_pbo (FIX v2): [PBO QUICK-CSCV APROX → reports/wf_pbo_*.json]
# -------------------------------------------------------------------------------------------------
# - Entrada: forward_{..}.parquet, wf_windows_{..}.json, wf_train_scores_{..}.parquet (7b v2).
# - Para cada ventana:
#     1) rank TRAIN por métrica efectiva (respeta sort_metric del config si existe; si no, fallback).
#     2) en TEST: ret_mean por entry_col de TODOS los candidatos de TRAIN.
#     3) Percentil del top-1 TRAIN dentro de la dist. TEST. Si ≤ 0.5 ⇒ "overfit".
# - PBO = proporción de ventanas marcadas como overfit (solo sobre ventanas "usables").
# - Salida: reports/wf_pbo_{SYMBOL}_{TF}.json   (JSON 100% serializable con tipos nativos)
# - Logs: impresiones detalladas por ventana.
# -------------------------------------------------------------------------------------------------

import os, json
from pathlib import Path
from datetime import datetime, timezone
import polars as pl

def _print_kv(k, v): 
    print(f"   - {k:<30}: {v}")

def _parse_iso(ts_str: str):
    return datetime.fromisoformat(ts_str.replace("Z", "+00:00")).astimezone(timezone.utc)

print("📖 Leyendo configuración ...")
need_env = ("TWF_SYMBOL","TWF_TF","TWF_START_DATE","TWF_END_DATE","TWF_SEED_BASE")
assert all(k in os.environ for k in need_env), "❌ Faltan TWF_*."
SYMBOL = os.environ["TWF_SYMBOL"]; TF = os.environ["TWF_TF"]

PROJ_ROOT   = Path(os.environ.get("TWF_BASE", Path.cwd())).resolve()
CFG         = json.loads((PROJ_ROOT / "outputs" / SYMBOL / TF / "logs" / f"config_{SYMBOL}_{TF}.json").read_text(encoding="utf-8"))
PATHS         = {k: Path(v) for k, v in CFG["paths"].items()}

FORWARD_PARQ  = PATHS["forward"] / f"forward_{SYMBOL}_{TF}.parquet"
WF_JSON       = PATHS["logs"]    / f"wf_windows_{SYMBOL}_{TF}.json"
SCORES_PARQ   = PATHS["reports"] / f"wf_train_scores_{SYMBOL}_{TF}.parquet"
PBO_JSON      = PATHS["reports"] / f"wf_pbo_{SYMBOL}_{TF}.json"
PATHS["reports"].mkdir(parents=True, exist_ok=True)

assert FORWARD_PARQ.exists(), f"❌ Falta {FORWARD_PARQ}"
assert WF_JSON.exists(),      f"❌ Falta {WF_JSON}"
assert SCORES_PARQ.exists(),  f"❌ Falta {SCORES_PARQ}"

SEL_CFG = (CFG.get("wf", {}).get("selection", {}) or CFG.get("selection", {}) or CFG.get("wf_selection", {}) or {})
MIN_TRAIN_SIGNALS = int(SEL_CFG.get("min_train_signals", 1000))
SORT_MET_REQ      = str(SEL_CFG.get("sort_metric", "profit_factor_stat"))

_print_kv("FORWARD_PARQ", FORWARD_PARQ)
_print_kv("WF_JSON", WF_JSON)
_print_kv("SCORES_PARQ", SCORES_PARQ)
_print_kv("PBO_JSON", PBO_JSON)
_print_kv("min_train_signals", MIN_TRAIN_SIGNALS)
_print_kv("sort_metric_requested", SORT_MET_REQ)

# Cargas
fwd = pl.read_parquet(FORWARD_PARQ)
for c in ["ts","entry_col","label","ret_event"]:
    assert c in fwd.columns, f"❌ Falta {c} en forward."

windows = json.loads(WF_JSON.read_text(encoding="utf-8")).get("windows", [])
train   = pl.read_parquet(SCORES_PARQ)

def _resolve_sort_metric(cols: list[str], requested: str) -> str:
    if requested in cols: 
        return requested
    for cand in ("score_train", "profit_factor_stat", "ret_mean"):
        if cand in cols:
            print(f"   ⚠️ sort_metric '{requested}' no existe; usando fallback '{cand}'.")
            return cand
    raise AssertionError("❌ No hay métrica válida para ranking (falta score_train/profit_factor_stat/ret_mean).")

print("🧮 Calculando PBO por ventana ...")
details = []
overfit_flags = []
usable = 0

for w in windows:
    wid = w["id"]
    sc_win = train.filter(pl.col("wf_id")==wid)
    if "n_signals" in sc_win.columns:
        sc_win = sc_win.filter(pl.col("n_signals") >= MIN_TRAIN_SIGNALS)
    _print_kv(f"[{wid}] train_candidates_after_minN", sc_win.height)
    if sc_win.height == 0:
        details.append({"wf_id": str(wid), "note": "no_train_candidates"}); 
        continue

    sort_met = _resolve_sort_metric(sc_win.columns, SORT_MET_REQ)
    sc_win = sc_win.sort([sort_met], descending=[True])

    top1 = sc_win["entry_col"][0]
    _print_kv(f"[{wid}] sort_metric_used", sort_met)
    _print_kv(f"[{wid}] top1_train", top1)

    # Subset TEST
    t0 = _parse_iso(w["test"]["start"]); t1 = _parse_iso(w["test"]["end"])
    sub = fwd.filter((pl.col("ts") >= t0) & (pl.col("ts") <= t1))
    _print_kv(f"[{wid}] test_rows", sub.height)
    if sub.height == 0:
        details.append({"wf_id": str(wid), "top_train": str(top1), "note": "no_test_rows"}); 
        continue

    cand_list = sc_win["entry_col"].to_list()
    kpi_test = (
        sub.filter(pl.col("entry_col").is_in(cand_list))
           .group_by("entry_col")
           .agg(pl.len().alias("n"), pl.col("ret_event").mean().alias("ret_mean"))
    )
    _print_kv(f"[{wid}] test_candidates_measured", kpi_test.height)
    if kpi_test.height == 0:
        details.append({"wf_id": str(wid), "top_train": str(top1), "note": "no_test_metrics"}); 
        continue

    # Dist TEST (tipos nativos)
    dist_vals = [float(x) for x in kpi_test["ret_mean"].to_list()]
    top_vals  = kpi_test.filter(pl.col("entry_col")==top1)["ret_mean"].to_list()
    if len(top_vals)==0:
        details.append({"wf_id": str(wid), "top_train": str(top1), "note": "top1_missing_in_test"}); 
        continue

    top_val = float(top_vals[0])
    pct = float(sum(1 for v in dist_vals if v <= top_val) / max(len(dist_vals), 1))
    overfit = bool(pct <= 0.5)

    usable += 1
    overfit_flags.append(1 if overfit else 0)
    details.append({
        "wf_id": str(wid), 
        "top_train": str(top1), 
        "sort_metric": str(sort_met),
        "test_percentile": float(pct), 
        "overfit": bool(overfit),
        "n_test_candidates": int(len(dist_vals))
    })
    _print_kv(f"[{wid}] test_percentile(top1)", pct)
    _print_kv(f"[{wid}] overfit", overfit)

PBO = (sum(overfit_flags) / usable) if usable > 0 else None
_print_kv("windows_usable", usable)
_print_kv("PBO", PBO if PBO is not None else "N/A")

print("💾 Guardando wf_pbo ...")
PBO_JSON.write_text(json.dumps({
    "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
    "symbol": str(SYMBOL), 
    "timeframe": str(TF),
    "min_train_signals": int(MIN_TRAIN_SIGNALS),
    "sort_metric_requested": str(SORT_MET_REQ),
    "windows_usable": int(usable),
    "PBO": (float(PBO) if PBO is not None else None),
    "details": details
}, indent=2, ensure_ascii=False), encoding="utf-8")

_print_kv("PBO_JSON", PBO_JSON)
print("✅ Celda 7c_bis_pbo finalizada sin errores.\n")


📖 Leyendo configuración ...
   - FORWARD_PARQ                  : C:\Quant\TWF\outputs\BTCUSDT\15m\forward\forward_BTCUSDT_15m.parquet
   - WF_JSON                       : C:\Quant\TWF\outputs\BTCUSDT\15m\logs\wf_windows_BTCUSDT_15m.json
   - SCORES_PARQ                   : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_train_scores_BTCUSDT_15m.parquet
   - PBO_JSON                      : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_pbo_BTCUSDT_15m.json
   - min_train_signals             : 1000
   - sort_metric_requested         : profit_factor_stat
🧮 Calculando PBO por ventana ...
   - [wf_000] train_candidates_after_minN: 8
   - [wf_000] sort_metric_used     : profit_factor_stat
   - [wf_000] top1_train           : entry_er10_ge_0p4
   - [wf_000] test_rows            : 7144
   - [wf_000] test_candidates_measured: 8
   - [wf_000] test_percentile(top1): 0.625
   - [wf_000] overfit              : False
   - [wf_001] train_candidates_after_minN: 9
   - [wf_001] sort_metric_used     : profit_f

In [241]:
# Celda 7c_bis_ds (v1.1 FIX): [DATASET SHIFT por ventana → reports/wf_ds_*.json]
# -------------------------------------------------------------------------------------------------
# FIXES:
#  - Evita pivot() sin índice: conteos de label via dict por compresión (robusto y sin deprecations).
#  - PSI/KS: bordes estrictamente crecientes con np.nextafter para evitar fallos si hay cuantiles iguales.
# -------------------------------------------------------------------------------------------------

import os, json, math
from pathlib import Path
from datetime import datetime, timezone
import polars as pl
import numpy as np

def _print_kv(k, v): 
    print(f"   - {k:<30}: {v}")

def _parse_iso(ts_str: str):
    return datetime.fromisoformat(ts_str.replace("Z", "+00:00")).astimezone(timezone.utc)

def _safe_div(a: float, b: float, eps: float = 1e-12) -> float:
    return float(a) / float(b if abs(b) > eps else eps)

def _make_edges_from_quantiles(q_edges: list[float]) -> list[float]:
    """Construye [-inf, q..., +inf] y fuerza monotonicidad estricta (maneja cuantiles repetidos)."""
    edges = [-np.inf] + [float(x) for x in q_edges] + [np.inf]
    cleaned = [edges[0]]
    for x in edges[1:]:
        if x <= cleaned[-1]:
            x = float(np.nextafter(cleaned[-1], np.inf))
        cleaned.append(x)
    return cleaned

def _psi_from_bins(train_vals: np.ndarray, test_vals: np.ndarray, q_edges: list[float]) -> float:
    edges = _make_edges_from_quantiles(q_edges)
    tr_hist, _ = np.histogram(train_vals, bins=edges)
    te_hist, _ = np.histogram(test_vals,  bins=edges)
    tr_p = tr_hist / max(tr_hist.sum(), 1)
    te_p = te_hist / max(te_hist.sum(), 1)
    tr_p = np.where(tr_p<=0, 1e-12, tr_p)
    te_p = np.where(te_p<=0, 1e-12, te_p)
    return float(np.sum((tr_p - te_p) * np.log(tr_p / te_p)))

def _ks_from_bins(train_vals: np.ndarray, test_vals: np.ndarray, q_edges: list[float]) -> float:
    edges = _make_edges_from_quantiles(q_edges)
    tr_hist, _ = np.histogram(train_vals, bins=edges)
    te_hist, _ = np.histogram(test_vals,  bins=edges)
    tr_cdf = np.cumsum(tr_hist) / max(tr_hist.sum(), 1)
    te_cdf = np.cumsum(te_hist) / max(te_hist.sum(), 1)
    return float(np.max(np.abs(tr_cdf - te_cdf)))

print("📖 Leyendo configuración ...")
need_env = ("TWF_SYMBOL","TWF_TF","TWF_START_DATE","TWF_END_DATE","TWF_SEED_BASE")
assert all(k in os.environ for k in need_env), "❌ Faltan TWF_*."
SYMBOL = os.environ["TWF_SYMBOL"]; TF = os.environ["TWF_TF"]

PROJ_ROOT   = Path(os.environ.get("TWF_BASE", Path.cwd())).resolve()
CFG         = json.loads((PROJ_ROOT / "outputs" / SYMBOL / TF / "logs" / f"config_{SYMBOL}_{TF}.json").read_text(encoding="utf-8"))
PATHS         = {k: Path(v) for k, v in CFG["paths"].items()}

FORWARD_PARQ  = PATHS["forward"] / f"forward_{SYMBOL}_{TF}.parquet"
WF_JSON       = PATHS["logs"]    / f"wf_windows_{SYMBOL}_{TF}.json"
DS_JSON       = PATHS["reports"] / f"wf_ds_{SYMBOL}_{TF}.json"
PATHS["reports"].mkdir(parents=True, exist_ok=True)

assert FORWARD_PARQ.exists(), f"❌ Falta {FORWARD_PARQ}"
assert WF_JSON.exists(),      f"❌ Falta {WF_JSON}"

_print_kv("FORWARD_PARQ", FORWARD_PARQ)
_print_kv("WF_JSON", WF_JSON)
_print_kv("DS_JSON", DS_JSON)

fwd = pl.read_parquet(FORWARD_PARQ)
for c in ["ts","entry_col","label","ret_event"]:
    assert c in fwd.columns, f"❌ Falta {c} en forward."

windows = json.loads(WF_JSON.read_text(encoding="utf-8")).get("windows", [])

print("🧮 Dataset shift por ventana ...")
win_rows = []

for w in windows:
    wid = w["id"]
    t0_tr = _parse_iso(w["train"]["start"]); t1_tr = _parse_iso(w["train"]["end"])
    t0_te = _parse_iso(w["test"]["start"]);  t1_te = _parse_iso(w["test"]["end"])

    tr = fwd.filter((pl.col("ts") >= t0_tr) & (pl.col("ts") <= t1_tr))
    te = fwd.filter((pl.col("ts") >= t0_te) & (pl.col("ts") <= t1_te))
    _print_kv(f"[{wid}] n_train", tr.height)
    _print_kv(f"[{wid}] n_test",  te.height)

    if tr.height==0 or te.height==0:
        win_rows.append({
            "wf_id": str(wid), "note": "empty_train_or_test",
            "n_train": int(tr.height), "n_test": int(te.height),
            "psi_ret_event": None, "ks_ret_event": None, 
            "delta_hit_excl": None, "drift_flag": None
        })
        continue

    # Conteos por label (SIN pivot): dict por comprensión → robusto
    tr_lbl = tr.group_by("label").agg(pl.len().alias("n")).to_dicts()
    te_lbl = te.group_by("label").agg(pl.len().alias("n")).to_dicts()
    tr_counts = { int(row["label"]): int(row["n"]) for row in tr_lbl }
    te_counts = { int(row["label"]): int(row["n"]) for row in te_lbl }

    tr_tp = tr_counts.get(1, 0);   tr_sl = tr_counts.get(-1, 0);  tr_to = tr_counts.get(0, 0)
    te_tp = te_counts.get(1, 0);   te_sl = te_counts.get(-1, 0);  te_to = te_counts.get(0, 0)

    tr_den = tr_tp + tr_sl
    te_den = te_tp + te_sl
    tr_hit_excl = (_safe_div(tr_tp, tr_den) if tr_den > 0 else None)
    te_hit_excl = (_safe_div(te_tp, te_den) if te_den > 0 else None)
    delta_hit_excl = (float(te_hit_excl - tr_hit_excl) if (tr_hit_excl is not None and te_hit_excl is not None) else None)

    # PSI / KS de ret_event (bins por cuantiles de TRAIN)
    qdf = tr.select([
        pl.col("ret_event").quantile(0.05).alias("q05"),
        pl.col("ret_event").quantile(0.25).alias("q25"),
        pl.col("ret_event").quantile(0.50).alias("q50"),
        pl.col("ret_event").quantile(0.75).alias("q75"),
        pl.col("ret_event").quantile(0.95).alias("q95"),
    ])
    qrow = qdf.to_dicts()[0]
    q_edges = [float(qrow["q05"]), float(qrow["q25"]), float(qrow["q50"]), float(qrow["q75"]), float(qrow["q95"])]

    tr_vals = np.asarray(tr["ret_event"].to_numpy(), dtype=float)
    te_vals = np.asarray(te["ret_event"].to_numpy(), dtype=float)
    psi_val = _psi_from_bins(tr_vals, te_vals, q_edges)
    ks_val  = _ks_from_bins(tr_vals, te_vals, q_edges)

    # Reglas de flag
    drift_flag = bool( (psi_val > 0.25) or (ks_val > 0.10) or ((delta_hit_excl is not None) and (abs(delta_hit_excl) > 0.05)) )

    _print_kv(f"[{wid}] tr_hit_excl", tr_hit_excl if tr_hit_excl is not None else "N/A")
    _print_kv(f"[{wid}] te_hit_excl", te_hit_excl if te_hit_excl is not None else "N/A")
    _print_kv(f"[{wid}] delta_hit_excl", delta_hit_excl if delta_hit_excl is not None else "N/A")
    _print_kv(f"[{wid}] PSI(ret_event)", psi_val)
    _print_kv(f"[{wid}] KS(ret_event)", ks_val)
    _print_kv(f"[{wid}] drift_flag", drift_flag)

    win_rows.append({
        "wf_id": str(wid),
        "n_train": int(tr.height), "n_test": int(te.height),
        "train_label_counts": {"tp": int(tr_tp), "sl": int(tr_sl), "timeout": int(tr_to)},
        "test_label_counts":  {"tp": int(te_tp), "sl": int(te_sl), "timeout": int(te_to)},
        "tr_hit_excl": (float(tr_hit_excl) if tr_hit_excl is not None else None),
        "te_hit_excl": (float(te_hit_excl) if te_hit_excl is not None else None),
        "delta_hit_excl": (float(delta_hit_excl) if delta_hit_excl is not None else None),
        "psi_ret_event": float(psi_val),
        "ks_ret_event": float(ks_val),
        "drift_flag": bool(drift_flag)
    })

# Resumen global
drift_count = sum(1 for r in win_rows if (r.get("drift_flag") is True))
drift_rate  = (float(drift_count) / float(len(win_rows))) if len(win_rows)>0 else None
_print_kv("drift_windows", drift_count)
_print_kv("drift_rate", drift_rate if drift_rate is not None else "N/A")

print("💾 Guardando wf_ds ...")
DS_JSON.write_text(json.dumps({
    "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
    "symbol": str(SYMBOL), 
    "timeframe": str(TF),
    "rules": {
        "psi_gt": 0.25,
        "ks_gt": 0.10,
        "abs_delta_hit_excl_gt": 0.05
    },
    "drift_windows": int(drift_count),
    "drift_rate": (float(drift_rate) if drift_rate is not None else None),
    "windows": win_rows
}, indent=2, ensure_ascii=False), encoding="utf-8")

_print_kv("DS_JSON", DS_JSON)
print("✅ Celda 7c_bis_ds finalizada sin errores.\n")


📖 Leyendo configuración ...
   - FORWARD_PARQ                  : C:\Quant\TWF\outputs\BTCUSDT\15m\forward\forward_BTCUSDT_15m.parquet
   - WF_JSON                       : C:\Quant\TWF\outputs\BTCUSDT\15m\logs\wf_windows_BTCUSDT_15m.json
   - DS_JSON                       : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_ds_BTCUSDT_15m.json
🧮 Dataset shift por ventana ...
   - [wf_000] n_train              : 95928
   - [wf_000] n_test               : 7144
   - [wf_000] tr_hit_excl          : 0.3124625429770053
   - [wf_000] te_hit_excl          : 0.27262459409854584
   - [wf_000] delta_hit_excl       : -0.039837948878459484
   - [wf_000] PSI(ret_event)       : 0.10705857356008333
   - [wf_000] KS(ret_event)        : 0.08471772362306451
   - [wf_000] drift_flag           : False
   - [wf_001] n_train              : 93625
   - [wf_001] n_test               : 7637
   - [wf_001] tr_hit_excl          : 0.3347800548231301
   - [wf_001] te_hit_excl          : 0.3209909429941396
   - [wf_001] delta

In [242]:
# Celda 7d (v3 FINAL, TZ-UTC + wf_id string): [EVALUACIÓN EN TEST ROBUSTA → reports/wf_select_eval_*]
# ---------------------------------------------------------------------------------------------------
# Ubicación: REEMPLAZA la Celda 7d actual (después de 7c y antes de 8b).
# Entradas:
#   • outputs/{SYMBOL}/{TF}/reports/wf_selected_{SYMBOL}_{TF}.parquet   (7c v3 FINAL)
#   • outputs/{SYMBOL}/{TF}/forward/forward_{SYMBOL}_{TF}.parquet       (eventos)
#   • outputs/{SYMBOL}/{TF}/logs/wf_windows_{SYMBOL}_{TF}.json          (ventanas WF)
# Salidas:
#   • outputs/{SYMBOL}/{TF}/reports/wf_select_eval_{SYMBOL}_{TF}.parquet / .json
#   • outputs/{SYMBOL}/{TF}/reports/run_log_records.csv (append)
# Método:
#   • L_b_H_test (ACF: dos lags consecutivos en banda ±1.96/√n_pairs), n_eff = max(1, n_pairs/L_b_H_test).
#   • Bootstrap por bloques no solapados (B=2000) para EV (ret_mean).
#   • Intervalo de Wilson para p_obs (hit-rate excluyendo timeouts).
#   • Profit Factor estimado desde ret_event (si hay pérdidas).
# ---------------------------------------------------------------------------------------------------

import os, json, math, csv
from pathlib import Path
from datetime import datetime, timezone
import polars as pl
import numpy as np

def _print_kv(k, v): print(f"   - {k:<32}: {v}")

def _ensure_ts_datetime_utc(df: pl.DataFrame) -> pl.DataFrame:
    # fuerza ts a Datetime[μs, UTC]
    try:
        _ = df.select(pl.col("ts").dt.year).height
    except Exception:
        try:
            df = df.with_columns(pl.col("ts").str.strptime(pl.Datetime, strict=False, fmt=None))
        except Exception:
            df = df.with_columns(pl.col("ts").cast(pl.Datetime))
    # añadir/convertir zona
    try:
        df = df.with_columns(pl.col("ts").dt.replace_time_zone("UTC"))
    except Exception:
        try:
            df = df.with_columns(pl.col("ts").dt.convert_time_zone("UTC"))
        except Exception:
            pass
    return df

def _z_acf_band(n: int) -> float:
    n = max(1, int(n))
    return 1.96 / math.sqrt(n)

def _acf(x: np.ndarray, max_lag: int) -> np.ndarray:
    x = np.asarray(x, dtype=float)
    x = x - np.nanmean(x)
    x[np.isnan(x)] = 0.0
    n = len(x)
    if n == 0: return np.zeros(max_lag+1)
    denom = np.dot(x, x)
    if denom <= 0: return np.zeros(max_lag+1)
    r = np.empty(max_lag+1, dtype=float)
    for k in range(max_lag+1):
        r[k] = np.dot(x[:n-k], x[k:]) / denom
    return r

def _estimate_block_len(labels_vec: np.ndarray) -> int:
    # serie y = +1 (tp), -1 (sl), 0 (timeout); usa N_pairs para la banda
    if labels_vec.size == 0:
        return 1
    y = np.zeros_like(labels_vec, dtype=float)
    y[labels_vec == 1]  = 1.0
    y[labels_vec == -1] = -1.0
    n_pairs = int(np.sum(labels_vec != 0))
    band = _z_acf_band(max(n_pairs, 1))
    max_lag = max(10, min(200, int(len(y) * 0.2)))
    r = _acf(y, max_lag)
    for H in range(1, max_lag-1):
        if abs(r[H]) <= band and abs(r[H+1]) <= band:
            return max(1, H)
    return max(1, int(np.ceil(np.sqrt(max(n_pairs, 1)))))  # fallback conservador

def _wilson_interval(tp: int, n: int, z: float = 1.96):
    n = int(max(0, n))
    if n == 0: return (None, None, None)
    p = tp / n
    denom = 1 + z*z/n
    center = (p + z*z/(2*n)) / denom
    half = z * math.sqrt((p*(1-p) + z*z/(4*n)) / n) / denom
    return (p, max(0.0, center - half), min(1.0, center + half))

def _block_bootstrap_mean(ret: np.ndarray, block_len: int, B: int = 2000, rng: np.random.Generator | None = None):
    n = len(ret)
    if n == 0: return (None, None, None, 0.0)
    if block_len <= 0: block_len = 1
    k = max(1, int(math.ceil(n / block_len)))
    starts = np.arange(0, max(1, n - block_len + 1))  # inicios de bloques no solapados
    if starts.size == 0: starts = np.array([0], dtype=int)
    rng = rng or np.random.default_rng(123)
    boots = np.empty(B, dtype=float); ok = 0
    for b in range(B):
        idxs = rng.choice(starts, size=k, replace=True)
        sample = []
        for s in idxs:
            sample.extend(ret[s:s+block_len])
        if len(sample) >= 1:
            sample = np.asarray(sample[:n], dtype=float)
            boots[b] = np.nanmean(sample); ok += 1
        else:
            boots[b] = np.nan
    mean = float(np.nanmean(boots)) if ok>0 else None
    ci_lo = float(np.nanpercentile(boots, 2.5)) if ok>0 else None
    ci_hi = float(np.nanpercentile(boots, 97.5)) if ok>0 else None
    return (mean, ci_lo, ci_hi, ok / B * 100.0)

print("📖 Leyendo configuración (Celda 7d v3 FINAL) ...")
need_env = ("TWF_SYMBOL","TWF_TF","TWF_START_DATE","TWF_END_DATE","TWF_SEED_BASE")
assert all(k in os.environ for k in need_env), "❌ Faltan TWF_*; ejecuta celdas previas."
SYMBOL = os.environ["TWF_SYMBOL"]; TF = os.environ["TWF_TF"]; SEED_BASE = int(os.environ["TWF_SEED_BASE"])

PROJ_ROOT   = Path(os.environ.get("TWF_BASE", Path.cwd())).resolve()
CONFIG_PATH = PROJ_ROOT / "outputs" / SYMBOL / TF / "logs" / f"config_{SYMBOL}_{TF}.json"
assert CONFIG_PATH.exists(), f"❌ No existe configuración: {CONFIG_PATH}"

CFG = json.loads(CONFIG_PATH.read_text(encoding="utf-8"))
PATHS = {k: Path(v) for k, v in CFG["paths"].items()}
PATHS["reports"].mkdir(parents=True, exist_ok=True)

SEL_PARQ   = PATHS["reports"] / f"wf_selected_{SYMBOL}_{TF}.parquet"
FWD_PARQ   = PATHS["forward"] / f"forward_{SYMBOL}_{TF}.parquet"
WF_JSON    = PATHS["logs"]    / f"wf_windows_{SYMBOL}_{TF}.json"
OUT_PARQ   = PATHS["reports"] / f"wf_select_eval_{SYMBOL}_{TF}.parquet"
OUT_JSON   = PATHS["reports"] / f"wf_select_eval_{SYMBOL}_{TF}.json"
RUNLOG_CSV = PATHS["reports"] / "run_log_records.csv"

_print_kv("SEL_PARQ", SEL_PARQ); _print_kv("FWD_PARQ", FWD_PARQ); _print_kv("WF_JSON", WF_JSON)
assert SEL_PARQ.exists(), "❌ Falta wf_selected"
assert FWD_PARQ.exists(), "❌ Falta forward"
assert WF_JSON.exists(),  "❌ Falta wf_windows"

sel = pl.read_parquet(SEL_PARQ).select(["wf_id","entry_col","rank_in_window","cluster_id","cluster_size"]).unique()
# normaliza wf_id a string
if sel.get_column("wf_id").dtype != pl.Utf8:
    sel = sel.with_columns(pl.col("wf_id").cast(pl.Utf8))

fwd = pl.read_parquet(FWD_PARQ).select(["ts","entry_col","label","ret_event"])
fwd = _ensure_ts_datetime_utc(fwd)

for c in ["ts","entry_col","label","ret_event"]:
    assert c in fwd.columns, f"❌ Falta columna en forward: {c}"

_print_kv("sel_shape", f"{sel.height}x{sel.width}")
_print_kv("fwd_shape", f"{fwd.height}x{fwd.width}")

# ventanas TEST → claves string
windows = json.loads(WF_JSON.read_text(encoding="utf-8")).get("windows", [])
wf_map: dict[str, tuple[datetime, datetime]] = {}
for idx, w in enumerate(windows):
    wf_key = str(w.get("wf_id", f"wf_{idx:03d}"))
    t0 = datetime.fromisoformat(w["test"]["start"].replace("Z","+00:00")).astimezone(timezone.utc)
    t1 = datetime.fromisoformat(w["test"]["end"].replace("Z","+00:00")).astimezone(timezone.utc)
    wf_map[wf_key] = (t0, t1)
_print_kv("windows_count", len(wf_map))

rows = []
rng_global = np.random.default_rng(SEED_BASE)

print("🧮 Evaluando TEST con bootstrap por bloques ...")
for r in sel.to_dicts():
    wf_id = str(r["wf_id"])
    entry = r["entry_col"]
    if wf_id not in wf_map:
        print(f"   ⚠️ wf_id {wf_id} no encontrado en wf_windows; se salta.")
        continue
    t0, t1 = wf_map[wf_id]

    sub = (
        fwd.filter((pl.col("entry_col")==entry) & (pl.col("ts")>=pl.lit(t0)) & (pl.col("ts")<=pl.lit(t1)))
           .select(["ts","label","ret_event"])
           .sort("ts")
    )

    n = sub.height
    tp = int(sub.select(((pl.col("label")==1).cast(pl.Int32)).sum().alias("__tp__")).item()) if n>0 else 0
    sl = int(sub.select(((pl.col("label")==-1).cast(pl.Int32)).sum().alias("__sl__")).item()) if n>0 else 0
    timeout = int(sub.select(((pl.col("label")==0).cast(pl.Int32)).sum().alias("__to__")).item()) if n>0 else 0
    ret_mean = float(sub.select(pl.col("ret_event").mean()).item()) if n>0 else None
    ret_std  = float(sub.select(pl.col("ret_event").std()).item()) if n>0 else None

    # Profit Factor
    if n>0:
        pos_sum = float(sub.filter(pl.col("ret_event")>0).select(pl.col("ret_event").sum()).item() or 0.0)
        neg_sum = float(sub.filter(pl.col("ret_event")<0).select(pl.col("ret_event").sum()).item() or 0.0)
        test_pf = (pos_sum / abs(neg_sum)) if abs(neg_sum) > 1e-12 else None
    else:
        test_pf = None

    labels_np = sub.get_column("label").to_numpy() if n>0 else np.array([], dtype=int)
    n_pairs = int(tp + sl)
    L_b_H = _estimate_block_len(labels_np)
    n_eff = max(1, int(round(n_pairs / max(1, L_b_H))))

    p_obs, p_lo, p_hi = _wilson_interval(tp, n_pairs)

    ret_np = sub.get_column("ret_event").to_numpy() if n>0 else np.array([], dtype=float)
    mean_b, ci_lo, ci_hi, ok_pct = _block_bootstrap_mean(
        ret_np, block_len=L_b_H, B=2000, rng=np.random.default_rng(SEED_BASE + (abs(hash(wf_id)) % 10_000))
    )

    rows.append({
        "wf_id": wf_id,
        "entry_col": entry,
        "rank_in_window": r.get("rank_in_window"),
        "cluster_id": r.get("cluster_id"),
        "cluster_size": r.get("cluster_size"),
        "test_n": n,
        "tp": tp,
        "sl": sl,
        "timeout": timeout,
        "hit_rate_tp_excl_to": (tp / n_pairs) if n_pairs>0 else None,
        "ret_mean": ret_mean,
        "ret_std": ret_std,
        "profit_factor_stat": test_pf,
        "L_b_H_test": L_b_H,
        "n_eff_hit_test": n_eff,
        "ev_boot_mean": mean_b,
        "ev_boot_ci_lo": ci_lo,
        "ev_boot_ci_hi": ci_hi,
        "p_obs_wilson": p_obs,
        "p_obs_ci_lo": p_lo,
        "p_obs_ci_hi": p_hi,
        "boot_ok_pct": ok_pct,
    })

eval_df = pl.DataFrame(rows) if rows else pl.DataFrame({"wf_id": pl.Series([], dtype=pl.Utf8)})
_print_kv("eval_rows", eval_df.height); _print_kv("eval_cols", eval_df.width)

print("💾 Guardando wf_select_eval ...")
eval_df.write_parquet(OUT_PARQ)
OUT_JSON.write_text(json.dumps({
    "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
    "symbol": SYMBOL, "timeframe": TF,
    "block_method": "nonoverlap", "block_bootstrap_B": 2000,
    "acf_band": "±1.96/sqrt(n_pairs)",
    "notes": "Wilson para p_obs; EV block-bootstrap; n_eff = n_pairs/L_b_H_test.",
    "rows_head": eval_df.head(50).to_dicts()
}, indent=2, ensure_ascii=False), encoding="utf-8")

RUNLOG_CSV.parent.mkdir(parents=True, exist_ok=True)
header = ["created_utc","symbol","tf","cell","rows","block_method","B","comment"]
append_header = not RUNLOG_CSV.exists()
with RUNLOG_CSV.open("a", newline="", encoding="utf-8") as fh:
    w = csv.writer(fh)
    if append_header: w.writerow(header)
    w.writerow([
        datetime.now(timezone.utc).isoformat(timespec="seconds"),
        SYMBOL, TF, "7d_v3_final", eval_df.height, "nonoverlap", 2000,
        "eval TEST robusto (Wilson + block-bootstrap, wf_id string)"
    ])

_print_kv("OUT_PARQ", OUT_PARQ); _print_kv("OUT_JSON", OUT_JSON)
print("✅ Celda 7d (v3 FINAL) completada.\n")
 

📖 Leyendo configuración (Celda 7d v3 FINAL) ...
   - SEL_PARQ                        : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_selected_BTCUSDT_15m.parquet
   - FWD_PARQ                        : C:\Quant\TWF\outputs\BTCUSDT\15m\forward\forward_BTCUSDT_15m.parquet
   - WF_JSON                         : C:\Quant\TWF\outputs\BTCUSDT\15m\logs\wf_windows_BTCUSDT_15m.json
   - sel_shape                       : 36x5
   - fwd_shape                       : 447948x4
   - windows_count                   : 4
🧮 Evaluando TEST con bootstrap por bloques ...
   - eval_rows                       : 36
   - eval_cols                       : 22
💾 Guardando wf_select_eval ...
   - OUT_PARQ                        : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_select_eval_BTCUSDT_15m.parquet
   - OUT_JSON                        : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_select_eval_BTCUSDT_15m.json
✅ Celda 7d (v3 FINAL) completada.



In [243]:
# Celda 7e (v1.1 FINAL): [DECAY TRAIN→TEST DE LOS SELECCIONADOS → reports/wf_decay_*]
# ------------------------------------------------------------------------------------
# Ubicación: OPCIONAL. Colócala después de 7d y antes de 8b (no bloquea 9a/9b/9c).
# Cambios clave (FIX):
#   - wf_id como string.
#   - Coalesce seguro de contadores: train_n ← (train_n | n_signals), test_n ← (test_n | n_signals).
#   - Renombrados seguros (sólo si existen) y propagación de nulls.
# ------------------------------------------------------------------------------------

import os, json
from pathlib import Path
from datetime import datetime, timezone
import polars as pl

def _print_kv(k, v): print(f"   - {k:<30}: {v}")

def _safe_pos(expr: pl.Expr, floor: float) -> pl.Expr:
    return pl.when(expr < pl.lit(floor)).then(pl.lit(floor)).otherwise(expr)

def _coalesce_alias(df: pl.DataFrame, target: str, candidates: list[str], dtype=None) -> pl.DataFrame:
    exprs = [pl.col(c).cast(dtype) if (dtype and c in df.columns) else pl.col(c) for c in candidates if c in df.columns]
    if exprs:
        df = df.with_columns(pl.coalesce(exprs).alias(target))
    return df

def _safe_rename(df: pl.DataFrame, mapping: dict[str,str]) -> pl.DataFrame:
    use = {k:v for k,v in mapping.items() if k in df.columns}
    return df.rename(use) if use else df

print("📖 Leyendo configuración (Celda 7e v1.1) ...")
need_env = ("TWF_SYMBOL","TWF_TF","TWF_START_DATE","TWF_END_DATE","TWF_SEED_BASE")
assert all(k in os.environ for k in need_env), "❌ Faltan TWF_*; ejecuta celdas previas."
SYMBOL = os.environ["TWF_SYMBOL"]; TF = os.environ["TWF_TF"]

PROJ_ROOT   = Path(os.environ.get("TWF_BASE", Path.cwd())).resolve()
CONFIG_PATH = PROJ_ROOT / "outputs" / SYMBOL / TF / "logs" / f"config_{SYMBOL}_{TF}.json"
assert CONFIG_PATH.exists(), f"❌ No existe configuración: {CONFIG_PATH}"

CFG = json.loads(CONFIG_PATH.read_text(encoding="utf-8"))
PATHS = {k: Path(v) for k, v in CFG["paths"].items()}
REPORTS_DIR = PATHS["reports"]; REPORTS_DIR.mkdir(parents=True, exist_ok=True)

SCORES_PARQ   = REPORTS_DIR / f"wf_train_scores_{SYMBOL}_{TF}.parquet"
SELECTED_PARQ = REPORTS_DIR / f"wf_selected_{SYMBOL}_{TF}.parquet"
EVAL_PARQ     = REPORTS_DIR / f"wf_select_eval_{SYMBOL}_{TF}.parquet"

DECAY_PARQ    = REPORTS_DIR / f"wf_decay_{SYMBOL}_{TF}.parquet"
DECAY_JSON    = REPORTS_DIR / f"wf_decay_{SYMBOL}_{TF}.json"
SUMMARY_JSON  = REPORTS_DIR / f"wf_decay_summary_{SYMBOL}_{TF}.json"

_print_kv("SCORES_PARQ", SCORES_PARQ)
_print_kv("SELECTED_PARQ", SELECTED_PARQ)
_print_kv("EVAL_PARQ", EVAL_PARQ)
assert SCORES_PARQ.exists(),  f"❌ Falta {SCORES_PARQ}"
assert SELECTED_PARQ.exists(),f"❌ Falta {SELECTED_PARQ}"
assert EVAL_PARQ.exists(),    f"❌ Falta {EVAL_PARQ}"

train = pl.read_parquet(SCORES_PARQ)
sel   = pl.read_parquet(SELECTED_PARQ)
test  = pl.read_parquet(EVAL_PARQ)

# wf_id como string
for name, df in (("train", train), ("sel", sel), ("test", test)):
    if "wf_id" in df.columns and df["wf_id"].dtype != pl.Utf8:
        df = df.with_columns(pl.col("wf_id").cast(pl.Utf8))
    locals()[name] = df

_print_kv("train_shape", f"{train.height}x{train.width}")
_print_kv("sel_shape",   f"{sel.height}x{sel.width}")
_print_kv("test_shape",  f"{test.height}x{test.width}")

# ----------- Reducir columnas y normalizar nombres -----------

# TRAIN: admite 'train_n' o 'n_signals'
train_keep = [c for c in ["wf_id","entry_col","train_n","n_signals","ret_mean","hit_rate_tp_excl_to","profit_factor_stat"] if c in train.columns]
train_k = train.select(train_keep)
train_k = _coalesce_alias(train_k, "train_n", ["train_n","n_signals"], dtype=pl.Int64)
train_k = _safe_rename(train_k, {
    "ret_mean": "train_ret_mean",
    "hit_rate_tp_excl_to": "train_hit_excl",
    "profit_factor_stat": "train_pf",
})

# TEST (7d): admite 'test_n' o 'n_signals'
test_keep = [c for c in [
    "wf_id","entry_col","test_n","n_signals","ret_mean","hit_rate_tp_excl_to",
    "profit_factor_stat","rank_in_window","cluster_id","cluster_size"
] if c in test.columns]
test_k = test.select(test_keep)
test_k = _coalesce_alias(test_k, "test_n", ["test_n","n_signals"], dtype=pl.Int64)
test_k = _safe_rename(test_k, {
    "ret_mean": "test_ret_mean",
    "hit_rate_tp_excl_to": "test_hit_excl",
    "profit_factor_stat": "test_pf",
})

# Seleccionados
sel_k = sel.select([c for c in ["wf_id","entry_col","rank_in_window","cluster_id","cluster_size"] if c in sel.columns]).unique()

print("🧮 Calculando decay por ventana/entry_col ...")
decay = (
    sel_k
    .join(train_k, on=["wf_id","entry_col"], how="left")
    .join(test_k,  on=["wf_id","entry_col"], how="left", suffix="_dup")
    .with_columns(
        # Deltas absolutos
        (pl.col("test_ret_mean") - pl.col("train_ret_mean")).alias("delta_ret_mean"),
        (pl.col("test_hit_excl") - pl.col("train_hit_excl")).alias("delta_hit_excl"),
        (pl.col("test_pf")       - pl.col("train_pf")).alias("delta_pf"),
        # Ratios (seguros ante 0/NA)
        (pl.col("test_ret_mean").abs() / _safe_pos(pl.col("train_ret_mean").abs(), 1e-12)).alias("ratio_ret_mean_abs"),
        (pl.col("test_hit_excl")       / _safe_pos(pl.col("train_hit_excl"), 1e-12)).alias("ratio_hit_excl"),
        (pl.col("test_pf")             / _safe_pos(pl.col("train_pf"), 1e-12)).alias("ratio_pf"),
        # Señales totales vistas (usa coalesce seguro)
        (pl.col("train_n").fill_null(0) + pl.col("test_n").fill_null(0)).alias("n_total_seen"),
    )
    .sort(["wf_id","rank_in_window","entry_col"])
)

_print_kv("decay_rows", decay.height); _print_kv("decay_cols", decay.width)

print("💾 Guardando wf_decay ...")
decay.write_parquet(DECAY_PARQ)
Path(DECAY_JSON).write_text(json.dumps({
    "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
    "symbol": SYMBOL, "timeframe": TF,
    "rows_head": decay.head(50).to_dicts(),
}, indent=2, ensure_ascii=False), encoding="utf-8")

print("📈 Resumen global (ponderado por test_n si existe) ...")
has_test_n = "test_n" in decay.columns
w = (pl.col("test_n").fill_null(0).cast(pl.Float64())) if has_test_n else pl.lit(1.0)

summary = (
    decay.with_columns(w.alias("__w__"))
         .select(
            ((pl.col("delta_ret_mean") * pl.col("__w__")).sum() / _safe_pos(pl.col("__w__").sum(), 1e-12)).alias("avg_delta_ret_mean"),
            ((pl.col("delta_hit_excl") * pl.col("__w__")).sum() / _safe_pos(pl.col("__w__").sum(), 1e-12)).alias("avg_delta_hit_excl"),
            ((pl.col("delta_pf")       * pl.col("__w__")).sum() / _safe_pos(pl.col("__w__").sum(), 1e-12)).alias("avg_delta_pf"),
            ((pl.col("ratio_ret_mean_abs") * pl.col("__w__")).sum() / _safe_pos(pl.col("__w__").sum(), 1e-12)).alias("avg_ratio_ret_mean_abs"),
            ((pl.col("ratio_hit_excl")     * pl.col("__w__")).sum() / _safe_pos(pl.col("__w__").sum(), 1e-12)).alias("avg_ratio_hit_excl"),
            ((pl.col("ratio_pf")           * pl.col("__w__")).sum() / _safe_pos(pl.col("__w__").sum(), 1e-12)).alias("avg_ratio_pf"),
         )
)

rank = (
    decay
    .select(
        "wf_id","entry_col","rank_in_window",
        "train_ret_mean","test_ret_mean","delta_ret_mean","ratio_ret_mean_abs",
        "train_hit_excl","test_hit_excl","delta_hit_excl","ratio_hit_excl",
        "train_pf","test_pf","delta_pf","ratio_pf",
        "train_n","test_n","n_total_seen"
    )
    .with_columns(
        pl.col("ratio_pf").fill_null(0.0).alias("_ratio_pf_"),
        pl.col("ratio_hit_excl").fill_null(0.0).alias("_ratio_hit_"),
        pl.col("delta_ret_mean").abs().fill_null(0.0).alias("_abs_dret_")
    )
    .with_columns(((pl.col("_ratio_pf_")+pl.col("_ratio_hit_"))/2.0 - pl.col("_abs_dret_")).alias("robustness_score"))
    .drop(["_ratio_pf_","_ratio_hit_","_abs_dret_"])
    .sort(["wf_id","robustness_score"], descending=[False, True])
)

Path(SUMMARY_JSON).write_text(json.dumps({
    "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
    "symbol": SYMBOL, "timeframe": TF,
    "global_summary": (summary.to_dicts()[0] if summary.height>0 else {}),
    "top5_by_robustness": rank.sort("robustness_score", descending=True).to_dicts()[:5],
    "bottom5_by_robustness": rank.sort("robustness_score", descending=False).to_dicts()[:5],
}, indent=2, ensure_ascii=False), encoding="utf-8")

_print_kv("DECAY_PARQ", DECAY_PARQ)
_print_kv("DECAY_JSON", DECAY_JSON)
_print_kv("SUMMARY_JSON", SUMMARY_JSON)
print("✅ Celda 7e (v1.1 FINAL) completada.\n")
 

📖 Leyendo configuración (Celda 7e v1.1) ...
   - SCORES_PARQ                   : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_train_scores_BTCUSDT_15m.parquet
   - SELECTED_PARQ                 : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_selected_BTCUSDT_15m.parquet
   - EVAL_PARQ                     : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_select_eval_BTCUSDT_15m.parquet
   - train_shape                   : 36x17
   - sel_shape                     : 36x5
   - test_shape                    : 36x22
🧮 Calculando decay por ventana/entry_col ...
   - decay_rows                    : 36
   - decay_cols                    : 24
💾 Guardando wf_decay ...
📈 Resumen global (ponderado por test_n si existe) ...
   - DECAY_PARQ                    : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_decay_BTCUSDT_15m.parquet
   - DECAY_JSON                    : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_decay_BTCUSDT_15m.json
   - SUMMARY_JSON                  : C:\Quant\TWF\outputs\BTCUSDT\15m\report

In [244]:
# Celda 8 (surfaces v1 FIX): [SUPERFICIES/HEATMAPS DE ESTABILIDAD POR ESTADO → reports/surfaces_*]
# ------------------------------------------------------------------------------------------------
# - Entrada: reports/states_{SYMBOL}_{TF}.parquet (deriva de Celda 6).
# - Objetivo: generar superficies (long y wide) por cada state_type (year, month, dow, hour, session, vol_bin),
#             con métricas clave: n_signals, ret_mean, ret_median, ret_std, hit_rate_tp_incl_to,
#             hit_rate_tp_excl_to, profit_factor_stat, etc. (solo si existen en el dataset).
# - Salida:
#     reports/surfaces_long_{SYMBOL}_{TF}_{state}.parquet / .csv
#     reports/surfaces_wide_ret_mean_{SYMBOL}_{TF}_{state}.parquet / .csv
#     reports/surfaces_wide_hit_excl_{SYMBOL}_{TF}_{state}.parquet / .csv
#     reports/surfaces_stability_{SYMBOL}_{TF}_{state}.parquet / .csv   (dispersion intra-estado por entry_col)
# - Notas:
#     * Mantiene rutas y nombres originales (drop-in replacement).
#     * Corrige DeprecationWarning de pivot: usa on="state_value".
#     * Prints exhaustivos para auditoría.
# ------------------------------------------------------------------------------------------------

import os, json
from pathlib import Path
from datetime import datetime, timezone
import polars as pl

def _print_kv(k, v):
    print(f"   - {k:<34}: {v}")

def _save(df: pl.DataFrame, base: Path):
    df.write_parquet(base.with_suffix(".parquet"))
    # CSV auxiliar (siempre best-effort)
    try:
        df.write_csv(base.with_suffix(".csv"))
    except Exception as e:
        _print_kv("csv_warn", f"{base.name}: {e}")

print("📖 Leyendo configuración (Celda 8 surfaces) ...")
need_env = ("TWF_SYMBOL","TWF_TF","TWF_START_DATE","TWF_END_DATE","TWF_SEED_BASE")
assert all(k in os.environ for k in need_env), "❌ Faltan TWF_*; ejecuta celdas previas."

SYMBOL = os.environ["TWF_SYMBOL"]
TF     = os.environ["TWF_TF"]

PROJ_ROOT   = Path(os.environ.get("TWF_BASE", Path.cwd())).resolve()
CONFIG_PATH = PROJ_ROOT / "outputs" / SYMBOL / TF / "logs" / f"config_{SYMBOL}_{TF}.json"
assert CONFIG_PATH.exists(), f"❌ No existe configuración: {CONFIG_PATH}"

with CONFIG_PATH.open("r", encoding="utf-8") as f:
    CFG = json.load(f)

PATHS       = {k: Path(v) for k, v in CFG["paths"].items()}
REPORTS_DIR = PATHS["reports"]; REPORTS_DIR.mkdir(parents=True, exist_ok=True)

STATES_PARQ = REPORTS_DIR / f"states_{SYMBOL}_{TF}.parquet"
assert STATES_PARQ.exists(), f"❌ No existe states: {STATES_PARQ}"

_print_kv("CONFIG_PATH", CONFIG_PATH)
_print_kv("REPORTS_DIR", REPORTS_DIR)
_print_kv("STATES_PARQ", STATES_PARQ)

print("📦 Cargando states ...")
st = pl.read_parquet(STATES_PARQ)

# Chequeos mínimos
required_cols = ["entry_col","state_type","state_value","n_signals","ret_mean","hit_rate_tp_excl_to"]
for c in required_cols:
    assert c in st.columns, f"❌ Falta columna en states: {c}"

_print_kv("states_rows", st.height)
_print_kv("states_cols", st.width)

# Lista de tipos de estado disponibles
state_types = st["state_type"].unique().to_list()
_print_kv("state_types", state_types)

# Columnas métricas candidatas (guardamos solo las que existan)
CAND_METRICS = [
    "n_signals","tp","sl","timeout",
    "ret_mean","ret_median","ret_std","ret_q05","ret_q25","ret_q50","ret_q75","ret_q95",
    "bars_mean","bars_median",
    "ret_mean_tp","ret_mean_sl","ret_mean_timeout",
    "sum_ret_tp","sum_abs_ret_sl",
    "hit_rate_tp_incl_to","hit_rate_tp_excl_to","profit_factor_stat"
]
METRICS = [c for c in CAND_METRICS if c in st.columns]
_print_kv("metrics_detected", METRICS)

print("🗺️  Generando superficies por state_type ...")
for s in state_types:
    sub = (
        st.filter(pl.col("state_type")==s)
          .with_columns(pl.col("state_value").cast(pl.Utf8))
    )
    _print_kv(f"[{s}] rows", sub.height)

    # Long (ya está long, pero filtramos columnas relevantes y ordenamos)
    long_cols = ["entry_col","state_type","state_value"] + METRICS
    long_cols = [c for c in long_cols if c in sub.columns]
    long_df = sub.select(long_cols).sort(["entry_col","state_value"])
    base_long = REPORTS_DIR / f"surfaces_long_{SYMBOL}_{TF}_{s}"
    _save(long_df, base_long)
    _print_kv(f"[{s}] saved_long", str(base_long.with_suffix('.parquet')))

    # Wide (ret_mean)
    if "ret_mean" in sub.columns:
        wide_ret = (
            sub
            .select(["entry_col","state_value","ret_mean"])
            .pivot(values="ret_mean", index="entry_col", on="state_value", aggregate_function="first")
            .sort("entry_col")
        )
        base_wide_ret = REPORTS_DIR / f"surfaces_wide_ret_mean_{SYMBOL}_{TF}_{s}"
        _save(wide_ret, base_wide_ret)
        _print_kv(f"[{s}] wide_ret_mean_shape", f"{wide_ret.height}x{wide_ret.width}")
        _print_kv(f"[{s}] saved_wide_ret", str(base_wide_ret.with_suffix('.parquet')))

    # Wide (hit_rate_tp_excl_to)
    if "hit_rate_tp_excl_to" in sub.columns:
        wide_hit = (
            sub
            .select(["entry_col","state_value","hit_rate_tp_excl_to"])
            .pivot(values="hit_rate_tp_excl_to", index="entry_col", on="state_value", aggregate_function="first")
            .sort("entry_col")
        )
        base_wide_hit = REPORTS_DIR / f"surfaces_wide_hit_excl_{SYMBOL}_{TF}_{s}"
        _save(wide_hit, base_wide_hit)
        _print_kv(f"[{s}] wide_hit_excl_shape", f"{wide_hit.height}x{wide_hit.width}")
        _print_kv(f"[{s}] saved_wide_hit", str(base_wide_hit.with_suffix('.parquet')))

    # Estabilidad: dispersión intra-estado por entry_col (std y rango de ret_mean)
    if "ret_mean" in sub.columns:
        stab = (
            sub.group_by("entry_col")
               .agg(
                   pl.len().alias("states_count"),
                   pl.col("ret_mean").mean().alias("ret_mean_avg"),
                   pl.col("ret_mean").std().alias("ret_mean_std"),
                   (pl.col("ret_mean").max() - pl.col("ret_mean").min()).alias("ret_mean_range"),
               )
               .sort(["ret_mean_std","ret_mean_range"])
        )
        base_stab = REPORTS_DIR / f"surfaces_stability_{SYMBOL}_{TF}_{s}"
        _save(stab, base_stab)
        _print_kv(f"[{s}] stability_rows", stab.height)
        _print_kv(f"[{s}] saved_stability", str(base_stab.with_suffix('.parquet')))

print("✅ Celda 8 (surfaces v1 FIX) finalizada sin errores.\n")
 

📖 Leyendo configuración (Celda 8 surfaces) ...
   - CONFIG_PATH                       : C:\Quant\TWF\outputs\BTCUSDT\15m\logs\config_BTCUSDT_15m.json
   - REPORTS_DIR                       : C:\Quant\TWF\outputs\BTCUSDT\15m\reports
   - STATES_PARQ                       : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\states_BTCUSDT_15m.parquet
📦 Cargando states ...
   - states_rows                       : 567
   - states_cols                       : 19
   - state_types                       : ['year', 'density', 'session', 'trend_er', 'dow', 'month', 'vol_bin', 'hour']
   - metrics_detected                  : ['n_signals', 'tp', 'sl', 'timeout', 'ret_mean', 'ret_median', 'ret_std', 'ret_q05', 'ret_q25', 'ret_q50', 'ret_q75', 'ret_q95', 'bars_mean', 'bars_median', 'hit_rate_tp_incl_to', 'hit_rate_tp_excl_to']
🗺️  Generando superficies por state_type ...
   - [year] rows                       : 45
   - [year] saved_long                 : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\surfaces_long_

In [245]:
# Celda 8b (v2.1 FINAL FIX): [CALIBRACIÓN y POTENCIA con n_eff_test]
# ---------------------------------------------------------------------------------
# Ubicación: DESPUÉS de 7d (y 7e si la usas) y ANTES de 9b.
# Cambios clave:
#   - Renombrado SEGURO (sólo si existe) + coalesce:
#       train_n ← coalesce(train.train_n, train.n_signals)
#       test_n  ← coalesce(test.test_n,  test.n_signals)
#   - wf_id como string para consistencia.
#   - Cálculo de n_pairs y n_eff_used en pasos separados.
# ---------------------------------------------------------------------------------

import os, json, math
from pathlib import Path
from datetime import datetime, timezone
import polars as pl

def _print_kv(k, v): print(f"   - {k:<34}: {v}")

def _norm_cdf(z: float) -> float:
    return 0.5 * (1.0 + math.erf(z / math.sqrt(2.0)))

def _power_two_sided(p0: float, delta: float, n_eff: float, alpha: float = 0.05) -> float:
    p0 = min(max(float(p0), 1e-9), 1-1e-9)
    n_eff = max(1.0, float(n_eff))
    se0 = math.sqrt(p0*(1-p0)/n_eff)
    zcrit = 1.9599639845400538  # ~N^{-1}(1-alpha/2)
    ncp = abs(delta) / max(se0, 1e-12)
    return _norm_cdf(-zcrit - ncp) + (1.0 - _norm_cdf(zcrit - ncp))

def _safe_rename(df: pl.DataFrame, mapping: dict[str, str]) -> pl.DataFrame:
    use = {k: v for k, v in mapping.items() if k in df.columns}
    return df.rename(use) if use else df

def _coalesce_alias(df: pl.DataFrame, target: str, candidates: list[str], dtype=None) -> pl.DataFrame:
    exprs = [pl.col(c).cast(dtype) if (dtype and c in df.columns) else pl.col(c)
             for c in candidates if c in df.columns]
    if exprs:
        df = df.with_columns(pl.coalesce(exprs).alias(target))
    return df

print("📖 Leyendo configuración (Celda 8b v2.1 FINAL FIX) ...")
need_env = ("TWF_SYMBOL","TWF_TF","TWF_START_DATE","TWF_END_DATE","TWF_SEED_BASE")
assert all(k in os.environ for k in need_env), "❌ Faltan TWF_*."
SYMBOL = os.environ["TWF_SYMBOL"]; TF = os.environ["TWF_TF"]

PROJ_ROOT   = Path(os.environ.get("TWF_BASE", Path.cwd())).resolve()
CONFIG_PATH = PROJ_ROOT / "outputs" / SYMBOL / TF / "logs" / f"config_{SYMBOL}_{TF}.json"
assert CONFIG_PATH.exists(), f"❌ No existe configuración: {CONFIG_PATH}"

CFG = json.loads(CONFIG_PATH.read_text(encoding="utf-8"))
PATHS = {k: Path(v) for k, v in CFG["paths"].items()}
REPORTS_DIR = PATHS["reports"]; REPORTS_DIR.mkdir(parents=True, exist_ok=True)

TRAIN_PARQ = REPORTS_DIR / f"wf_train_scores_{SYMBOL}_{TF}.parquet"
SEL_PARQ   = REPORTS_DIR / f"wf_selected_{SYMBOL}_{TF}.parquet"
TEST_PARQ  = REPORTS_DIR / f"wf_select_eval_{SYMBOL}_{TF}.parquet"

CAL_Q_PARQ = REPORTS_DIR / f"calib_score_quart_{SYMBOL}_{TF}.parquet"
CAL_Q_JSON = REPORTS_DIR / f"calib_score_quart_{SYMBOL}_{TF}.json"
CAL_D_PARQ = REPORTS_DIR / f"calib_delta1_terc_{SYMBOL}_{TF}.parquet"
CAL_D_JSON = REPORTS_DIR / f"calib_delta1_terc_{SYMBOL}_{TF}.json"
PWR_PARQ   = REPORTS_DIR / f"power_hitrate_{SYMBOL}_{TF}.parquet"
PWR_JSON   = REPORTS_DIR / f"power_hitrate_{SYMBOL}_{TF}.json"

_print_kv("TRAIN_PARQ", TRAIN_PARQ); _print_kv("SEL_PARQ", SEL_PARQ); _print_kv("TEST_PARQ", TEST_PARQ)
assert TRAIN_PARQ.exists(), f"❌ Falta {TRAIN_PARQ}"
assert SEL_PARQ.exists(),   f"❌ Falta {SEL_PARQ}"
assert TEST_PARQ.exists(),  f"❌ Falta {TEST_PARQ}"

train = pl.read_parquet(TRAIN_PARQ)
sel   = pl.read_parquet(SEL_PARQ)
test  = pl.read_parquet(TEST_PARQ)

# wf_id como string
for name, df in (("train", train), ("sel", sel), ("test", test)):
    if "wf_id" in df.columns and df["wf_id"].dtype != pl.Utf8:
        df = df.with_columns(pl.col("wf_id").cast(pl.Utf8))
    locals()[name] = df

# -------------------- TRAIN --------------------
train_keep = [c for c in [
    "wf_id","entry_col","train_n","n_signals","hit_rate_tp_excl_to",
    "ret_mean","ret_std","score_train","p_pred_train"
] if c in train.columns]
train_k = train.select(train_keep)
# p_pred_train ← coalesce(p_pred_train, hit_rate_tp_excl_to), fallback 0.5 si ninguna existe
if "p_pred_train" not in train_k.columns:
    if "hit_rate_tp_excl_to" in train_k.columns:
        train_k = _safe_rename(train_k, {"hit_rate_tp_excl_to": "p_pred_train"})
    else:
        train_k = train_k.with_columns(pl.lit(0.5).alias("p_pred_train"))
# train_n ← coalesce(train_n, n_signals)
train_k = _coalesce_alias(train_k, "train_n", ["train_n","n_signals"], dtype=pl.Int64)
# renombres métricas
train_k = _safe_rename(train_k, {"ret_mean": "train_ret_mean", "ret_std": "train_ret_std"})

# -------------------- SELECTED --------------------
sel_k = sel.select([c for c in ["wf_id","entry_col","rank_in_window","cluster_id","cluster_size"] if c in sel.columns]).unique()

# -------------------- TEST (7d) --------------------
test_keep = [c for c in [
    "wf_id","entry_col","test_n","n_signals","tp","sl","timeout",
    "ret_mean","ret_std","n_eff_hit_test"
] if c in test.columns]
test_k = test.select(test_keep)
# test_n ← coalesce(test_n, n_signals)
test_k = _coalesce_alias(test_k, "test_n", ["test_n","n_signals"], dtype=pl.Int64)
# renombres métricas
test_k = _safe_rename(test_k, {"ret_mean": "test_ret_mean", "ret_std": "test_ret_std"})

print("🔗 Join (seleccionados ∩ train ∩ test) ...")
df = (sel_k.join(train_k, on=["wf_id","entry_col"], how="left")
          .join(test_k,  on=["wf_id","entry_col"], how="left", suffix="_dup"))

# ---------------- Calibración por CUARTILES de score_train ----------------
print("🧮 Calibración por CUARTILES de score_train ...")
if "score_train" not in df.columns or df.select(pl.col("score_train").is_not_null().sum()).item() == 0:
    calib_q = pl.DataFrame({"bucket": pl.Series([], dtype=pl.Utf8)})
    _print_kv("warn", "Falta score_train → calib vacía")
else:
    df_q = df.filter(pl.col("score_train").is_not_null())
    q25 = float(df_q.select(pl.col("score_train").quantile(0.25)).item())
    q50 = float(df_q.select(pl.col("score_train").quantile(0.50)).item())
    q75 = float(df_q.select(pl.col("score_train").quantile(0.75)).item())
    _print_kv("score_q25", q25); _print_kv("score_q50", q50); _print_kv("score_q75", q75)

    df_b = df_q.with_columns(
        pl.when(pl.col("score_train") <= q25).then(pl.lit("Q1"))
         .when(pl.col("score_train") <= q50).then(pl.lit("Q2"))
         .when(pl.col("score_train") <= q75).then(pl.lit("Q3"))
         .otherwise(pl.lit("Q4")).alias("score_quart"),
        (pl.col("tp").fill_null(0) + pl.col("sl").fill_null(0)).alias("__w__"),
        (pl.col("tp").fill_null(0) * (1.0 - pl.col("p_pred_train").fill_null(0.5))**2
         + pl.col("sl").fill_null(0) * (pl.col("p_pred_train").fill_null(0.5))**2).alias("__brier_num__"),
    )

    calib_q = (
        df_b.group_by("score_quart")
            .agg(
                pl.col("__w__").sum().alias("support_excl_to"),
                pl.col("tp").sum().alias("tp_sum"),
                pl.col("sl").sum().alias("sl_sum"),
                ((pl.col("p_pred_train").fill_null(0.5) * pl.col("__w__")).sum()
                 / pl.when(pl.col("__w__").sum()<=0).then(1.0).otherwise(pl.col("__w__").sum())).alias("p_pred_bucket"),
                (pl.col("__brier_num__").sum()
                 / pl.when(pl.col("__w__").sum()<=0).then(1.0).otherwise(pl.col("__w__").sum())).alias("brier_bucket"),
                pl.col("score_train").mean().alias("score_train_mean"),
                pl.col("p_pred_train").mean().alias("p_pred_mean_unw"),
            )
            .with_columns(
                (pl.col("tp_sum")+pl.col("sl_sum")).alias("n_pairs"),
                (pl.col("tp_sum") / pl.when((pl.col("tp_sum")+pl.col("sl_sum"))<=0).then(1.0)
                 .otherwise(pl.col("tp_sum")+pl.col("sl_sum"))).alias("p_obs_bucket"),
            )
            .sort("score_quart")
            .rename({"score_quart":"bucket"})
    )

    total_support = float(calib_q.select(pl.col("n_pairs").sum()).item())
    if total_support > 0:
        ECE = sum(
            (float(r["n_pairs"])/total_support) * abs(float(r["p_obs_bucket"]) - float(r["p_pred_bucket"]))
            for r in calib_q.to_dicts()
        )
    else:
        ECE = None
    _print_kv("ECE_score_quart", ECE if ECE is not None else "N/A")
    _print_kv("Brier_avg_score_quart", float(calib_q["brier_bucket"].mean()) if calib_q.height>0 else "N/A")

# ---------------- Calibración por TERCILES de delta1 ----------------
print("🧮 Calibración por TERCILES de delta1 := test_ret_mean - train_ret_mean ...")
if ("test_ret_mean" not in df.columns) or ("train_ret_mean" not in df.columns) \
   or df.select(pl.col("test_ret_mean").is_not_null().sum()).item()==0 \
   or df.select(pl.col("train_ret_mean").is_not_null().sum()).item()==0:
    calib_d = pl.DataFrame({"bucket": pl.Series([], dtype=pl.Utf8)})
    _print_kv("warn", "Faltan ret_mean train/test → calib_delta vacía")
else:
    df_dsrc = df.filter(pl.col("test_ret_mean").is_not_null() & pl.col("train_ret_mean").is_not_null()) \
                .with_columns(
                    (pl.col("test_ret_mean") - pl.col("train_ret_mean")).alias("delta1"),
                    (pl.col("tp").fill_null(0) + pl.col("sl").fill_null(0)).alias("__w__"),
                    (pl.col("tp").fill_null(0) * (1.0 - pl.col("p_pred_train").fill_null(0.5))**2
                     + pl.col("sl").fill_null(0) * (pl.col("p_pred_train").fill_null(0.5))**2).alias("__brier_num__"),
                )
    if df_dsrc.height > 0:
        t33 = float(df_dsrc.select(pl.col("delta1").quantile(0.3333333333)).item())
        t66 = float(df_dsrc.select(pl.col("delta1").quantile(0.6666666667)).item())
    else:
        t33 = t66 = 0.0
    _print_kv("delta1_t33", t33); _print_kv("delta1_t66", t66)

    df_db = df_dsrc.with_columns(
        pl.when(pl.col("delta1") <= t33).then(pl.lit("T1"))
         .when(pl.col("delta1") <= t66).then(pl.lit("T2"))
         .otherwise(pl.lit("T3")).alias("delta1_terc")
    )

    calib_d = (
        df_db.group_by("delta1_terc")
             .agg(
                 pl.col("__w__").sum().alias("support_excl_to"),
                 pl.col("tp").sum().alias("tp_sum"),
                 pl.col("sl").sum().alias("sl_sum"),
                 ((pl.col("p_pred_train").fill_null(0.5) * pl.col("__w__")).sum()
                  / pl.when(pl.col("__w__").sum()<=0).then(1.0).otherwise(pl.col("__w__").sum())).alias("p_pred_bucket"),
                 (pl.col("__brier_num__").sum()
                  / pl.when(pl.col("__w__").sum()<=0).then(1.0).otherwise(pl.col("__w__").sum())).alias("brier_bucket"),
                 pl.col("delta1").mean().alias("delta1_mean"),
                 pl.col("p_pred_train").mean().alias("p_pred_mean_unw"),
             )
             .with_columns(
                 (pl.col("tp_sum")+pl.col("sl_sum")).alias("n_pairs"),
                 (pl.col("tp_sum") / pl.when((pl.col("tp_sum")+pl.col("sl_sum"))<=0).then(1.0)
                  .otherwise(pl.col("tp_sum")+pl.col("sl_sum"))).alias("p_obs_bucket"),
             )
             .sort("delta1_terc")
             .rename({"delta1_terc":"bucket"})
    )

    total_support_d = float(calib_d.select(pl.col("n_pairs").sum()).item())
    if total_support_d > 0:
        ECE_d = sum(
            (float(r["n_pairs"])/total_support_d) * abs(float(r["p_obs_bucket"]) - float(r["p_pred_bucket"]))
            for r in calib_d.to_dicts()
        )
    else:
        ECE_d = None
    _print_kv("ECE_delta1_terc", ECE_d if ECE_d is not None else "N/A")
    _print_kv("Brier_avg_delta1_terc", float(calib_d["brier_bucket"].mean()) if calib_d.height>0 else "N/A")

# ---------------- Potencia usando n_eff_hit_test ----------------
print("⚡ Potencia (hitrate) con n_eff_test ...")
use_neff = "n_eff_hit_test" in df.columns

pwr_step1 = df.with_columns(
    (pl.col("tp").fill_null(0) + pl.col("sl").fill_null(0)).alias("n_pairs"),
    (pl.col("tp").fill_null(0).cast(pl.Float64())
     / pl.when((pl.col("tp").fill_null(0)+pl.col("sl").fill_null(0))<=0).then(1.0)
       .otherwise(pl.col("tp").fill_null(0)+pl.col("sl").fill_null(0))).alias("p_obs")
)

pwr = (
    pwr_step1.with_columns(
        pl.when(use_neff).then(pl.col("n_eff_hit_test").fill_null(pl.col("n_pairs")))
         .otherwise(pl.col("n_pairs")).cast(pl.Float64()).alias("n_eff_used")
    )
    .select("wf_id","entry_col","rank_in_window","cluster_id","cluster_size",
            "p_pred_train","p_obs","n_eff_used","n_pairs")
    .with_columns(
        pl.struct(["p_pred_train","n_eff_used"]).map_elements(
            lambda s: _power_two_sided(float(s["p_pred_train"]) if s["p_pred_train"] is not None else 0.5,
                                       0.05, max(float(s["n_eff_used"]),1.0))
        ).alias("power_delta_0p05"),
        pl.struct(["p_pred_train","n_eff_used"]).map_elements(
            lambda s: _power_two_sided(float(s["p_pred_train"]) if s["p_pred_train"] is not None else 0.5,
                                       0.10, max(float(s["n_eff_used"]),1.0))
        ).alias("power_delta_0p10"),
    )
    .sort(["wf_id","rank_in_window","entry_col"])
)

print("💾 Guardando calibraciones y potencia ...")
calib_q.write_parquet(CAL_Q_PARQ)
Path(CAL_Q_JSON).write_text(json.dumps({
    "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
    "symbol": SYMBOL, "timeframe": TF,
    "note": "ECE/Brier micro; pred = p_pred_train ponderada por soporte.",
    "rows": calib_q.to_dicts()
}, indent=2, ensure_ascii=False), encoding="utf-8")
_print_kv("CAL_Q_PARQ", CAL_Q_PARQ); _print_kv("CAL_Q_JSON", CAL_Q_JSON)

calib_d.write_parquet(CAL_D_PARQ)
Path(CAL_D_JSON).write_text(json.dumps({
    "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
    "symbol": SYMBOL, "timeframe": TF,
    "delta1_def": "test_ret_mean - train_ret_mean",
    "rows": calib_d.to_dicts()
}, indent=2, ensure_ascii=False), encoding="utf-8")
_print_kv("CAL_D_PARQ", CAL_D_PARQ); _print_kv("CAL_D_JSON", CAL_D_JSON)

pwr.write_parquet(PWR_PARQ)
Path(PWR_JSON).write_text(json.dumps({
    "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
    "symbol": SYMBOL, "timeframe": TF,
    "alpha": 0.05,
    "delta_list": [0.05, 0.10],
    "n_eff_used": "n_eff_hit_test si existe; si no, n_pairs",
    "rows": pwr.to_dicts()
}, indent=2, ensure_ascii=False), encoding="utf-8")
_print_kv("PWR_PARQ", PWR_PARQ); _print_kv("PWR_JSON", PWR_JSON)

print("✅ Celda 8b (v2.1 FINAL FIX) finalizada.\n")
 

📖 Leyendo configuración (Celda 8b v2.1 FINAL FIX) ...
   - TRAIN_PARQ                        : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_train_scores_BTCUSDT_15m.parquet
   - SEL_PARQ                          : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_selected_BTCUSDT_15m.parquet
   - TEST_PARQ                         : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_select_eval_BTCUSDT_15m.parquet
🔗 Join (seleccionados ∩ train ∩ test) ...
🧮 Calibración por CUARTILES de score_train ...
   - score_q25                         : -0.0003192229613169593
   - score_q50                         : 0.0002411707977396477
   - score_q75                         : 0.0006464070960267863
   - ECE_score_quart                   : 0.015963611324844335
   - Brier_avg_score_quart             : 0.21699351563500063
🧮 Calibración por TERCILES de delta1 := test_ret_mean - train_ret_mean ...
   - delta1_t33                        : -0.0006883242693972503
   - delta1_t66                        : 0.0001410574768

In [248]:
# Celda 9a (NUEVA v1.2 FIX DEFINITIVO): [SUPERFICIES GOOD×BAD & THRESHOLDS (TEST)]
# --------------------------------------------------------------------------------------------------------------
# Ubicación: después de 8b y antes de 9b (orden: ... 8b → 9a → 9b → 9c)
# Cambios clave vs v1.1:
#   1) Dos pasos para columnas dependientes:
#        df1 = ... .with_columns(n_pairs=tp+sl)
#        df  = df1.with_columns(n_eff_used = coalesce(n_eff_hit_test, n_pairs))
#   2) p_pred_train siempre existe por coalesce (p_pred_train | hit_rate_tp_excl_to | 0.5)
#   3) test_n y train_n normalizados con coalesce(..., n_signals)
#   4) El grid de score sólo si existe score_train
# --------------------------------------------------------------------------------------------------------------

import os, json, math, csv
from pathlib import Path
from datetime import datetime, timezone
import polars as pl
import numpy as np

def _print_kv(k, v): print(f"   - {k:<34}: {v}")

def _norm_cdf(z: float) -> float:
    return 0.5 * (1.0 + math.erf(z / math.sqrt(2.0)))

def _power_two_sided(p0: float, delta: float, n_eff: float, alpha: float = 0.05) -> float:
    p0 = min(max(float(p0 if p0 is not None else 0.5), 1e-9), 1-1e-9)
    n_eff = max(1.0, float(n_eff if n_eff is not None else 1.0))
    se0 = math.sqrt(p0*(1-p0)/n_eff)
    zcrit = 1.9599639845400538
    ncp = abs(delta) / max(se0, 1e-12)
    return _norm_cdf(-zcrit - ncp) + (1.0 - _norm_cdf(zcrit - ncp))

def _coalesce_alias(df: pl.DataFrame, target: str, candidates: list[str], dtype=None) -> pl.DataFrame:
    exprs = [pl.col(c).cast(dtype) if (dtype and c in df.columns) else pl.col(c)
             for c in candidates if c in df.columns]
    if exprs:
        df = df.with_columns(pl.coalesce(exprs).alias(target))
    else:
        df = df.with_columns(pl.lit(None).alias(target))
    return df

print("📖 Leyendo configuración (Celda 9a v1.2 FIX) ...")
need_env = ("TWF_SYMBOL","TWF_TF","TWF_START_DATE","TWF_END_DATE","TWF_SEED_BASE")
assert all(k in os.environ for k in need_env), "❌ Faltan TWF_*; ejecuta celdas previas."
SYMBOL = os.environ["TWF_SYMBOL"]; TF = os.environ["TWF_TF"]

PROJ_ROOT   = Path(os.environ.get("TWF_BASE", Path.cwd())).resolve()
CONFIG_PATH = PROJ_ROOT / "outputs" / SYMBOL / TF / "logs" / f"config_{SYMBOL}_{TF}.json"
assert CONFIG_PATH.exists(), f"❌ No existe configuración: {CONFIG_PATH}"

CFG = json.loads(CONFIG_PATH.read_text(encoding="utf-8"))
PATHS = {k: Path(v) for k, v in CFG["paths"].items()}
REPORTS = PATHS["reports"]; REPORTS.mkdir(parents=True, exist_ok=True)

TRAIN_PARQ = REPORTS / f"wf_train_scores_{SYMBOL}_{TF}.parquet"   # 7b
SEL_PARQ   = REPORTS / f"wf_selected_{SYMBOL}_{TF}.parquet"       # 7c
TEST_PARQ  = REPORTS / f"wf_select_eval_{SYMBOL}_{TF}.parquet"    # 7d

SURF_PARQ  = REPORTS / f"surfaces_goodbad_{SYMBOL}_{TF}.parquet"
SURF_JSON  = REPORTS / f"surfaces_goodbad_{SYMBOL}_{TF}.json"
RANK_PARQ  = REPORTS / f"rankings_thresholds_{SYMBOL}_{TF}.parquet"
RUNLOG_CSV = REPORTS / "run_log_records.csv"

_print_kv("TRAIN_PARQ", TRAIN_PARQ); _print_kv("SEL_PARQ", SEL_PARQ); _print_kv("TEST_PARQ", TEST_PARQ)
assert TRAIN_PARQ.exists(), "❌ Falta 7b (wf_train_scores)"
assert SEL_PARQ.exists(),   "❌ Falta 7c (wf_selected)"
assert TEST_PARQ.exists(),  "❌ Falta 7d (wf_select_eval)"

print("📦 Cargando artefactos ...")
train = pl.read_parquet(TRAIN_PARQ)
sel   = pl.read_parquet(SEL_PARQ)
test  = pl.read_parquet(TEST_PARQ)

# wf_id como string para consistencia
for name, df in (("train", train), ("sel", sel), ("test", test)):
    if "wf_id" in df.columns and df["wf_id"].dtype != pl.Utf8:
        df = df.with_columns(pl.col("wf_id").cast(pl.Utf8))
    locals()[name] = df

# -------------------------- TRAIN (normalización) --------------------------
train_keep = [c for c in ["wf_id","entry_col","score_train","p_pred_train","hit_rate_tp_excl_to","train_n","n_signals"] if c in train.columns]
train_k = train.select(train_keep)
# p_pred_train ← coalesce(p_pred_train, hit_rate_tp_excl_to, 0.5)
if "p_pred_train" not in train_k.columns or train_k.select(pl.col("p_pred_train").is_null().any()).item():
    candidates = []
    if "p_pred_train" in train_k.columns: candidates.append("p_pred_train")
    if "hit_rate_tp_excl_to" in train_k.columns: candidates.append("hit_rate_tp_excl_to")
    train_k = _coalesce_alias(train_k, "p_pred_train", candidates, dtype=pl.Float64)
    train_k = train_k.with_columns(pl.col("p_pred_train").fill_null(0.5))
# train_n ← coalesce(train_n, n_signals)
train_k = _coalesce_alias(train_k, "train_n", ["train_n","n_signals"], dtype=pl.Int64)

# -------------------------- SELECTED --------------------------
sel_k = sel.select([c for c in ["wf_id","entry_col","rank_in_window","cluster_id","cluster_size"] if c in sel.columns]).unique()

# -------------------------- TEST (7d) --------------------------
test_keep = [c for c in ["wf_id","entry_col","test_n","n_signals","tp","sl","timeout","ret_mean","ev_boot_mean","n_eff_hit_test"] if c in test.columns]
test_k = test.select(test_keep)
# test_n ← coalesce(test_n, n_signals)
test_k = _coalesce_alias(test_k, "test_n", ["test_n","n_signals"], dtype=pl.Int64)
# renombre métrica
if "ret_mean" in test_k.columns:
    test_k = test_k.rename({"ret_mean":"test_ret_mean"})

# -------------------------- Join principal --------------------------
df1 = (
    sel_k.join(train_k, on=["wf_id","entry_col"], how="left")
         .join(test_k,  on=["wf_id","entry_col"], how="left")
         .with_columns(
             (pl.col("tp").fill_null(0) + pl.col("sl").fill_null(0)).alias("n_pairs")
         )
)

# Paso 2: usar n_pairs YA materializado
df = (
    df1.with_columns(
        pl.coalesce([pl.col("n_eff_hit_test"), pl.col("n_pairs")]).cast(pl.Float64).alias("n_eff_used")
    )
)

_print_kv("df_shape", f"{df.height}x{df.width}")

# ---------------------- Grids de umbrales ----------------------
print("🧮 Construyendo grids de umbral ...")
# p_pred_train: [0.50, 0.90] paso 0.05, recortado al rango observado
pmin_obs = float(df.select(pl.col("p_pred_train").min()).item() or 0.5)
pmax_obs = float(df.select(pl.col("p_pred_train").max()).item() or 0.9)
p_lo = max(0.50, min(pmin_obs, 0.90))
p_hi = min(0.90, max(pmax_obs, 0.50))
p_grid = [round(x,2) for x in np.arange(math.floor(p_lo*20)/20, math.ceil(p_hi*20)/20 + 1e-9, 0.05)]
if not p_grid: p_grid = [0.50]

# score_train: cuantiles 50/60/70/80/90 + "sin filtro" SOLO si existe la columna
score_grid_vals = []
if "score_train" in df.columns and df.select(pl.col("score_train").is_not_null().sum()).item() > 0:
    for q in (0.50, 0.60, 0.70, 0.80, 0.90):
        try:
            score_grid_vals.append(float(df.select(pl.col("score_train").quantile(q)).item()))
        except Exception:
            pass
score_grid_vals = sorted(list({round(s, 12) for s in score_grid_vals}))
score_grid = [None] + score_grid_vals  # None = sin filtro de score

_print_kv("p_pred_grid", p_grid)
_print_kv("score_grid", ("none+" + ",".join([f"{x:.3g}" for x in score_grid_vals])) if score_grid_vals else "none")

# Parámetros
MIN_PAIRS_CELL = 30
MIN_REPS_CELL  = 3

# ---------------------- Agregación por celda de umbrales ----------------------
print("📊 Agregando métricas en la cuadrícula ...")
rows = []
fusion_suggestions = []

base_cols = ["wf_id","entry_col","p_pred_train","score_train","n_pairs","tp","sl","test_n","n_eff_used","ev_boot_mean","test_ret_mean"]
used_cols = [c for c in base_cols if c in df.columns]
df_local = df.select(used_cols)

for p_thr in p_grid:
    for s_thr in score_grid:
        mask = (pl.col("p_pred_train") >= p_thr)
        if s_thr is not None and "score_train" in df_local.columns:
            mask = mask & (pl.col("score_train").fill_null(float("-inf")) >= s_thr)
        sub = df_local.filter(mask)

        n_reps = sub.height
        tp_sum = int(sub.select(pl.col("tp").fill_null(0).sum().alias("__s")).item() or 0)
        sl_sum = int(sub.select(pl.col("sl").fill_null(0).sum().alias("__s")).item() or 0)
        n_pairs = tp_sum + sl_sum
        n_eff_total = float(sub.select(pl.col("n_eff_used").fill_null(0).sum().alias("__s")).item() or 0.0)
        test_n_sum = float(sub.select(pl.col("test_n").fill_null(0).sum().alias("__s")).item() or 0.0)

        p_obs_pool = (tp_sum / n_pairs) if n_pairs > 0 else None

        # p0_pool: promedio ponderado por n_eff_used
        if n_eff_total > 0 and sub.height > 0:
            p0_pool = float(sub.select((pl.col("p_pred_train") * pl.col("n_eff_used")).sum() / pl.col("n_eff_used").sum()).item())
        else:
            p0_pool = None

        # EV_pool: usa ev_boot_mean si existe; si no, test_ret_mean (ponderado por test_n)
        chosen_ev_col = "ev_boot_mean" if "ev_boot_mean" in sub.columns else ("test_ret_mean" if "test_ret_mean" in sub.columns else None)
        if chosen_ev_col and test_n_sum > 0:
            ev_pool = float(sub.select((pl.col(chosen_ev_col).fill_null(0.0) * pl.col("test_n").fill_null(0.0)).sum()
                                       / pl.col("test_n").fill_null(0.0).sum()).item())
        elif chosen_ev_col:
            ev_pool = float(sub.select(pl.col(chosen_ev_col).mean()).item() or 0.0)
        else:
            ev_pool = None

        pow_005 = _power_two_sided(p0_pool if p0_pool is not None else 0.5, 0.05, n_eff_total)
        pow_010 = _power_two_sided(p0_pool if p0_pool is not None else 0.5, 0.10, n_eff_total)

        low_support = (n_pairs < MIN_PAIRS_CELL) or (n_reps < MIN_REPS_CELL)
        if low_support:
            fusion_suggestions.append({
                "p_thr": p_thr, "score_thr": (None if s_thr is None else float(s_thr)),
                "reason": f"low_support: n_pairs={n_pairs} < {MIN_PAIRS_CELL} or n_reps={n_reps} < {MIN_REPS_CELL}",
                "suggestion": "relajar umbral (p_thr-0.05 o quitar score_thr) y re-evaluar"
            })

        rows.append({
            "p_min": float(p_thr),
            "score_min": (None if s_thr is None else float(s_thr)),
            "n_reps": n_reps,
            "n_pairs": n_pairs,
            "n_eff_total": n_eff_total,
            "tp_sum": tp_sum,
            "sl_sum": sl_sum,
            "p_obs_pool": (None if p_obs_pool is None else float(p_obs_pool)),
            "p0_pool": (None if p0_pool is None else float(p0_pool)),
            "ev_pool": (None if ev_pool is None else float(ev_pool)),
            "power_delta_0p05": float(pow_005),
            "power_delta_0p10": float(pow_010),
            "low_support_flag": bool(low_support),
        })

surfaces = pl.DataFrame(rows) if rows else pl.DataFrame({"p_min": pl.Series([], dtype=pl.Float64), "score_min": pl.Series([], dtype=pl.Float64)})
_print_kv("surfaces_rows", surfaces.height); _print_kv("surfaces_cols", surfaces.width)

# ---------------------- Rankings de thresholds ----------------------
print("🏆 Generando rankings de thresholds ...")
valid = surfaces.filter(pl.col("low_support_flag")==False)
rank_power_005 = valid.sort(["power_delta_0p05","ev_pool","n_pairs"], descending=[True, True, True]).with_columns(pl.lit("power_0p05").alias("criterion")).head(25)
rank_power_010 = valid.sort(["power_delta_0p10","ev_pool","n_pairs"], descending=[True, True, True]).with_columns(pl.lit("power_0p10").alias("criterion")).head(25)
rank_ev        = valid.sort(["ev_pool","n_pairs","power_delta_0p05"], descending=[True, True, True]).with_columns(pl.lit("ev_pool").alias("criterion")).head(25)
rankings = pl.concat([rank_power_005, rank_power_010, rank_ev], how="vertical") if valid.height>0 else valid

# ---------------------- Guardado ----------------------
print("💾 Guardando surfaces/rankings ...")
if surfaces.height > 0:
    surfaces.write_parquet(SURF_PARQ)
    SURF_JSON.write_text(json.dumps({
        "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
        "symbol": SYMBOL, "timeframe": TF,
        "grid": {"p_min_values": [float(x) for x in p_grid], "score_min_quantiles": [None] + [float(x) for x in score_grid_vals]},
        "rows": surfaces.head(50).to_dicts(),
        "fusions_suggestions": fusion_suggestions[:50]
    }, indent=2, ensure_ascii=False), encoding="utf-8")
    _print_kv("SURF_PARQ", SURF_PARQ); _print_kv("SURF_JSON", SURF_JSON)

if rankings.height > 0:
    rankings.write_parquet(RANK_PARQ)
    _print_kv("RANK_PARQ", RANK_PARQ)

# run_log append
RUNLOG_CSV.parent.mkdir(parents=True, exist_ok=True)
header = ["created_utc","symbol","tf","cell","rows","block_method","B","comment"]
append_header = not RUNLOG_CSV.exists()
with RUNLOG_CSV.open("a", newline="", encoding="utf-8") as fh:
    w = csv.writer(fh)
    if append_header:
        w.writerow(header)
    w.writerow([
        datetime.now(timezone.utc).isoformat(timespec="seconds"),
        SYMBOL, TF, "9a_surfaces_goodbad_v1_2_fixdef",
        surfaces.height, "", "",
        f"rankings={rankings.height}; low_support={int(surfaces.select(pl.col('low_support_flag').sum()).item() or 0)}"
    ])

print("✅ Celda 9a (v1.2 FIX DEFINITIVO) finalizada.\n")


📖 Leyendo configuración (Celda 9a v1.2 FIX) ...
   - TRAIN_PARQ                        : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_train_scores_BTCUSDT_15m.parquet
   - SEL_PARQ                          : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_selected_BTCUSDT_15m.parquet
   - TEST_PARQ                         : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_select_eval_BTCUSDT_15m.parquet
📦 Cargando artefactos ...
   - df_shape                          : 36x19
🧮 Construyendo grids de umbral ...
   - p_pred_grid                       : [np.float64(0.5)]
   - score_grid                        : none+0.000241,0.000437,0.000464,0.00086,0.00116
📊 Agregando métricas en la cuadrícula ...
   - surfaces_rows                     : 6
   - surfaces_cols                     : 13
🏆 Generando rankings de thresholds ...
💾 Guardando surfaces/rankings ...
   - SURF_PARQ                         : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\surfaces_goodbad_BTCUSDT_15m.parquet
   - SURF_JSON            

In [251]:
# Celda 9b (v1.3 FINAL ROBUST): [ALPHA-DECAY MENSUAL & RÉGIMEN → alpha_decay_monthly_*]
# --------------------------------------------------------------------------------------------------------------
# Ubicación: NUEVA. Colócala **después** de la Celda 9a y **antes** de la Celda 9c. (Orden: ... 8b → 9a → 9b → 9c)
#
# Entradas:
#   • outputs/{SYMBOL}/{TF}/forward/forward_{SYMBOL}_{TF}.parquet           (4c/4d)
#   • outputs/{SYMBOL}/{TF}/reports/wf_selected_{SYMBOL}_{TF}.parquet       (7c v3 FINAL)
#   • outputs/{SYMBOL}/{TF}/logs/wf_windows_{SYMBOL}_{TF}.json              (7a)
#
# Objetivo:
#   • Para cada (wf_id, entry_col) seleccionado, medir tendencia mensual en TEST:
#       - Theil–Sen slope sobre ret_mean mensual
#       - Kendall τ y Z de Mann–Kendall (aprox. sin empates)
#       - Half-life lineal (meses) si slope<0
#   • Señalizar soportes y meses efectivos por ventana.
#
# Salidas:
#   • outputs/{SYMBOL}/{TF}/reports/alpha_decay_monthly_{SYMBOL}_{TF}.parquet
#   • outputs/{SYMBOL}/{TF}/reports/alpha_decay_monthly_{SYMBOL}_{TF}.json
#   • outputs/{SYMBOL}/{TF}/reports/run_log_records.csv  (append)
# --------------------------------------------------------------------------------------------------------------

import os, json, math, csv
from pathlib import Path
from datetime import datetime, timezone
import polars as pl
import numpy as np

def _print_kv(k, v): print(f"   - {k:<34}: {v}")

def _parse_iso_utc(s: str):
    s = str(s).strip()
    if not s:
        return None
    # Normaliza 'Z' y convierte a UTC
    dt = datetime.fromisoformat(s.replace("Z", "+00:00"))
    if dt.tzinfo is None:
        return dt.replace(tzinfo=timezone.utc)
    return dt.astimezone(timezone.utc)

def _ensure_ts(df: pl.DataFrame) -> pl.DataFrame:
    # Asegura 'ts' Datetime[us, UTC], robusto a string/naive
    if "ts" not in df.columns:
        raise RuntimeError("❌ forward no tiene columna 'ts'")
    col = pl.col("ts")
    # 1) si es texto → strptime flexible
    if df["ts"].dtype == pl.Utf8:
        df = df.with_columns(pl.col("ts").str.strptime(pl.Datetime, strict=False, fmt=None))
    # 2) si no es datetime → cast
    if str(df.schema["ts"]).startswith("Datetime") is False:
        df = df.with_columns(pl.col("ts").cast(pl.Datetime))
    # 3) añadir/convertir zona horaria a UTC
    dtype_str = str(df.schema.get("ts"))
    if "time_zone=" not in dtype_str:
        df = df.with_columns(pl.col("ts").dt.replace_time_zone("UTC"))
    elif "UTC" not in dtype_str:
        df = df.with_columns(pl.col("ts").dt.convert_time_zone("UTC"))
    return df

def _theil_sen(y: np.ndarray) -> float | None:
    y = np.asarray(y, dtype=float)
    y = y[np.isfinite(y)]
    n = y.size
    if n < 2:
        return None
    x = np.arange(n, dtype=float)
    # median of pairwise slopes (j>i): (y[j]-y[i])/(x[j]-x[i]) ; aquí x es 0..n-1 ⇒ dj>0
    slopes = []
    for i in range(n-1):
        di = y[i+1:] - y[i]
        if di.size:
            slopes.extend((di / np.arange(1, di.size+1, dtype=float)).tolist())
    return float(np.median(slopes)) if slopes else None

def _kendall_tau_mk(y: np.ndarray) -> tuple[float | None, float | None, float | None]:
    # τ, Z y p bilateral (aprox. sin corrección por empates)
    y = np.asarray(y, dtype=float)
    y = y[np.isfinite(y)]
    n = y.size
    if n < 2:
        return None, None, None
    s = 0
    for i in range(n-1):
        s += int(np.sign(y[i+1:] - y[i]).sum())
    tau = s / (n*(n-1)/2.0)
    var_s = n*(n-1)*(2*n+5)/18.0
    if var_s <= 0:
        return float(tau), None, None
    if s > 0:
        z = (s - 1)/math.sqrt(var_s)
    elif s < 0:
        z = (s + 1)/math.sqrt(var_s)
    else:
        z = 0.0
    # p bilateral ~ 2*(1 - Phi(|z|))
    p = 2.0*(1.0 - 0.5*(1.0 + math.erf(abs(z)/math.sqrt(2.0))))
    return float(tau), float(z), float(p)

def _half_life_linear(ev_mean: float | None, slope_per_month: float | None) -> float | None:
    if ev_mean is None or slope_per_month is None:
        return None
    if slope_per_month >= 0:
        return float("inf")
    if ev_mean <= 0:
        return None
    return float(ev_mean / (2.0*abs(slope_per_month))) if abs(slope_per_month) > 0 else float("inf")

def _load_wf_windows(path: Path):
    raw = path.read_text(encoding="utf-8").strip()
    # 1) JSON estándar
    try:
        obj = json.loads(raw)
    except json.JSONDecodeError:
        # 2) NDJSON/JSONL
        lines = [json.loads(ln) for ln in raw.splitlines() if ln.strip()]
        return lines

    # Normalizaciones
    if isinstance(obj, list):
        # lista de dicts o lista de strings con JSON embebido
        if obj and isinstance(obj[0], str):
            items = []
            for s in obj:
                try:
                    items.append(json.loads(s))
                except Exception:
                    pass
            return items if items else obj
        return obj

    if isinstance(obj, dict):
        # dict con clave contenedora
        for k in ("windows", "wf_windows", "data"):
            v = obj.get(k)
            if isinstance(v, list):
                return v
        # dict de wf_id -> ventana
        if obj and all(isinstance(v, dict) for v in obj.values()):
            return list(obj.values())
        # JSON como string
        for k in ("text", "content", "value"):
            v = obj.get(k)
            if isinstance(v, str):
                try:
                    inner = json.loads(v)
                    if isinstance(inner, list):
                        return inner
                except Exception:
                    pass
        # un solo objeto con campos de test
        if {"wf_id","test_start","test_end"}.intersection(obj.keys()):
            return [obj]
        raise RuntimeError("Formato inesperado en wf_windows (dict).")

    if isinstance(obj, str):
        try:
            inner = json.loads(obj)
            if isinstance(inner, list):
                return inner
        except Exception:
            lines = [json.loads(ln) for ln in obj.splitlines() if ln.strip()]
            if lines:
                return lines
        raise RuntimeError("wf_windows: string JSON no parseable.")

    raise RuntimeError("wf_windows: formato no soportado.")

def _pick_time(w: dict, candidates: list[str]):
    for k in candidates:
        v = w.get(k)
        if v:
            return v
    return None

print("📖 Leyendo configuración (Celda 9b v1.3 FINAL ROBUST) ...")
need_env = ("TWF_SYMBOL","TWF_TF","TWF_START_DATE","TWF_END_DATE","TWF_SEED_BASE")
assert all(k in os.environ for k in need_env), "❌ Faltan TWF_*; ejecuta celdas previas."
SYMBOL = os.environ["TWF_SYMBOL"]; TF = os.environ["TWF_TF"]

PROJ_ROOT   = Path(os.environ.get("TWF_BASE", Path.cwd())).resolve()
CONFIG_PATH = PROJ_ROOT / "outputs" / SYMBOL / TF / "logs" / f"config_{SYMBOL}_{TF}.json"
assert CONFIG_PATH.exists(), f"❌ No existe configuración: {CONFIG_PATH}"

CFG = json.loads(CONFIG_PATH.read_text(encoding="utf-8"))
PATHS = {k: Path(v) for k, v in CFG["paths"].items()}

FWD_PARQ  = PATHS["forward"] / f"forward_{SYMBOL}_{TF}.parquet"
SEL_PARQ  = PATHS["reports"] / f"wf_selected_{SYMBOL}_{TF}.parquet"
WF_JSON   = PATHS["logs"]    / f"wf_windows_{SYMBOL}_{TF}.json"

OUT_PARQ  = PATHS["reports"] / f"alpha_decay_monthly_{SYMBOL}_{TF}.parquet"
OUT_JSON  = PATHS["reports"] / f"alpha_decay_monthly_{SYMBOL}_{TF}.json"
RUNLOG_CSV= PATHS["reports"] / "run_log_records.csv"

_print_kv("FWD_PARQ", FWD_PARQ); _print_kv("SEL_PARQ", SEL_PARQ); _print_kv("WF_JSON", WF_JSON)
assert FWD_PARQ.exists(), f"❌ Falta {FWD_PARQ}"
assert SEL_PARQ.exists(), f"❌ Falta {SEL_PARQ}"
assert WF_JSON.exists(),  f"❌ Falta {WF_JSON}"

print("📦 Cargando artefactos ...")
fwd = pl.read_parquet(FWD_PARQ)
fwd = _ensure_ts(fwd)

need_cols = {"ts","entry_col"}
missing = sorted([c for c in need_cols if c not in fwd.columns])
assert not missing, f"❌ forward le faltan columnas: {missing}"

# normaliza entry_col a texto
if fwd["entry_col"].dtype != pl.Utf8:
    fwd = fwd.with_columns(pl.col("entry_col").cast(pl.Utf8))

# EV fila: ret_event si existe; si no, fallback con label (1/-1/0). Si tampoco hay label, error.
has_ret = "ret_event" in fwd.columns
if has_ret:
    fwd = fwd.with_columns(pl.col("ret_event").cast(pl.Float64).alias("ev"))
else:
    if "label" not in fwd.columns:
        raise RuntimeError("❌ forward no tiene 'ret_event' ni 'label' para construir EV")
    fwd = fwd.with_columns(
        pl.when(pl.col("label")==1).then(1.0)
         .when(pl.col("label")==-1).then(-1.0)
         .otherwise(0.0).cast(pl.Float64).alias("ev")
    )

sel = pl.read_parquet(SEL_PARQ)
keep_sel = [c for c in ["wf_id","entry_col","rank_in_window"] if c in sel.columns]
sel = sel.select(keep_sel).unique()
if "wf_id" in sel.columns and sel["wf_id"].dtype != pl.Utf8:
    sel = sel.with_columns(pl.col("wf_id").cast(pl.Utf8))

wf_windows = _load_wf_windows(WF_JSON)

# Mapa wf_id(str) -> (t0_test, t1_test)
wf_map = {}
for idx, w in enumerate(wf_windows):
    if not isinstance(w, dict):
        try:
            w = json.loads(w)
        except Exception:
            continue
    key = str(w.get("wf_id", f"wf_{idx:03d}"))
    t0s = _pick_time(w, ["test_start","test_t0","t0_test","t0","start","testStart"])
    t1s = _pick_time(w, ["test_end","test_t1","t1_test","t1","end","testEnd"])
    if not (t0s and t1s):
        # nested: "test": {"start":..., "end":...}
        test_block = w.get("test") if isinstance(w.get("test"), dict) else {}
        t0s = t0s or _pick_time(test_block, ["start","t0"])
        t1s = t1s or _pick_time(test_block, ["end","t1"])
    if not (t0s and t1s):
        continue
    t0 = _parse_iso_utc(t0s); t1 = _parse_iso_utc(t1s)
    if (t0 is None) or (t1 is None):
        continue
    wf_map[key] = (t0, t1)

_print_kv("windows_count", len(wf_map))
assert len(wf_map) > 0, "❌ No se pudo construir ninguna ventana de TEST desde wf_windows_*.json"

# ------------------------ Agregación mensual por (wf_id, entry_col) en TEST ------------------------
print("🧮 Agregando por mes (TEST) ...")
rows = []
for r in sel.to_dicts():
    wf_id = str(r.get("wf_id"))
    en    = r.get("entry_col")
    if (wf_id not in wf_map) or (en is None):
        rows.append({"wf_id": wf_id, "entry_col": en, "n_months": 0, "any_test_month": False})
        continue

    t0, t1 = wf_map[wf_id]
    sub = fwd.filter(
        (pl.col("entry_col") == en) &
        (pl.col("ts") >= pl.lit(t0)) &
        (pl.col("ts") <= pl.lit(t1))
    )
    if sub.is_empty():
        rows.append({"wf_id": wf_id, "entry_col": en, "n_months": 0, "any_test_month": False})
        continue

    # Mes calendario UTC
    sub = sub.with_columns(month = pl.col("ts").dt.truncate("1mo"))

    # Agregación mensual
    ag = (
        sub.group_by("month")
           .agg(
               pl.col("ev").mean().alias("ret_mean_month"),
               # soporte si existe label:
               (pl.when(pl.col("label")==1).then(1).otherwise(0)).sum().alias("tp_sum") if "label" in sub.columns else pl.lit(None).alias("tp_sum"),
               (pl.when(pl.col("label")==-1).then(1).otherwise(0)).sum().alias("sl_sum") if "label" in sub.columns else pl.lit(None).alias("sl_sum"),
               pl.len().alias("n_obs")
           )
           .sort("month")
    )

    n_m = ag.height
    if n_m == 0:
        rows.append({"wf_id": wf_id, "entry_col": en, "n_months": 0, "any_test_month": False})
        continue

    y = ag["ret_mean_month"].to_numpy()
    slope = _theil_sen(y)
    tau, z, p = _kendall_tau_mk(y)
    ev_overall = float(np.nanmean(y)) if y.size>0 else None
    hl = _half_life_linear(ev_overall, slope)

    support_pairs_total = None
    if "tp_sum" in ag.columns and "sl_sum" in ag.columns and ag["tp_sum"].dtype is not None:
        try:
            support_pairs_total = int(ag.select((pl.col("tp_sum").fill_null(0)+pl.col("sl_sum").fill_null(0)).sum()).item() or 0)
        except Exception:
            support_pairs_total = None

    rows.append({
        "wf_id": wf_id,
        "entry_col": en,
        "n_months": int(n_m),
        "any_test_month": True,
        "ev_mean_overall": (None if ev_overall is None else float(ev_overall)),
        "theil_sen_slope_per_month": (None if slope is None else float(slope)),
        "kendall_tau": (None if tau is None else float(tau)),
        "mk_Z": (None if z is None else float(z)),
        "mk_p_value_two_sided": (None if p is None else float(p)),
        "half_life_linear_months": (None if hl is None else (float("inf") if math.isinf(hl) else float(hl))),
        "support_pairs_total": support_pairs_total,
    })

alpha = pl.DataFrame(rows)
_print_kv("alpha_rows", alpha.height); _print_kv("alpha_cols", alpha.width)

print("💾 Guardando alpha-decay mensual ...")
alpha.write_parquet(OUT_PARQ)
Path(OUT_JSON).write_text(json.dumps({
    "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
    "symbol": SYMBOL, "timeframe": TF,
    "rows_head": alpha.head(50).to_dicts()
}, indent=2, ensure_ascii=False), encoding="utf-8")
_print_kv("OUT_PARQ", OUT_PARQ); _print_kv("OUT_JSON", OUT_JSON)

# run_log append
RUNLOG_CSV.parent.mkdir(parents=True, exist_ok=True)
header = ["created_utc","symbol","tf","cell","rows","block_method","B","comment"]
append_header = not RUNLOG_CSV.exists()
with RUNLOG_CSV.open("a", newline="", encoding="utf-8") as fh:
    w = csv.writer(fh)
    if append_header:
        w.writerow(header)
    w.writerow([
        datetime.now(timezone.utc).isoformat(timespec="seconds"),
        SYMBOL, TF, "9b_alpha_decay_monthly_v1_3_final",
        alpha.height, "", "",
        "theil_sen/kendall/MK + half-life lineal (loader robusto wf_windows)"
    ])

print("✅ Celda 9b (v1.3 FINAL ROBUST) finalizada.\n")


📖 Leyendo configuración (Celda 9b v1.3 FINAL ROBUST) ...
   - FWD_PARQ                          : C:\Quant\TWF\outputs\BTCUSDT\15m\forward\forward_BTCUSDT_15m.parquet
   - SEL_PARQ                          : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_selected_BTCUSDT_15m.parquet
   - WF_JSON                           : C:\Quant\TWF\outputs\BTCUSDT\15m\logs\wf_windows_BTCUSDT_15m.json
📦 Cargando artefactos ...
   - windows_count                     : 4
🧮 Agregando por mes (TEST) ...
   - alpha_rows                        : 36
   - alpha_cols                        : 11
💾 Guardando alpha-decay mensual ...
   - OUT_PARQ                          : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\alpha_decay_monthly_BTCUSDT_15m.parquet
   - OUT_JSON                          : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\alpha_decay_monthly_BTCUSDT_15m.json
✅ Celda 9b (v1.3 FINAL ROBUST) finalizada.



In [252]:
# Celda 9c (v1.3 FINAL): [RUPTURAS EN ALPHA (BIC, seg≥6m) → alpha_decay_breaks_*]
# --------------------------------------------------------------------------------------------------------------
# Ubicación: después de 9b. (Orden: ... 8b → 9a → 9b → 9c)
#
# Entradas:
#   • outputs/{SYMBOL}/{TF}/forward/forward_{SYMBOL}_{TF}.parquet
#   • outputs/{SYMBOL}/{TF}/reports/wf_selected_{SYMBOL}_{TF}.parquet
#   • outputs/{SYMBOL}/{TF}/logs/wf_windows_{SYMBOL}_{TF}.json
#
# Salidas:
#   • outputs/{SYMBOL}/{TF}/reports/alpha_decay_breaks_{SYMBOL}_{TF}.parquet
#   • outputs/{SYMBOL}/{TF}/reports/alpha_decay_breaks_{SYMBOL}_{TF}.json
#   • outputs/{SYMBOL}/{TF}/reports/run_log_records.csv (append)
# --------------------------------------------------------------------------------------------------------------

import os, json, math, csv
from pathlib import Path
from datetime import datetime, timezone
import polars as pl
import numpy as np

def _print_kv(k, v): print(f"   - {k:<34}: {v}")

def _parse_iso_utc(s: str):
    s = str(s).strip()
    if not s:
        return None
    dt = datetime.fromisoformat(s.replace("Z", "+00:00"))
    if dt.tzinfo is None:
        return dt.replace(tzinfo=timezone.utc)
    return dt.astimezone(timezone.utc)

def _ensure_ts(df: pl.DataFrame) -> pl.DataFrame:
    if "ts" not in df.columns:
        raise RuntimeError("❌ forward no tiene columna 'ts'")
    if df["ts"].dtype == pl.Utf8:
        df = df.with_columns(pl.col("ts").str.strptime(pl.Datetime, strict=False, fmt=None))
    if str(df.schema["ts"]).startswith("Datetime") is False:
        df = df.with_columns(pl.col("ts").cast(pl.Datetime))
    dtype_str = str(df.schema.get("ts"))
    if "time_zone=" not in dtype_str:
        df = df.with_columns(pl.col("ts").dt.replace_time_zone("UTC"))
    elif "UTC" not in dtype_str:
        df = df.with_columns(pl.col("ts").dt.convert_time_zone("UTC"))
    return df

def _load_wf_windows(path: Path):
    raw = path.read_text(encoding="utf-8").strip()
    try:
        obj = json.loads(raw)
    except json.JSONDecodeError:
        return [json.loads(ln) for ln in raw.splitlines() if ln.strip()]
    if isinstance(obj, list):
        return [json.loads(s) if isinstance(s, str) else s for s in obj]
    if isinstance(obj, dict):
        for k in ("windows","wf_windows","data"):
            v = obj.get(k)
            if isinstance(v, list):
                return v
        if obj and all(isinstance(v, dict) for v in obj.values()):
            return list(obj.values())
        for k in ("text","content","value"):
            v = obj.get(k)
            if isinstance(v, str):
                try:
                    inner = json.loads(v)
                    if isinstance(inner, list):
                        return inner
                except Exception:
                    pass
        if {"wf_id","test_start","test_end"}.intersection(obj.keys()):
            return [obj]
    if isinstance(obj, str):
        try:
            inner = json.loads(obj)
            if isinstance(inner, list):
                return inner
        except Exception:
            lines = [json.loads(ln) for ln in obj.splitlines() if ln.strip()]
            if lines: return lines
    raise RuntimeError("wf_windows: formato no soportado.")

def _pick_time(w: dict, ks: list[str]):
    for k in ks:
        v = w.get(k)
        if v: return v
    return None

def _bic_piecewise_const(y: np.ndarray, breaks: list[int]) -> tuple[float, list[tuple[int,int]], list[float], float]:
    """
    y: serie (n,)
    breaks: índices de inicio de cada segmento EXCEPTO el 0; e.g., [b1, b2] con 0<b1<b2<n
    Retorna: (BIC, segmentos [(s,e)], medias_segmento, sse_total)
    """
    y = np.asarray(y, dtype=float)
    n = int(y.size)
    idx = [0] + sorted(breaks) + [n]
    segs = [(idx[i], idx[i+1]) for i in range(len(idx)-1)]
    means = []
    sse = 0.0
    for s,e in segs:
        seg = y[s:e]
        mu = float(np.nanmean(seg)) if seg.size>0 else 0.0
        means.append(mu)
        sse += float(np.nansum((seg - mu)**2))
    k = len(segs)  # nº segmentos ⇒ parámetros: k medias + 1 var
    # verosimilitud normal con var común estimada: sse/n
    sigma2 = sse / max(n,1)
    if sigma2 <= 0: sigma2 = 1e-12
    loglik = -0.5*n*(math.log(2*math.pi*sigma2) + 1.0)
    p = k + 1  # k medias + var
    bic = -2.0*loglik + p*math.log(max(n,1))
    return bic, segs, means, sse

def _best_breaks(y: np.ndarray, minseg: int = 6, max_breaks: int = 2):
    """
    Explora 0,1,2 rupturas con segmentos >= minseg.
    Retorna: dict con 'k', 'break_idxs', 'bic', 'segments', 'means'.
    """
    y = np.asarray(y, dtype=float)
    n = int(y.size)
    best = {"k":0, "break_idxs":[], "bic":None, "segments":[(0,n)], "means":[float(np.nanmean(y)) if n>0 else None]}
    # 0 rupturas
    bic0, segs0, means0, _ = _bic_piecewise_const(y, [])
    best.update({"k":0, "break_idxs":[], "bic":bic0, "segments":segs0, "means":means0})
    # 1 ruptura
    for b1 in range(minseg, n-minseg+1):
        bic1, segs1, means1, _ = _bic_piecewise_const(y, [b1])
        if (best["bic"] is None) or (bic1 < best["bic"]):
            best = {"k":1, "break_idxs":[b1], "bic":bic1, "segments":segs1, "means":means1}
    # 2 rupturas
    if max_breaks >= 2:
        for b1 in range(minseg, n-2*minseg+1):
            for b2 in range(b1+minseg, n-minseg+1):
                bic2, segs2, means2, _ = _bic_piecewise_const(y, [b1,b2])
                if (best["bic"] is None) or (bic2 < best["bic"]):
                    best = {"k":2, "break_idxs":[b1,b2], "bic":bic2, "segments":segs2, "means":means2}
    return best

print("📖 Leyendo configuración (Celda 9c v1.3 FINAL) ...")
need_env = ("TWF_SYMBOL","TWF_TF","TWF_START_DATE","TWF_END_DATE","TWF_SEED_BASE")
assert all(k in os.environ for k in need_env), "❌ Faltan TWF_*; ejecuta celdas previas."
SYMBOL = os.environ["TWF_SYMBOL"]; TF = os.environ["TWF_TF"]

PROJ_ROOT   = Path(os.environ.get("TWF_BASE", Path.cwd())).resolve()
CONFIG_PATH = PROJ_ROOT / "outputs" / SYMBOL / TF / "logs" / f"config_{SYMBOL}_{TF}.json"
assert CONFIG_PATH.exists(), f"❌ No existe configuración: {CONFIG_PATH}"

CFG = json.loads(CONFIG_PATH.read_text(encoding="utf-8"))
PATHS = {k: Path(v) for k, v in CFG["paths"].items()}

FWD_PARQ  = PATHS["forward"] / f"forward_{SYMBOL}_{TF}.parquet"
SEL_PARQ  = PATHS["reports"] / f"wf_selected_{SYMBOL}_{TF}.parquet"
WF_JSON   = PATHS["logs"]    / f"wf_windows_{SYMBOL}_{TF}.json"

OUT_PARQ  = PATHS["reports"] / f"alpha_decay_breaks_{SYMBOL}_{TF}.parquet"
OUT_JSON  = PATHS["reports"] / f"alpha_decay_breaks_{SYMBOL}_{TF}.json"
RUNLOG_CSV= PATHS["reports"] / "run_log_records.csv"

_print_kv("FWD_PARQ", FWD_PARQ); _print_kv("SEL_PARQ", SEL_PARQ); _print_kv("WF_JSON", WF_JSON)
assert FWD_PARQ.exists(), f"❌ Falta {FWD_PARQ}"
assert SEL_PARQ.exists(), f"❌ Falta {SEL_PARQ}"
assert WF_JSON.exists(),  f"❌ Falta {WF_JSON}"

print("📦 Cargando artefactos ...")
fwd = pl.read_parquet(FWD_PARQ)
fwd = _ensure_ts(fwd)
if "entry_col" in fwd.columns and fwd["entry_col"].dtype != pl.Utf8:
    fwd = fwd.with_columns(pl.col("entry_col").cast(pl.Utf8))

# EV fila
has_ret = "ret_event" in fwd.columns
if has_ret:
    fwd = fwd.with_columns(pl.col("ret_event").cast(pl.Float64).alias("ev"))
else:
    if "label" not in fwd.columns:
        raise RuntimeError("❌ forward no tiene 'ret_event' ni 'label' para construir EV")
    fwd = fwd.with_columns(
        pl.when(pl.col("label")==1).then(1.0)
         .when(pl.col("label")==-1).then(-1.0)
         .otherwise(0.0).cast(pl.Float64).alias("ev")
    )

sel = pl.read_parquet(SEL_PARQ).select([c for c in ["wf_id","entry_col","rank_in_window"] if c in pl.read_parquet(SEL_PARQ).columns]).unique()
if "wf_id" in sel.columns and sel["wf_id"].dtype != pl.Utf8:
    sel = sel.with_columns(pl.col("wf_id").cast(pl.Utf8))

# ventanas TEST
raw_windows = _load_wf_windows(WF_JSON)
wf_map = {}
for idx, w in enumerate(raw_windows):
    if not isinstance(w, dict):
        try: w = json.loads(w)
        except Exception: continue
    key = str(w.get("wf_id", f"wf_{idx:03d}"))
    t0s = _pick_time(w, ["test_start","test_t0","t0_test","t0","start","testStart"])
    t1s = _pick_time(w, ["test_end","test_t1","t1_test","t1","end","testEnd"])
    if not (t0s and t1s):
        tb = w.get("test") if isinstance(w.get("test"), dict) else {}
        t0s = t0s or _pick_time(tb, ["start","t0"])
        t1s = t1s or _pick_time(tb, ["end","t1"])
    if not (t0s and t1s): 
        continue
    t0 = _parse_iso_utc(t0s); t1 = _parse_iso_utc(t1s)
    if (t0 is None) or (t1 is None): 
        continue
    wf_map[key] = (t0, t1)

_print_kv("windows_count", len(wf_map))
assert len(wf_map) > 0, "❌ No se pudo construir ninguna ventana de TEST."

# ---- Serie mensual y rupturas ----
print("🧮 Detectando rupturas mensuales (BIC, seg≥6m) ...")
rows = []
MINSEG = 6
EPS = 1e-9

for r in sel.to_dicts():
    wf_id = str(r.get("wf_id"))
    en    = r.get("entry_col")
    if (wf_id not in wf_map) or (en is None):
        rows.append({"wf_id": wf_id, "entry_col": en, "n_months": 0, "any_test_month": False})
        continue

    t0, t1 = wf_map[wf_id]
    sub = fwd.filter(
        (pl.col("entry_col")==en) & (pl.col("ts")>=pl.lit(t0)) & (pl.col("ts")<=pl.lit(t1))
    )
    if sub.is_empty():
        rows.append({"wf_id": wf_id, "entry_col": en, "n_months": 0, "any_test_month": False})
        continue

    g = (sub.with_columns(month = pl.col("ts").dt.truncate("1mo"))
             .group_by("month")
             .agg(pl.col("ev").mean().alias("ret_mean_month"), pl.len().alias("n_obs"))
             .sort("month"))

    y = g["ret_mean_month"].to_numpy()
    n = int(g.height)
    if n < 2:
        rows.append({"wf_id": wf_id, "entry_col": en, "n_months": n, "any_test_month": n>0})
        continue

    # si n insuficiente para seg>=6m, sólo 0 rupturas
    if n < 2*MINSEG:
        best = {"k":0, "break_idxs":[], "segments":[(0,n)], "means":[float(np.nanmean(y))]}
    else:
        best = _best_breaks(y, minseg=MINSEG, max_breaks=2)

    # materializar info legible
    months = g["month"].to_list()
    breaks_iso = []
    if best["k"] >= 1:
        for b in best["break_idxs"]:
            if 0 <= b < n:
                breaks_iso.append(months[b].isoformat())
    seg_means = [None if m is None else float(m) for m in best["means"]]
    last_mean = seg_means[-1] if seg_means else None
    prev_mean = seg_means[-2] if len(seg_means) >= 2 else None
    adverse_recent = (last_mean is not None) and ( (last_mean <= 0.0) or (prev_mean is not None and last_mean < (prev_mean - EPS)) )

    rows.append({
        "wf_id": wf_id,
        "entry_col": en,
        "n_months": n,
        "any_test_month": True,
        "best_k_breaks": int(best["k"]),
        "break_months_iso": breaks_iso,
        "segment_means": seg_means,
        "last_segment_start_iso": (months[best["segments"][-1][0]].isoformat() if best["segments"] else None),
        "last_segment_mean": last_mean,
        "prev_segment_mean": prev_mean,
        "adverse_recent_break": bool(adverse_recent),
        "bic": float(best["bic"]) if best.get("bic") is not None else None,
    })

out = pl.DataFrame(rows)
_print_kv("breaks_rows", out.height); _print_kv("breaks_cols", out.width)

print("💾 Guardando alpha-decay breaks ...")
out.write_parquet(OUT_PARQ)
Path(OUT_JSON).write_text(json.dumps({
    "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
    "symbol": SYMBOL, "timeframe": TF,
    "rows_head": out.head(50).to_dicts()
}, indent=2, ensure_ascii=False), encoding="utf-8")
_print_kv("OUT_PARQ", OUT_PARQ); _print_kv("OUT_JSON", OUT_JSON)

# run_log append
RUNLOG_CSV.parent.mkdir(parents=True, exist_ok=True)
header = ["created_utc","symbol","tf","cell","rows","block_method","B","comment"]
append_header = not RUNLOG_CSV.exists()
with RUNLOG_CSV.open("a", newline="", encoding="utf-8") as fh:
    w = csv.writer(fh)
    if append_header: w.writerow(header)
    w.writerow([
        datetime.now(timezone.utc).isoformat(timespec="seconds"),
        SYMBOL, TF, "9c_alpha_decay_breaks_v1_3_final",
        out.height, "", "",
        "BIC piecewise-const, seg>=6m, adverse_recent_break flag"
    ])

print("✅ Celda 9c (v1.3 FINAL) finalizada.\n")


📖 Leyendo configuración (Celda 9c v1.3 FINAL) ...
   - FWD_PARQ                          : C:\Quant\TWF\outputs\BTCUSDT\15m\forward\forward_BTCUSDT_15m.parquet
   - SEL_PARQ                          : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_selected_BTCUSDT_15m.parquet
   - WF_JSON                           : C:\Quant\TWF\outputs\BTCUSDT\15m\logs\wf_windows_BTCUSDT_15m.json
📦 Cargando artefactos ...
   - windows_count                     : 4
🧮 Detectando rupturas mensuales (BIC, seg≥6m) ...
   - breaks_rows                       : 36
   - breaks_cols                       : 12
💾 Guardando alpha-decay breaks ...
   - OUT_PARQ                          : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\alpha_decay_breaks_BTCUSDT_15m.parquet
   - OUT_JSON                          : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\alpha_decay_breaks_BTCUSDT_15m.json
✅ Celda 9c (v1.3 FINAL) finalizada.



In [ ]:
# Celda 9 (decay v1.1 TOLERANT): [DECAY TRAIN→TEST DE LOS SELECCIONADOS → reports/wf_decay_*]
# -----------------------------------------------------------------------------------
# - Entradas:
#     • reports/wf_train_scores_{SYMBOL}_{TF}.parquet  (7b)
#     • reports/wf_selected_{SYMBOL}_{TF}.parquet      (7c)
#     • reports/wf_select_eval_{SYMBOL}_{TF}.parquet   (7d)
# - Objetivo:
#     • Δ y ratio TEST vs TRAIN para: ret_mean, hit_rate_tp_excl_to, profit_factor_stat (si existe)
#     • Resumen global (ponderado por test_n si existe)
#     • Ranking de robustez (simple)
# - Salidas:
#     • reports/wf_decay_{SYMBOL}_{TF}.parquet / .json
#     • reports/wf_decay_summary_{SYMBOL}_{TF}.json
# - Notas (FIX):
#     • Renombrado tolerante: solo renombra columnas presentes (evita ColumnNotFoundError).
#     • Si falta profit_factor_stat, crea train_pf/test_pf = null y continúa.
# -----------------------------------------------------------------------------------

import os, json
from pathlib import Path
from datetime import datetime, timezone
import polars as pl

def _print_kv(k, v): print(f"   - {k:<30}: {v}")

def _safe_pos(expr: pl.Expr, floor: float) -> pl.Expr:
    return pl.when(expr < pl.lit(floor)).then(pl.lit(floor)).otherwise(expr)

def _rename_if_exists(df: pl.DataFrame, mapping: dict) -> pl.DataFrame:
    # filtra solo las claves presentes para evitar errores
    m = {k: v for k, v in mapping.items() if k in df.columns}
    return df.rename(m)

print("📖 Leyendo configuración (Celda 9 v1.1 TOLERANT) ...")
need_env = ("TWF_SYMBOL","TWF_TF","TWF_START_DATE","TWF_END_DATE","TWF_SEED_BASE")
assert all(k in os.environ for k in need_env), "❌ Faltan TWF_*; ejecuta celdas previas."
SYMBOL = os.environ["TWF_SYMBOL"]; TF = os.environ["TWF_TF"]

PROJ_ROOT   = Path(os.environ.get("TWF_BASE", Path.cwd())).resolve()
CONFIG_PATH = PROJ_ROOT / "outputs" / SYMBOL / TF / "logs" / f"config_{SYMBOL}_{TF}.json"
assert CONFIG_PATH.exists(), f"❌ No existe configuración: {CONFIG_PATH}"

CFG = json.loads(CONFIG_PATH.read_text(encoding="utf-8"))
PATHS       = {k: Path(v) for k, v in CFG["paths"].items()}
REPORTS_DIR = PATHS["reports"]; REPORTS_DIR.mkdir(parents=True, exist_ok=True)

SCORES_PARQ   = REPORTS_DIR / f"wf_train_scores_{SYMBOL}_{TF}.parquet"
SELECTED_PARQ = REPORTS_DIR / f"wf_selected_{SYMBOL}_{TF}.parquet"
EVAL_PARQ     = REPORTS_DIR / f"wf_select_eval_{SYMBOL}_{TF}.parquet"

DECAY_PARQ    = REPORTS_DIR / f"wf_decay_{SYMBOL}_{TF}.parquet"
DECAY_JSON    = REPORTS_DIR / f"wf_decay_{SYMBOL}_{TF}.json"
SUMMARY_JSON  = REPORTS_DIR / f"wf_decay_summary_{SYMBOL}_{TF}.json"

_print_kv("SCORES_PARQ", SCORES_PARQ)
_print_kv("SELECTED_PARQ", SELECTED_PARQ)
_print_kv("EVAL_PARQ", EVAL_PARQ)
assert SCORES_PARQ.exists(),  f"❌ Falta entrada: {SCORES_PARQ}"
assert SELECTED_PARQ.exists(),f"❌ Falta entrada: {SELECTED_PARQ}"
assert EVAL_PARQ.exists(),    f"❌ Falta entrada: {EVAL_PARQ}"

print("📦 Cargando datasets WF ...")
train = pl.read_parquet(SCORES_PARQ)
sel   = pl.read_parquet(SELECTED_PARQ)
test  = pl.read_parquet(EVAL_PARQ)

_print_kv("train_rows", train.height); _print_kv("train_cols", train.width)
_print_kv("sel_rows", sel.height);     _print_kv("sel_cols", sel.width)
_print_kv("test_rows", test.height);   _print_kv("test_cols", test.width)

# Chequeo de llaves
for c in ["wf_id","entry_col"]:
    assert (c in train.columns) and (c in sel.columns) and (c in test.columns), f"❌ Falta {c} en alguno de los datasets."

# TRAIN -> seleccionar y renombrar tolerante
keep_train = [c for c in ["wf_id","entry_col","n_signals","ret_mean","hit_rate_tp_excl_to","profit_factor_stat"] if c in train.columns]
train_k = train.select(keep_train)
train_k = _rename_if_exists(train_k, {
    "n_signals": "train_n",
    "ret_mean": "train_ret_mean",
    "hit_rate_tp_excl_to": "train_hit_excl",
    "profit_factor_stat": "train_pf",
})
# Asegurar columnas esperadas aunque no existan en origen
if "train_pf" not in train_k.columns:
    train_k = train_k.with_columns(pl.lit(None).cast(pl.Float64()).alias("train_pf"))
if "train_hit_excl" not in train_k.columns:
    train_k = train_k.with_columns(pl.lit(None).cast(pl.Float64()).alias("train_hit_excl"))
if "train_ret_mean" not in train_k.columns:
    train_k = train_k.with_columns(pl.lit(None).cast(pl.Float64()).alias("train_ret_mean"))
if "train_n" not in train_k.columns:
    train_k = train_k.with_columns(pl.lit(None).cast(pl.Int64()).alias("train_n"))

# TEST -> seleccionar y renombrar tolerante
keep_test = [c for c in [
    "wf_id","entry_col","n_signals","ret_mean","hit_rate_tp_excl_to","profit_factor_stat",
    "rank_in_window","cluster_id","cluster_size"
] if c in test.columns]
test_k = test.select(keep_test)
test_k = _rename_if_exists(test_k, {
    "n_signals": "test_n",
    "ret_mean": "test_ret_mean",
    "hit_rate_tp_excl_to": "test_hit_excl",
    "profit_factor_stat": "test_pf",
})
# Asegurar columnas esperadas aunque no existan en origen
for colname, dtype in [
    ("test_pf", pl.Float64()), ("test_hit_excl", pl.Float64()),
    ("test_ret_mean", pl.Float64()), ("test_n", pl.Int64())
]:
    if colname not in test_k.columns:
        test_k = test_k.with_columns(pl.lit(None).cast(dtype).alias(colname))

# Seleccionados (únicos)
sel_cols = [c for c in ["wf_id","entry_col","rank_in_window","cluster_id","cluster_size"] if c in sel.columns]
sel_k = sel.select(sel_cols).unique()

print("🧮 Calculando decay por ventana/entry_col ...")
decay = (
    sel_k
    .join(train_k, on=["wf_id","entry_col"], how="left")
    .join(test_k,  on=["wf_id","entry_col"], how="left", suffix="_dup")
    .with_columns(
        # Deltas absolutos
        (pl.col("test_ret_mean") - pl.col("train_ret_mean")).alias("delta_ret_mean"),
        (pl.col("test_hit_excl") - pl.col("train_hit_excl")).alias("delta_hit_excl"),
        (pl.col("test_pf")       - pl.col("train_pf")).alias("delta_pf"),
        # Ratios (seguros)
        (pl.col("test_ret_mean").abs() / _safe_pos(pl.col("train_ret_mean").abs(), 1e-12)).alias("ratio_ret_mean_abs"),
        (pl.col("test_hit_excl")       / _safe_pos(pl.col("train_hit_excl"), 1e-12)).alias("ratio_hit_excl"),
        (pl.col("test_pf")             / _safe_pos(pl.col("train_pf"), 1e-12)).alias("ratio_pf"),
        # Señales totales vistas (si faltan, 0 en null-safe)
        (pl.col("train_n").fill_null(0) + pl.col("test_n").fill_null(0)).alias("n_total_seen"),
    )
    .sort(["wf_id","rank_in_window","entry_col"])
)

_print_kv("decay_rows", decay.height)
_print_kv("decay_cols", decay.width)

print("💾 Guardando wf_decay ...")
decay.write_parquet(DECAY_PARQ)
DECAY_JSON.write_text(json.dumps({
    "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
    "symbol": SYMBOL, "timeframe": TF,
    "rows_head": decay.head(50).to_dicts(),
}, indent=2, ensure_ascii=False), encoding="utf-8")

print("📈 Resumen de decay (global y ranking) ...")
has_test_n = "test_n" in decay.columns
w = (pl.col("test_n").fill_null(0).cast(pl.Float64())) if has_test_n else pl.lit(1.0)

summary = (
    decay
    .with_columns(w.alias("__w__"))
    .select(
        ((pl.col("delta_ret_mean") * pl.col("__w__")).sum() / _safe_pos(pl.col("__w__").sum(), 1e-12)).alias("avg_delta_ret_mean"),
        ((pl.col("delta_hit_excl") * pl.col("__w__")).sum() / _safe_pos(pl.col("__w__").sum(), 1e-12)).alias("avg_delta_hit_excl"),
        ((pl.col("delta_pf")       * pl.col("__w__")).sum() / _safe_pos(pl.col("__w__").sum(), 1e-12)).alias("avg_delta_pf"),
        ((pl.col("ratio_ret_mean_abs") * pl.col("__w__")).sum() / _safe_pos(pl.col("__w__").sum(), 1e-12)).alias("avg_ratio_ret_mean_abs"),
        ((pl.col("ratio_hit_excl")     * pl.col("__w__")).sum() / _safe_pos(pl.col("__w__").sum(), 1e-12)).alias("avg_ratio_hit_excl"),
        ((pl.col("ratio_pf")           * pl.col("__w__")).sum() / _safe_pos(pl.col("__w__").sum(), 1e-12)).alias("avg_ratio_pf"),
    )
)

rank = (
    decay
    .select(
        "wf_id","entry_col","rank_in_window",
        "train_ret_mean","test_ret_mean","delta_ret_mean","ratio_ret_mean_abs",
        "train_hit_excl","test_hit_excl","delta_hit_excl","ratio_hit_excl",
        "train_pf","test_pf","delta_pf","ratio_pf",
        "train_n","test_n","n_total_seen"
    )
    .with_columns(
        pl.col("ratio_pf").fill_null(0.0).alias("_ratio_pf_"),
        pl.col("ratio_hit_excl").fill_null(0.0).alias("_ratio_hit_"),
        pl.col("delta_ret_mean").abs().fill_null(0.0).alias("_abs_dret_"),
    )
    .with_columns(
        (((pl.col("_ratio_pf_") + pl.col("_ratio_hit_"))/2.0) - pl.col("_abs_dret_")).alias("robustness_score")
    )
    .drop(["_ratio_pf_","_ratio_hit_","_abs_dret_"])
    .sort(["wf_id","robustness_score"], descending=[False, True])
)

SUMMARY_JSON.write_text(json.dumps({
    "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
    "symbol": SYMBOL, "timeframe": TF,
    "global_summary": (summary.to_dicts()[0] if summary.height>0 else {}),
    "top5_by_robustness": rank.sort("robustness_score", descending=True).to_dicts()[:5],
    "bottom5_by_robustness": rank.sort("robustness_score", descending=False).to_dicts()[:5],
}, indent=2, ensure_ascii=False), encoding="utf-8")

_print_kv("DECAY_PARQ", DECAY_PARQ)
_print_kv("DECAY_JSON", DECAY_JSON)
_print_kv("SUMMARY_JSON", SUMMARY_JSON)
print("✅ Celda 9 (decay v1.1 TOLERANT) finalizada sin errores.\n")
 

📖 Leyendo configuración (Celda 9 v1.1 TOLERANT) ...
   - SCORES_PARQ                   : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_train_scores_BTCUSDT_15m.parquet
   - SELECTED_PARQ                 : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_selected_BTCUSDT_15m.parquet
   - EVAL_PARQ                     : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_select_eval_BTCUSDT_15m.parquet
📦 Cargando datasets WF ...
   - train_rows                    : 36
   - train_cols                    : 32
   - sel_rows                      : 12
   - sel_cols                      : 11
   - test_rows                     : 12
   - test_cols                     : 22
🧮 Calculando decay por ventana/entry_col ...
   - decay_rows                    : 12
   - decay_cols                    : 23
💾 Guardando wf_decay ...
📈 Resumen de decay (global y ranking) ...
   - DECAY_PARQ                    : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_decay_BTCUSDT_15m.parquet
   - DECAY_JSON                    : C:\Quant

In [ ]:
# Celda 9b (FINAL): [ALPHA-DECAY MENSUAL y DIAGNÓSTICO DE RÉGIMEN]
# ------------------------------------------------------------------------------------
# Entradas:
#   • forward_{SYMBOL}_{TF}.parquet   (ts, entry_col, label, ret_event)
#   • reports/wf_selected_{SYMBOL}_{TF}.parquet
#   • logs/wf_windows_{SYMBOL}_{TF}.json
# Salidas:
#   • reports/alpha_decay_monthly_{SYMBOL}_{TF}.parquet / .json
#   • reports/alpha_decay_diag_{SYMBOL}_{TF}.parquet / .json
# ------------------------------------------------------------------------------------

import os, json, math
from pathlib import Path
from datetime import datetime, timezone
import polars as pl
import numpy as np

def _print_kv(k, v): print(f"   - {k:<32}: {v}")

def _theil_sen(y: np.ndarray):
    y = np.asarray(y, dtype=float)
    y = y[~np.isnan(y)]
    n = len(y)
    if n < 3: return None
    slopes = []
    for i in range(n-1):
        dy = y[i+1:] - y[i]
        dx = np.arange(1, n - i, dtype=float)
        slopes.extend(list(dy / dx))
    return float(np.median(slopes)) if slopes else None

def _kendall_tau(y: np.ndarray):
    y = np.asarray(y, dtype=float)
    y = y[~np.isnan(y)]
    n = len(y)
    if n < 3: return None
    x = np.arange(n, dtype=float)
    conc = 0; disc = 0
    for i in range(n-1):
        s = np.sign(y[i+1:] - y[i]) * np.sign(x[i+1:] - x[i])
        conc += np.sum(s > 0); disc += np.sum(s < 0)
    denom = n*(n-1)/2
    return float((conc - disc) / denom) if denom>0 else None

def _mann_kendall(y: np.ndarray):
    y = np.asarray(y, dtype=float)
    y = y[~np.isnan(y)]
    n = len(y)
    if n < 3: return (None,None,None,None)
    S = 0
    for i in range(n-1): S += np.sum(np.sign(y[i+1:] - y[i]))
    VarS = (n*(n-1)*(2*n+5))/18.0
    if VarS <= 0: return (float(S), None, None, None)
    if S>0: Z = (S-1)/math.sqrt(VarS)
    elif S<0: Z = (S+1)/math.sqrt(VarS)
    else: Z = 0.0
    p = 2.0 * (1.0 - 0.5*(1.0 + math.erf(abs(Z)/math.sqrt(2.0))))
    return (float(S), float(VarS), float(Z), float(p))

def _half_life_linear(y: np.ndarray, slope):
    if slope is None or np.all(np.isnan(y)): return None
    y = y[~np.isnan(y)]
    if y.size==0: return None
    edge0 = float(y[0])
    if edge0 == 0: return None
    if slope == 0 or np.sign(edge0) == np.sign(slope): return None
    t_half = (abs(edge0)/2.0) / abs(slope)
    return float(t_half) if np.isfinite(t_half) and t_half>=0 else None

print("📖 Leyendo configuración (Celda 9b FINAL) ...")
need_env = ("TWF_SYMBOL","TWF_TF","TWF_START_DATE","TWF_END_DATE","TWF_SEED_BASE")
assert all(k in os.environ for k in need_env), "❌ Faltan TWF_*; ejecuta celdas previas."
SYMBOL = os.environ["TWF_SYMBOL"]; TF = os.environ["TWF_TF"]

PROJ_ROOT   = Path(os.environ.get("TWF_BASE", Path.cwd())).resolve()
CONFIG_PATH = PROJ_ROOT / "outputs" / SYMBOL / TF / "logs" / f"config_{SYMBOL}_{TF}.json"
assert CONFIG_PATH.exists(), f"❌ No existe configuración: {CONFIG_PATH}"

CFG = json.loads(CONFIG_PATH.read_text(encoding="utf-8"))
PATHS = {k: Path(v) for k, v in CFG["paths"].items()}
PATHS["reports"].mkdir(parents=True, exist_ok=True)

FORWARD_PARQ = PATHS["forward"] / f"forward_{SYMBOL}_{TF}.parquet"
SEL_PARQ     = PATHS["reports"] / f"wf_selected_{SYMBOL}_{TF}.parquet"
WF_JSON      = PATHS["logs"]    / f"wf_windows_{SYMBOL}_{TF}.json"

MONTHLY_PARQ = PATHS["reports"] / f"alpha_decay_monthly_{SYMBOL}_{TF}.parquet"
MONTHLY_JSON = PATHS["reports"] / f"alpha_decay_monthly_{SYMBOL}_{TF}.json"
DIAG_PARQ    = PATHS["reports"] / f"alpha_decay_diag_{SYMBOL}_{TF}.parquet"
DIAG_JSON    = PATHS["reports"] / f"alpha_decay_diag_{SYMBOL}_{TF}.json"

_print_kv("FORWARD_PARQ", FORWARD_PARQ)
_print_kv("SEL_PARQ",     SEL_PARQ)
_print_kv("WF_JSON",      WF_JSON)

assert FORWARD_PARQ.exists(), f"❌ Falta {FORWARD_PARQ}"
assert SEL_PARQ.exists(),     f"❌ Falta {SEL_PARQ}"
assert WF_JSON.exists(),      f"❌ Falta {WF_JSON}"

print("📦 Cargando forward y seleccionados ...")
fwd = pl.read_parquet(FORWARD_PARQ)
sel = pl.read_parquet(SEL_PARQ).select(["wf_id","entry_col","rank_in_window","cluster_id","cluster_size"]).unique()
_print_kv("fwd_shape", f"{fwd.height}x{fwd.width}")
_print_kv("sel_shape", f"{sel.height}x{sel.width}")

for c in ["ts","entry_col","label","ret_event"]:
    assert c in fwd.columns, f"❌ Falta columna en forward: {c}"

windows = json.loads(WF_JSON.read_text(encoding="utf-8")).get("windows", [])
test_ors = []
for w in windows:
    t0 = datetime.fromisoformat(w["test"]["start"].replace("Z","+00:00")).astimezone(timezone.utc)
    t1 = datetime.fromisoformat(w["test"]["end"].replace("Z","+00:00")).astimezone(timezone.utc)
    test_ors.append(((pl.col("ts") >= pl.lit(t0)) & (pl.col("ts") <= pl.lit(t1))))
any_test_expr = pl.lit(False) if len(test_ors)==0 else pl.fold(False, lambda a,b: a|b, test_ors)

entry_list = sel.get_column("entry_col").unique().to_list()
_print_kv("entry_selected_unique", len(entry_list))
fwd_sel = fwd.filter(pl.col("entry_col").is_in(entry_list)) \
             .with_columns(
                 pl.col("ts").dt.strftime("%Y-%m").alias("year_month"),
                 any_test_expr.alias("any_test_month")
             )

print("🧮 Agregando serie mensual por entry_col ...")
monthly = (
    fwd_sel
    .group_by(["entry_col","year_month"])
    .agg(
        pl.len().alias("n_signals"),
        pl.sum((pl.col("label")==1).cast(pl.Int32)).alias("tp"),
        pl.sum((pl.col("label")==-1).cast(pl.Int32)).alias("sl"),
        pl.sum((pl.col("label")==0).cast(pl.Int32)).alias("timeout"),
        pl.col("ret_event").mean().alias("ret_mean"),
        pl.col("ret_event").std().alias("ret_std"),
        pl.any("any_test_month").alias("any_test_month")
    )
    .with_columns(
        (pl.col("tp") + pl.col("sl")).alias("n_excl_to"),
        (pl.col("tp").cast(pl.Float64) / pl.when((pl.col("tp")+pl.col("sl"))<=0).then(1.0)
             .otherwise(pl.col("tp")+pl.col("sl"))).alias("hit_excl"),
    )
    .sort(["entry_col","year_month"])
)

_print_kv("monthly_rows", monthly.height)
_print_kv("monthly_cols", monthly.width)
print("📊 Vista mensual (primeras 10 filas):")
print(monthly.head(10))

print("🧪 Diagnóstico por entry_col ...")
diag_rows = []
for entry in entry_list:
    m = monthly.filter(pl.col("entry_col")==entry).sort("year_month")
    y_ret = np.array(m.get_column("ret_mean").to_list(), dtype=float)
    y_hit = np.array(m.get_column("hit_excl").to_list(), dtype=float)

    slope_ret = _theil_sen(y_ret)
    slope_hit = _theil_sen(y_hit)
    tau_ret   = _kendall_tau(y_ret)
    tau_hit   = _kendall_tau(y_hit)
    S, VarS, Z, p = _mann_kendall(y_ret)
    t_half    = _half_life_linear(y_ret, slope_ret)

    diag_rows.append({
        "entry_col": entry,
        "months": int(m.height),
        "n_total": int(m.get_column("n_signals").sum()) if m.height>0 else 0,
        "slope_ret_theil_sen": slope_ret,
        "slope_hit_theil_sen": slope_hit,
        "kendall_tau_ret": tau_ret,
        "kendall_tau_hit": tau_hit,
        "mann_kendall_S": S,
        "mann_kendall_VarS": VarS,
        "mann_kendall_Z": Z,
        "mann_kendall_p": p,
        "half_life_months_linear": t_half,
        "any_test_months": int(m.get_column("any_test_month").sum()) if ("any_test_month" in m.columns and m.height>0) else 0,
    })

diag = pl.DataFrame(diag_rows) if len(diag_rows) else pl.DataFrame({"entry_col": pl.Series([], dtype=pl.Utf8)})
_print_kv("diag_rows", diag.height)

print("💾 Guardando monthly y diag ...")
MONTHLY_PARQ.parent.mkdir(parents=True, exist_ok=True)
monthly.write_parquet(MONTHLY_PARQ)
MONTHLY_JSON.write_text(json.dumps({
    "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
    "symbol": SYMBOL, "timeframe": TF,
    "note": "Serie mensual por entry_col (seleccionados); any_test_month indica si el mes cae en alguna ventana TEST.",
    "rows_head": monthly.head(50).to_dicts()
}, indent=2, ensure_ascii=False), encoding="utf-8")
diag.write_parquet(DIAG_PARQ)
DIAG_JSON.write_text(json.dumps({
    "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
    "symbol": SYMBOL, "timeframe": TF,
    "metrics": ["Theil–Sen(ret_mean/hit_excl)","Kendall τ","Mann–Kendall(ret_mean)","Half-life(lineal)"],
    "rows": diag.to_dicts()
}, indent=2, ensure_ascii=False), encoding="utf-8")

_print_kv("MONTHLY_PARQ", MONTHLY_PARQ)
_print_kv("MONTHLY_JSON", MONTHLY_JSON)
_print_kv("DIAG_PARQ", DIAG_PARQ)
_print_kv("DIAG_JSON", DIAG_JSON)
print("✅ Celda 9b (FINAL) completada.\n")


📖 Leyendo configuración (Celda 9b NUEVA) ...
   - FORWARD_PARQ                    : C:\Quant\TWF\outputs\BTCUSDT\15m\forward\forward_BTCUSDT_15m.parquet
   - SEL_PARQ                        : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_selected_BTCUSDT_15m.parquet
   - WF_JSON                         : C:\Quant\TWF\outputs\BTCUSDT\15m\logs\wf_windows_BTCUSDT_15m.json
📦 Cargando forward y seleccionados ...
   - fwd_shape                       : 447948x11
   - sel_shape                       : 12x5
   - ts dtype=Datetime(time_unit='us', time_zone='UTC') OK
   - entry_selected_unique           : 7
🧮 Agregando serie mensual por entry_col ...
   - monthly_rows                    : 412
   - monthly_cols                    : 11
📊 Vista mensual (primeras 10 filas):
shape: (10, 11)
┌──────────────┬────────────┬───────────┬─────┬───┬──────────┬──────────────┬───────────┬──────────┐
│ entry_col    ┆ year_month ┆ n_signals ┆ tp  ┆ … ┆ ret_std  ┆ any_test_mon ┆ n_excl_to ┆ hit_excl │
│ ---         

In [ ]:
# Celda 9c (FINAL): [DETECCIÓN DE RUPTURAS EN SERIE MENSUAL → alpha_decay_breaks_*]
# ---------------------------------------------------------------------------------
# - Entrada: reports/alpha_decay_monthly_{SYMBOL}_{TF}.parquet
# - Salida : reports/alpha_decay_breaks_{SYMBOL}_{TF}.parquet / .json
# - Nota   : Segmentos mínimos de 6 meses; JSON seguro con casting a tipos nativos.
# ---------------------------------------------------------------------------------

import os, json, math
from pathlib import Path
from datetime import datetime, timezone
import polars as pl
import numpy as np

def _print_kv(k, v): print(f"   - {k:<32}: {v}")

def _bic_of_segments(y: np.ndarray, cuts: list[int]):
    # y: serie; cuts: índices fin de segmentos (exclusivo), p.ej. [j1, j2, ..., n]
    # Modelo: media por segmento, varianza común (MLE); BIC ~ -2*loglik + k*log(n)
    y = np.asarray(y, dtype=float)
    n = len(y)
    if n == 0: return (np.inf, None)
    segs = []
    last = 0
    for c in cuts:
        segs.append(y[last:c])
        last = c
    if last < n:
        segs.append(y[last:n])
    # cada segmento debe tener al menos 6 obs
    for s in segs:
        if len(s) < 6: return (np.inf, None)
    mu = [np.nanmean(s) for s in segs]
    resid = []
    last = 0
    for m, c in zip(mu, cuts + [n]):
        seg = y[last:c]
        resid.extend((seg - m))
        last = c
    resid = np.asarray(resid, dtype=float)
    sigma2 = np.nanmean(resid**2) if resid.size>0 else 0.0
    if sigma2 <= 0: sigma2 = 1e-12
    loglik = -0.5*n*(math.log(2*math.pi*sigma2) + 1.0)
    k = len(mu)  # parámetros de medias; sigma2 tratada como MLE común
    bic = -2*loglik + k * math.log(max(n,1))
    return (bic, mu)

print("📖 Leyendo configuración (Celda 9c FINAL) ...")
need_env = ("TWF_SYMBOL","TWF_TF","TWF_START_DATE","TWF_END_DATE","TWF_SEED_BASE")
assert all(k in os.environ for k in need_env), "❌ Faltan TWF_*; ejecuta celdas previas."
SYMBOL = os.environ["TWF_SYMBOL"]; TF = os.environ["TWF_TF"]

PROJ_ROOT   = Path(os.environ.get("TWF_BASE", Path.cwd())).resolve()
CONFIG_PATH = PROJ_ROOT / "outputs" / SYMBOL / TF / "logs" / f"config_{SYMBOL}_{TF}.json"
assert CONFIG_PATH.exists(), f"❌ No existe configuración: {CONFIG_PATH}"

CFG = json.loads(CONFIG_PATH.read_text(encoding="utf-8"))
PATHS = {k: Path(v) for k, v in CFG["paths"].items()}
PATHS["reports"].mkdir(parents=True, exist_ok=True)

MONTHLY_PARQ = PATHS["reports"] / f"alpha_decay_monthly_{SYMBOL}_{TF}.parquet"
BREAKS_PARQ  = PATHS["reports"] / f"alpha_decay_breaks_{SYMBOL}_{TF}.parquet"
BREAKS_JSON  = PATHS["reports"] / f"alpha_decay_breaks_{SYMBOL}_{TF}.json"

_print_kv("MONTHLY_PARQ", MONTHLY_PARQ)
assert MONTHLY_PARQ.exists(), "❌ Falta monthly"

monthly = pl.read_parquet(MONTHLY_PARQ)
_print_kv("monthly_rows", monthly.height); _print_kv("monthly_cols", monthly.width)

print("🧮 Detectando rupturas (BIC, seg≥6m) ...")
entries = monthly.get_column("entry_col").unique().to_list()
_print_kv("entries", len(entries))

break_rows = []
summary = []
for entry in entries:
    m = monthly.filter(pl.col("entry_col")==entry).sort("year_month")
    y = np.asarray(m.get_column("ret_mean").to_list(), dtype=float)
    n = len(y)
    # cortes candidatos: posiciones 6..n-6 (fin de segmento), permitiendo 0..k cortes
    candidates = list(range(6, max(6, n-5)))
    best_bic = np.inf; best_cuts = []; best_mu = None
    # 0 cortes
    bic0, mu0 = _bic_of_segments(y, [n])
    if bic0 < best_bic: best_bic, best_cuts, best_mu = bic0, [n], mu0
    # 1 corte
    for j in candidates:
        bic1, mu1 = _bic_of_segments(y, [j, n])
        if bic1 < best_bic:
            best_bic, best_cuts, best_mu = bic1, [j, n], mu1
    # 2 cortes (opcional; coste O(n^2))
    for j1 in candidates:
        for j2 in candidates:
            if j2 <= j1: continue
            bic2, mu2 = _bic_of_segments(y, [j1, j2, n])
            if bic2 < best_bic:
                best_bic, best_cuts, best_mu = bic2, [j1, j2, n], mu2

    # construir filas
    last = 0
    seg_idx = 0
    for c in best_cuts:
        seg = y[last:c]
        if seg.size >= 1:
            break_rows.append({
                "entry_col": entry,
                "seg_idx": seg_idx,
                "start_idx": last,
                "end_idx_exclusive": c,
                "n_months": int(seg.size),
                "ret_mean_seg": float(np.nanmean(seg)),
                "hit_excl_seg": float(np.nanmean(
                    np.asarray(
                        monthly.filter((pl.col("entry_col")==entry) & (pl.col("year_month").is_in(m.get_column("year_month").to_list()[last:c])))
                               .get_column("hit_excl").to_list(), dtype=float
                    )
                )) if seg.size>0 else None
            })
        last = c
        seg_idx += 1

    summary.append({
        "entry_col": entry,
        "n": int(n),
        "best_bic": float(best_bic),
        "n_segments": int(len(best_cuts)),
        "cuts": [int(x) for x in best_cuts],
    })

breaks_df = pl.DataFrame(break_rows) if break_rows else pl.DataFrame({
    "entry_col": pl.Series([], dtype=pl.Utf8),
    "seg_idx": pl.Series([], dtype=pl.Int32),
    "start_idx": pl.Series([], dtype=pl.Int32),
    "end_idx_exclusive": pl.Series([], dtype=pl.Int32),
    "n_months": pl.Series([], dtype=pl.Int32),
    "ret_mean_seg": pl.Series([], dtype=pl.Float64),
    "hit_excl_seg": pl.Series([], dtype=pl.Float64),
})

print("💾 Guardando rupturas ...")
breaks_df.write_parquet(BREAKS_PARQ)
BREAKS_JSON.write_text(json.dumps({
    "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
    "symbol": os.environ["TWF_SYMBOL"],
    "timeframe": os.environ["TWF_TF"],
    "note": "Rupturas por particionado óptimo con penalización BIC (min 6 meses).",
    "entries": summary
}, indent=2, ensure_ascii=False), encoding="utf-8")

_print_kv("BREAKS_PARQ", BREAKS_PARQ)
_print_kv("BREAKS_JSON", BREAKS_JSON)
print("✅ Celda 9c (FINAL) completada.\n")
 

📖 Leyendo configuración (Celda 9c NUEVA) ...
   - MONTHLY_PARQ                      : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\alpha_decay_monthly_BTCUSDT_15m.parquet
📦 Cargando monthly ...
   - monthly_rows                      : 412
   - monthly_cols                      : 11
🧮 Detectando rupturas (BIC, seg>=6m) ...
   - entries                           : 7
💾 Guardando rupturas ...
   - BREAKS_PARQ                       : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\alpha_decay_breaks_BTCUSDT_15m.parquet
   - BREAKS_JSON                       : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\alpha_decay_breaks_BTCUSDT_15m.json
✅ Celda 9c (rupturas BIC, seg>=6m, JSON-safe) finalizada sin errores.



In [ ]:
# Celda 9 (v1 FIX FINAL): [DECAY TRAIN→TEST DE LOS SELECCIONADOS → reports/wf_decay_*]
# -----------------------------------------------------------------------------------

import os, json
from pathlib import Path
from datetime import datetime, timezone
import polars as pl

def _print_kv(k, v): print(f"   - {k:<30}: {v}")
def _safe_pos(expr: pl.Expr, floor: float) -> pl.Expr:
    return pl.when(expr < pl.lit(floor)).then(pl.lit(floor)).otherwise(expr)

print("📖 Leyendo configuración ...")
need_env = ("TWF_SYMBOL","TWF_TF","TWF_START_DATE","TWF_END_DATE","TWF_SEED_BASE")
assert all(k in os.environ for k in need_env), "❌ Faltan TWF_*; ejecuta celdas previas."
SYMBOL = os.environ["TWF_SYMBOL"]; TF = os.environ["TWF_TF"]

PROJ_ROOT   = Path(os.environ.get("TWF_BASE", Path.cwd())).resolve()
CONFIG_PATH = PROJ_ROOT / "outputs" / SYMBOL / TF / "logs" / f"config_{SYMBOL}_{TF}.json"
assert CONFIG_PATH.exists(), f"❌ No existe configuración: {CONFIG_PATH}"

CFG = json.loads(CONFIG_PATH.read_text(encoding="utf-8"))
PATHS       = {k: Path(v) for k, v in CFG["paths"].items()}
REPORTS_DIR = PATHS["reports"]; REPORTS_DIR.mkdir(parents=True, exist_ok=True)

SCORES_PARQ   = REPORTS_DIR / f"wf_train_scores_{SYMBOL}_{TF}.parquet"
SELECTED_PARQ = REPORTS_DIR / f"wf_selected_{SYMBOL}_{TF}.parquet"
EVAL_PARQ     = REPORTS_DIR / f"wf_select_eval_{SYMBOL}_{TF}.parquet"

DECAY_PARQ    = REPORTS_DIR / f"wf_decay_{SYMBOL}_{TF}.parquet"
DECAY_JSON    = REPORTS_DIR / f"wf_decay_{SYMBOL}_{TF}.json"
SUMMARY_JSON  = REPORTS_DIR / f"wf_decay_summary_{SYMBOL}_{TF}.json"

_print_kv("SCORES_PARQ", SCORES_PARQ)
_print_kv("SELECTED_PARQ", SELECTED_PARQ)
_print_kv("EVAL_PARQ", EVAL_PARQ)
assert SCORES_PARQ.exists(),  f"❌ Falta entrada: {SCORES_PARQ}"
assert SELECTED_PARQ.exists(),f"❌ Falta entrada: {SELECTED_PARQ}"
assert EVAL_PARQ.exists(),    f"❌ Falta entrada: {EVAL_PARQ}"

print("📦 Cargando datasets WF ...")
train = pl.read_parquet(SCORES_PARQ)
sel   = pl.read_parquet(SELECTED_PARQ)
test  = pl.read_parquet(EVAL_PARQ)

_print_kv("train_rows", train.height); _print_kv("train_cols", train.width)
_print_kv("sel_rows", sel.height);     _print_kv("sel_cols", sel.width)
_print_kv("test_rows", test.height);   _print_kv("test_cols", test.width)

for c in ["wf_id","entry_col"]:
    assert (c in train.columns) and (c in sel.columns) and (c in test.columns), f"❌ Falta {c} en alguno de los datasets."

keep_train = [c for c in ["wf_id","entry_col","n_signals","ret_mean","hit_rate_tp_excl_to","profit_factor_stat"] if c in train.columns]
train_k = train.select(keep_train).rename({
    "n_signals":"train_n",
    "ret_mean":"train_ret_mean",
    "hit_rate_tp_excl_to":"train_hit_excl",
    "profit_factor_stat":"train_pf",
}, strict=False)  # <— tolerante

keep_test = [c for c in ["wf_id","entry_col","n_signals","ret_mean","hit_rate_tp_excl_to","profit_factor_stat","rank_in_window","cluster_id","cluster_size"] if c in test.columns]
test_k = test.select(keep_test).rename({
    "n_signals":"test_n",
    "ret_mean":"test_ret_mean",
    "hit_rate_tp_excl_to":"test_hit_excl",
    "profit_factor_stat":"test_pf",
}, strict=False)  # <— tolerante

sel_cols = [c for c in ["wf_id","entry_col","rank_in_window","cluster_id","cluster_size"] if c in sel.columns]
sel_k = sel.select(sel_cols).unique()

print("🧮 Calculando decay por ventana/entry_col ...")
decay = (
    sel_k
    .join(train_k, on=["wf_id","entry_col"], how="left")
    .join(test_k,  on=["wf_id","entry_col"], how="left", suffix="_testdup")
    .with_columns(
        (pl.col("test_ret_mean") - pl.col("train_ret_mean")).alias("delta_ret_mean"),
        (pl.col("test_hit_excl") - pl.col("train_hit_excl")).alias("delta_hit_excl"),
        (pl.col("test_pf")       - pl.col("train_pf")).alias("delta_pf"),
        (pl.col("test_ret_mean").abs() / _safe_pos(pl.col("train_ret_mean").abs(), 1e-12)).alias("ratio_ret_mean_abs"),
        (pl.col("test_hit_excl")       / _safe_pos(pl.col("train_hit_excl"), 1e-12)).alias("ratio_hit_excl"),
        (pl.col("test_pf")             / _safe_pos(pl.col("train_pf"), 1e-12)).alias("ratio_pf"),
        (pl.col("train_n").fill_null(0) + pl.col("test_n").fill_null(0)).alias("n_total_seen"),
    )
    .sort(["wf_id","rank_in_window","entry_col"])
)

_print_kv("decay_rows", decay.height)
_print_kv("decay_cols", decay.width)

print("💾 Guardando wf_decay ...")
decay.write_parquet(DECAY_PARQ)
with DECAY_JSON.open("w", encoding="utf-8") as f:
    json.dump({
        "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
        "symbol": SYMBOL,
        "timeframe": TF,
        "rows_head": decay.head(50).to_dicts(),
    }, f, indent=2, ensure_ascii=False)

print("📈 Resumen de decay (global y ranking) ...")
has_test_n = "test_n" in decay.columns
w = (pl.col("test_n").fill_null(0).cast(pl.Float64())) if has_test_n else pl.lit(1.0)

summary = (
    decay
    .with_columns(w.alias("__w__"))
    .select(
        ( (pl.col("delta_ret_mean") * pl.col("__w__")).sum() / _safe_pos(pl.col("__w__").sum(), 1e-12) ).alias("avg_delta_ret_mean"),
        ( (pl.col("delta_hit_excl") * pl.col("__w__")).sum() / _safe_pos(pl.col("__w__").sum(), 1e-12) ).alias("avg_delta_hit_excl"),
        ( (pl.col("delta_pf")       * pl.col("__w__")).sum() / _safe_pos(pl.col("__w__").sum(), 1e-12) ).alias("avg_delta_pf"),
        ( (pl.col("ratio_ret_mean_abs") * pl.col("__w__")).sum() / _safe_pos(pl.col("__w__").sum(), 1e-12) ).alias("avg_ratio_ret_mean_abs"),
        ( (pl.col("ratio_hit_excl")     * pl.col("__w__")).sum() / _safe_pos(pl.col("__w__").sum(), 1e-12) ).alias("avg_ratio_hit_excl"),
        ( (pl.col("ratio_pf")           * pl.col("__w__")).sum() / _safe_pos(pl.col("__w__").sum(), 1e-12) ).alias("avg_ratio_pf"),
    )
)

rank = (
    decay
    .select(
        "wf_id","entry_col","rank_in_window",
        "train_ret_mean","test_ret_mean","delta_ret_mean","ratio_ret_mean_abs",
        "train_hit_excl","test_hit_excl","delta_hit_excl","ratio_hit_excl",
        "train_pf","test_pf","delta_pf","ratio_pf",
        "train_n","test_n","n_total_seen"
    )
    .with_columns(
        pl.col("ratio_pf").fill_null(0.0).alias("_ratio_pf_"),
        pl.col("ratio_hit_excl").fill_null(0.0).alias("_ratio_hit_"),
        pl.col("delta_ret_mean").abs().fill_null(0.0).alias("_abs_dret_")
    )
    .with_columns(
        ( (pl.col("_ratio_pf_") + pl.col("_ratio_hit_"))/2.0 - pl.col("_abs_dret_") ).alias("robustness_score")
    )
    .drop(["_ratio_pf_","_ratio_hit_","_abs_dret_"])
    .sort(["wf_id","robustness_score"], descending=[False, True])
)

with SUMMARY_JSON.open("w", encoding="utf-8") as f:
    json.dump({
        "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
        "symbol": SYMBOL,
        "timeframe": TF,
        "global_summary": (summary.to_dicts()[0] if summary.height>0 else {}),
        "top5_by_robustness": rank.sort("robustness_score", descending=True).to_dicts()[:5],
        "bottom5_by_robustness": rank.sort("robustness_score", descending=False).to_dicts()[:5],
    }, f, indent=2, ensure_ascii=False)

_print_kv("DECAY_PARQ", DECAY_PARQ)
_print_kv("DECAY_JSON", DECAY_JSON)
_print_kv("SUMMARY_JSON", SUMMARY_JSON)
print("✅ Celda 9 (v1 FIX FINAL) completada.\n")


In [ ]:
# Celda 10 (wf_gate_export v1): [WF GATES & EXPORT CHAMPIONS → reports/wf_champions_*, champions_global_*, prod_manifest_*]
# ---------------------------------------------------------------------------------------------------------------
# - Requisitos previos (celdas 5–9, 7c_bis):
#     forward_meta_{..}.json (dedup_completed==true), wf_train_scores_{..}.parquet, wf_selected_{..}.parquet,
#     wf_select_eval_{..}.parquet, wf_decay_{..}.parquet/.json, wf_pbo_{..}.json, wf_ds_{..}.json (si se exige DS).
# - Gates (configurables desde config_{..}.json → cfg['wf']['champions'] / cfg['champions']):
#     * PBO global <= max_pbo
#     * DS ok por ventana (opcional)
#     * En TEST por combo: n>=min_test_n, hit_excl>=min_test_hit, pf>=min_test_pf,
#                          ratio_pf>=min_ratio_pf, ratio_hit_excl>=min_ratio_hit, |Δret_mean|<=max_delta_abs (opcional)
# - Selección: por ventana, toma hasta K (max_champions_per_window) entre los que pasan gates, ordenados por rank_metric.
# - Salidas:
#     reports/wf_champions_{SYMBOL}_{TF}.parquet / .json
#     reports/champions_global_{SYMBOL}_{TF}.json
#     reports/prod_manifest_{SYMBOL}_{TF}.json
# - Compat "legacy": no pl.sqrt(); manejo seguro de nulos/divisiones.
# ---------------------------------------------------------------------------------------------------------------

import os, json
from pathlib import Path
from datetime import datetime, timezone
import polars as pl

# -------------------------------------- Helpers --------------------------------------
def _print_kv(k, v): print(f"   - {k:<30}: {v}")

def _safe_pos(expr: pl.Expr, floor: float) -> pl.Expr:
    return pl.when(expr < pl.lit(floor)).then(pl.lit(floor)).otherwise(expr)

def _read_json(path: Path) -> dict:
    try:
        return json.loads(path.read_text(encoding="utf-8"))
    except Exception as e:
        raise RuntimeError(f"❌ Error leyendo JSON: {path} → {e}")

def _parse_cfg_gate(cfg: dict, section_keys: list[str], key: str, default):
    """
    Busca cfg[section_keys[0]][section_keys[1]]...[key]; si no existe, devuelve default.
    """
    node = cfg
    try:
        for s in section_keys:
            node = (node.get(s, {}) if isinstance(node, dict) else {})
        return node.get(key, default) if isinstance(node, dict) else default
    except Exception:
        return default

def _extract_ds_ok_map(ds_json: dict, wf_ids: list[str]) -> dict[str, bool]:
    """
    Intenta extraer 'ok' por ventana desde estructuras flexibles:
    - details: [{wf_id, ds_ok/shift_ok/ok, ...}]
    - windows: idem
    - global: ds_ok/shift_ok/ok
    Por defecto: True si no se puede inferir.
    """
    m: dict[str, bool] = {}
    details = []
    for k in ("details", "windows"):
        if isinstance(ds_json.get(k, None), list):
            details = ds_json[k]
            break

    for it in (details or []):
        wid = it.get("wf_id") or it.get("id")
        val = None
        for kk in ("ds_ok", "shift_ok", "ok"):
            if kk in it and isinstance(it[kk], bool):
                val = bool(it[kk]); break
        if wid is not None and isinstance(val, bool):
            m[str(wid)] = val

    if not m:
        # Fallback global
        global_ok = None
        for kk in ("ds_ok", "shift_ok", "ok"):
            if kk in ds_json and isinstance(ds_json[kk], bool):
                global_ok = bool(ds_json[kk]); break
        if isinstance(global_ok, bool):
            for wid in wf_ids:
                m[str(wid)] = global_ok

    # Si aún faltan ids, asumir True (no bloqueante)
    for wid in wf_ids:
        if str(wid) not in m:
            m[str(wid)] = True

    return m

# -------------------------------------- A) Config/paths --------------------------------------
print("📖 Leyendo configuración & gates ...")
need_env = ("TWF_SYMBOL","TWF_TF","TWF_START_DATE","TWF_END_DATE","TWF_SEED_BASE")
assert all(k in os.environ for k in need_env), "❌ Faltan TWF_*; ejecuta celdas previas."
SYMBOL = os.environ["TWF_SYMBOL"]; TF = os.environ["TWF_TF"]

PROJ_ROOT   = Path(os.environ.get("TWF_BASE", Path.cwd())).resolve()
CONFIG_PATH = PROJ_ROOT / "outputs" / SYMBOL / TF / "logs" / f"config_{SYMBOL}_{TF}.json"
CFG = _read_json(CONFIG_PATH)

PATHS           = {k: Path(v) for k, v in CFG["paths"].items()}
REPORTS_DIR     = PATHS["reports"]; REPORTS_DIR.mkdir(parents=True, exist_ok=True)
LOGS_DIR        = PATHS["logs"]
FORWARD_META    = LOGS_DIR / f"forward_meta_{SYMBOL}_{TF}.json"
WF_JSON         = LOGS_DIR / f"wf_windows_{SYMBOL}_{TF}.json"
TRAIN_SCORES_PQ = REPORTS_DIR / f"wf_train_scores_{SYMBOL}_{TF}.parquet"
SEL_PARQ        = REPORTS_DIR / f"wf_selected_{SYMBOL}_{TF}.parquet"
EVAL_PARQ       = REPORTS_DIR / f"wf_select_eval_{SYMBOL}_{TF}.parquet"
DECAY_PARQ      = REPORTS_DIR / f"wf_decay_{SYMBOL}_{TF}.parquet"
DECAY_JSON      = REPORTS_DIR / f"wf_decay_{SYMBOL}_{TF}.json"
PBO_JSON        = REPORTS_DIR / f"wf_pbo_{SYMBOL}_{TF}.json"
DS_JSON         = REPORTS_DIR / f"wf_ds_{SYMBOL}_{TF}.json"  # creado en 7c_bis_ds

# Salidas
WF_CHAMPS_PARQ  = REPORTS_DIR / f"wf_champions_{SYMBOL}_{TF}.parquet"
WF_CHAMPS_JSON  = REPORTS_DIR / f"wf_champions_{SYMBOL}_{TF}.json"
GLOBAL_JSON     = REPORTS_DIR / f"champions_global_{SYMBOL}_{TF}.json"
MANIFEST_JSON   = REPORTS_DIR / f"prod_manifest_{SYMBOL}_{TF}.json"

# Gates/params (con defaults razonables; se pueden ajustar en config)
GATE_REQUIRE_DS_OK      = bool(_parse_cfg_gate(CFG, ["wf","champions"], "require_ds_ok", False))
GATE_MAX_PBO            = float(_parse_cfg_gate(CFG, ["wf","champions"], "max_pbo", 0.70))
MIN_TEST_N              = int(_parse_cfg_gate(CFG, ["wf","champions"], "min_test_n_signals", 100))
MIN_TEST_HIT_EXCL       = float(_parse_cfg_gate(CFG, ["wf","champions"], "min_test_hit_excl", 0.30))
MIN_TEST_PF             = float(_parse_cfg_gate(CFG, ["wf","champions"], "min_test_pf", 0.90))
MIN_RATIO_PF            = float(_parse_cfg_gate(CFG, ["wf","champions"], "min_ratio_pf", 0.85))
MIN_RATIO_HIT_EXCL      = float(_parse_cfg_gate(CFG, ["wf","champions"], "min_ratio_hit_excl", 0.90))
MAX_DELTA_RET_MEAN_ABS  = _parse_cfg_gate(CFG, ["wf","champions"], "max_delta_ret_mean_abs", None)  # None => no gate
RANK_METRIC             = str(_parse_cfg_gate(CFG, ["wf","champions"], "rank_metric", "test_pf"))
MAX_CHAMPIONS_PER_WF    = int(_parse_cfg_gate(CFG, ["wf","champions"], "max_champions_per_window", 1))
MIN_WINDOWS_PASS_GLOBAL = int(_parse_cfg_gate(CFG, ["wf","champions"], "min_windows_pass", 1))

# Dedup gate
assert FORWARD_META.exists(), f"❌ Falta META de dedup: {FORWARD_META}. Ejecuta 4d."
meta = _read_json(FORWARD_META)
assert meta.get("dedup_completed", False) is True, "❌ forward no está deduplicado (Celda 4d)."

# Archivos base
for p in [TRAIN_SCORES_PQ, SEL_PARQ, EVAL_PARQ, DECAY_PARQ, DECAY_JSON, PBO_JSON, WF_JSON]:
    assert Path(p).exists(), f"❌ Falta entrada requerida: {p}"
if GATE_REQUIRE_DS_OK:
    assert DS_JSON.exists(), f"❌ Falta {DS_JSON}. Ejecuta 07c_bis_ds."

_print_kv("SYMBOL", SYMBOL); _print_kv("TF", TF)
_print_kv("GATE_REQUIRE_DS_OK", GATE_REQUIRE_DS_OK)
_print_kv("GATE_MAX_PBO", GATE_MAX_PBO)
_print_kv("MIN_TEST_N", MIN_TEST_N)
_print_kv("MIN_TEST_HIT_EXCL", MIN_TEST_HIT_EXCL)
_print_kv("MIN_TEST_PF", MIN_TEST_PF)
_print_kv("MIN_RATIO_PF", MIN_RATIO_PF)
_print_kv("MIN_RATIO_HIT_EXCL", MIN_RATIO_HIT_EXCL)
_print_kv("MAX_DELTA_RET_MEAN_ABS", MAX_DELTA_RET_MEAN_ABS)
_print_kv("RANK_METRIC", RANK_METRIC)
_print_kv("MAX_CHAMPIONS_PER_WF", MAX_CHAMPIONS_PER_WF)
_print_kv("MIN_WINDOWS_PASS_GLOBAL", MIN_WINDOWS_PASS_GLOBAL)

# -------------------------------------- B) Carga datasets --------------------------------------
print("📦 Cargando datasets WF ...")
eval_df   = pl.read_parquet(EVAL_PARQ)
decay_df  = pl.read_parquet(DECAY_PARQ)
sel_df    = pl.read_parquet(SEL_PARQ)
wf_obj    = _read_json(WF_JSON)
wf_ids    = [w.get("id") for w in wf_obj.get("windows", []) if "id" in w]
pbo_obj   = _read_json(PBO_JSON)
ds_obj    = _read_json(DS_JSON) if DS_JSON.exists() else {}

# PBO gate (global)
pbo_val = pbo_obj.get("PBO", None)
pbo_ok  = True if (pbo_val is None) else (pbo_val <= GATE_MAX_PBO)
_print_kv("PBO_value", pbo_val); _print_kv("PBO_OK", pbo_ok)

# DS gate (por ventana)
ds_ok_map = _extract_ds_ok_map(ds_obj, [str(x) for x in wf_ids])
ds_ok_df  = pl.DataFrame({"wf_id": list(ds_ok_map.keys()), "ds_ok": list(ds_ok_map.values())}) if ds_ok_map else pl.DataFrame({"wf_id":[], "ds_ok":[]})

# Reducimos eval a metadatos (evita colisiones con decay)
keep_eval_cols = [c for c in ["wf_id","test_start","test_end","entry_col","rank_in_window","cluster_id","cluster_size"] if c in eval_df.columns]
eval_meta = eval_df.select(keep_eval_cols) if keep_eval_cols else pl.DataFrame({"wf_id": pl.Series([], dtype=pl.Utf8)})

# Decay trae train_* y test_* + ratios/deltas; lo usaremos como base
need_decay_cols = [
    "wf_id","entry_col","rank_in_window","cluster_id","cluster_size",  # por si viniera de decay (tenemos algunos)
    "train_n","test_n",
    "train_ret_mean","test_ret_mean","delta_ret_mean","ratio_ret_mean_abs",
    "train_hit_excl","test_hit_excl","delta_hit_excl","ratio_hit_excl",
    "train_pf","test_pf","delta_pf","ratio_pf"
]
have_decay_cols = [c for c in need_decay_cols if c in decay_df.columns]
base = decay_df.select(have_decay_cols)

# Unimos metadatos (para fechas de test y clusters si faltan)
if eval_meta.height > 0:
    base = base.join(eval_meta, on=["wf_id","entry_col"], how="left", suffix="_evaldup")

# DS por ventana
if ds_ok_df.height > 0:
    base = base.join(ds_ok_df, on="wf_id", how="left")
else:
    base = base.with_columns(pl.lit(True).alias("ds_ok"))

# PBO global
base = base.with_columns(pl.lit(bool(pbo_ok)).alias("pbo_ok"))

_print_kv("rows_base_before_gates", base.height)
assert base.height > 0, "❌ No hay filas en base para evaluar gates."

# -------------------------------------- C) Aplicar Gates --------------------------------------
print("🚧 Aplicando gates por combo/ventana ...")

# Condiciones dinámicas
gate_expr = pl.lit(True)

# n señales en TEST
if "test_n" in base.columns and MIN_TEST_N is not None and MIN_TEST_N > 0:
    gate_expr = gate_expr & (pl.col("test_n").fill_null(0) >= pl.lit(MIN_TEST_N))

# hit rate excl TO en TEST
if "test_hit_excl" in base.columns and MIN_TEST_HIT_EXCL is not None:
    gate_expr = gate_expr & (pl.col("test_hit_excl").fill_null(-1.0) >= pl.lit(MIN_TEST_HIT_EXCL))

# PF en TEST
if "test_pf" in base.columns and MIN_TEST_PF is not None:
    gate_expr = gate_expr & (pl.col("test_pf").fill_null(-1.0) >= pl.lit(MIN_TEST_PF))

# Ratios
if "ratio_pf" in base.columns and MIN_RATIO_PF is not None:
    gate_expr = gate_expr & (pl.col("ratio_pf").fill_null(-1.0) >= pl.lit(MIN_RATIO_PF))

if "ratio_hit_excl" in base.columns and MIN_RATIO_HIT_EXCL is not None:
    gate_expr = gate_expr & (pl.col("ratio_hit_excl").fill_null(-1.0) >= pl.lit(MIN_RATIO_HIT_EXCL))

# |Δret_mean| absoluto
if ("delta_ret_mean" in base.columns) and (MAX_DELTA_RET_MEAN_ABS is not None):
    gate_expr = gate_expr & (pl.col("delta_ret_mean").abs().fill_null(1e9) <= pl.lit(float(MAX_DELTA_RET_MEAN_ABS)))

# DS gate (si se exige)
if GATE_REQUIRE_DS_OK:
    gate_expr = gate_expr & (pl.col("ds_ok").fill_null(False) == pl.lit(True))

# PBO gate global
gate_expr = gate_expr & (pl.col("pbo_ok") == pl.lit(True))

scored = base.with_columns(gate_expr.alias("gate_pass"))

n_pass = int(scored.filter(pl.col("gate_pass")==True).height)
_print_kv("passed_gate_rows", n_pass)

# -------------------------------------- D) Campeones por ventana --------------------------------------
print("🏆 Seleccionando campeones por ventana ...")

# Orden de ranking (desc) para métricas; rank_in_window asc como último criterio
rank_primary = RANK_METRIC if RANK_METRIC in scored.columns else ("test_pf" if "test_pf" in scored.columns else "test_hit_excl")
sort_cols = [c for c in [rank_primary, "ratio_pf", "test_hit_excl"] if c in scored.columns]
sort_desc = [True] * len(sort_cols)

# Filtramos aprobados y ordenamos
approved = (
    scored
    .filter(pl.col("gate_pass")==True)
    .sort(["wf_id"] + sort_cols + (["rank_in_window"] if "rank_in_window" in scored.columns else []),
          descending=[False] + sort_desc + ([False] if "rank_in_window" in scored.columns else []))
)

# Tomar hasta K por wf_id
# Nota: group_by("wf_id").head(K) respeta el orden de aparición
if approved.height > 0:
    champs = approved.group_by("wf_id").head(MAX_CHAMPIONS_PER_WF)
else:
    champs = pl.DataFrame({"wf_id": pl.Series([], dtype=pl.Utf8)})

_print_kv("wf_champions_rows", champs.height)
_print_kv("wf_champions_cols", champs.width)

# Persistimos campeones por ventana
print("💾 Guardando wf_champions ...")
champs.write_parquet(WF_CHAMPS_PARQ)
WF_CHAMPS_JSON.write_text(json.dumps({
    "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
    "symbol": SYMBOL, "timeframe": TF,
    "rank_metric": rank_primary,
    "max_champions_per_window": int(MAX_CHAMPIONS_PER_WF),
    "gates": {
        "require_ds_ok": bool(GATE_REQUIRE_DS_OK),
        "max_pbo": float(GATE_MAX_PBO),
        "min_test_n": int(MIN_TEST_N),
        "min_test_hit_excl": float(MIN_TEST_HIT_EXCL),
        "min_test_pf": float(MIN_TEST_PF),
        "min_ratio_pf": float(MIN_RATIO_PF),
        "min_ratio_hit_excl": float(MIN_RATIO_HIT_EXCL),
        "max_delta_ret_mean_abs": (float(MAX_DELTA_RET_MEAN_ABS) if MAX_DELTA_RET_MEAN_ABS is not None else None),
    },
    "rows": champs.to_dicts()[:200]
}, indent=2, ensure_ascii=False), encoding="utf-8")
_print_kv("WF_CHAMPS_PARQ", WF_CHAMPS_PARQ); _print_kv("WF_CHAMPS_JSON", WF_CHAMPS_JSON)

# -------------------------------------- E) Resumen global & Manifest --------------------------------------
print("📈 Construyendo resumen global y manifest ...")

if champs.height > 0:
    # Agregados por entry_col sobre campeones
    agg_cols = [c for c in ["test_pf","test_hit_excl","ratio_pf","ratio_hit_excl","delta_ret_mean","test_n"] if c in champs.columns]
    global_tbl = (
        champs
        .group_by("entry_col")
        .agg(
            pl.len().alias("windows_pass"),
            *([pl.col("test_pf").mean().alias("avg_test_pf")] if "test_pf" in champs.columns else []),
            *([pl.col("test_hit_excl").mean().alias("avg_test_hit_excl")] if "test_hit_excl" in champs.columns else []),
            *([pl.col("ratio_pf").mean().alias("avg_ratio_pf")] if "ratio_pf" in champs.columns else []),
            *([pl.col("ratio_hit_excl").mean().alias("avg_ratio_hit_excl")] if "ratio_hit_excl" in champs.columns else []),
            *([pl.col("delta_ret_mean").abs().mean().alias("avg_abs_delta_ret_mean")] if "delta_ret_mean" in champs.columns else []),
            *([pl.col("test_n").sum().alias("sum_test_n")] if "test_n" in champs.columns else []),
        )
        .with_columns(
            (
                (
                    (pl.col("avg_ratio_pf") if "avg_ratio_pf" in champs.group_by("entry_col").agg(pl.len()).columns else pl.lit(0.0))
                    +
                    (pl.col("avg_ratio_hit_excl") if "avg_ratio_hit_excl" in champs.group_by("entry_col").agg(pl.len()).columns else pl.lit(0.0))
                ) / 2.0
                - (pl.col("avg_abs_delta_ret_mean") if "avg_abs_delta_ret_mean" in champs.group_by("entry_col").agg(pl.len()).columns else pl.lit(0.0))
            ).alias("score_global")
        )
        .sort(["windows_pass","score_global"], descending=[True, True])
    )
else:
    global_tbl = pl.DataFrame({"entry_col": pl.Series([], dtype=pl.Utf8)})

topN_preview = global_tbl.head(10).to_dicts() if global_tbl.height > 0 else []

GLOBAL_JSON.write_text(json.dumps({
    "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
    "symbol": SYMBOL, "timeframe": TF,
    "rank_metric": rank_primary,
    "summary_top10": topN_preview
}, indent=2, ensure_ascii=False), encoding="utf-8")
_print_kv("GLOBAL_JSON", GLOBAL_JSON)

# Manifest de producción: combos con >= MIN_WINDOWS_PASS_GLOBAL apariciones como campeón
if global_tbl.height > 0:
    manifest_df = global_tbl.filter(pl.col("windows_pass") >= pl.lit(int(MIN_WINDOWS_PASS_GLOBAL))).select("entry_col","windows_pass","score_global")
    manifest = {
        "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
        "symbol": SYMBOL, "timeframe": TF,
        "min_windows_pass": int(MIN_WINDOWS_PASS_GLOBAL),
        "pbo_ok": bool(pbo_ok),
        "pbo_value": pbo_val,
        "require_ds_ok": bool(GATE_REQUIRE_DS_OK),
        "entries": manifest_df.to_dicts()
    }
else:
    manifest = {
        "created_utc": datetime.now(timezone.utc).isoformat(timespec="seconds"),
        "symbol": SYMBOL, "timeframe": TF,
        "min_windows_pass": int(MIN_WINDOWS_PASS_GLOBAL),
        "pbo_ok": bool(pbo_ok),
        "pbo_value": pbo_val,
        "require_ds_ok": bool(GATE_REQUIRE_DS_OK),
        "entries": []
    }

MANIFEST_JSON.write_text(json.dumps(manifest, indent=2, ensure_ascii=False), encoding="utf-8")
_print_kv("MANIFEST_JSON", MANIFEST_JSON)

print("✅ Celda 10 finalizada sin errores.\n")


📖 Leyendo configuración & gates ...
   - SYMBOL                        : BTCUSDT
   - TF                            : 15m
   - GATE_REQUIRE_DS_OK            : False
   - GATE_MAX_PBO                  : 0.7
   - MIN_TEST_N                    : 100
   - MIN_TEST_HIT_EXCL             : 0.3
   - MIN_TEST_PF                   : 0.9
   - MIN_RATIO_PF                  : 0.85
   - MIN_RATIO_HIT_EXCL            : 0.9
   - MAX_DELTA_RET_MEAN_ABS        : None
   - RANK_METRIC                   : test_pf
   - MAX_CHAMPIONS_PER_WF          : 1
   - MIN_WINDOWS_PASS_GLOBAL       : 1
📦 Cargando datasets WF ...
   - PBO_value                     : 0.25
   - PBO_OK                        : True
   - rows_base_before_gates        : 12
🚧 Aplicando gates por combo/ventana ...
   - passed_gate_rows              : 0
🏆 Seleccionando campeones por ventana ...
   - wf_champions_rows             : 0
   - wf_champions_cols             : 1
💾 Guardando wf_champions ...
   - WF_CHAMPS_PARQ                : C:\Quan

In [ ]:
# Celda 11 (reportes HTML v1.1): fallback sin jinja2 (HTML manual con degradado inline)
# ------------------------------------------------------------------------------------------------------------
# Entradas: kpi_*.parquet, wf_select_eval_*.parquet, wf_decay_*.parquet/json, wf_champions_*.parquet,
#           wf_pbo_*.json, states_*.parquet, surfaces_* y surfaces_qc_summary_*.json
# Salidas : report_global_{SYMBOL}_{TF}.html  y  report_windows_{SYMBOL}_{TF}.html
# Naturaleza: 100% estadístico (sin métricas de backtest). No modifica artefactos previos.
# ------------------------------------------------------------------------------------------------------------

import os, json, math, importlib.util
from pathlib import Path
from datetime import datetime, timezone
import polars as pl
import pandas as pd

HAS_J2 = importlib.util.find_spec("jinja2") is not None

def _print_kv(k, v): 
    print(f"   - {k:<30}: {v}")

def _read_json_safe(p: Path, default=None):
    if p.exists():
        try:
            return json.loads(p.read_text(encoding="utf-8"))
        except Exception:
            return default
    return default

# -------- helpers de render --------

def _is_numeric_dtype(ser):
    try:
        import numpy as np
        return pd.api.types.is_numeric_dtype(ser)
    except Exception:
        return False

def _df_to_html_manual(pdf: pd.DataFrame, caption: str = "", cmap_columns=None, precision=6):
    """Render HTML manual con degradado por-columna vía HSL (sin Styler)."""
    if pdf.shape[0] == 0:
        return f'<h3>{caption}</h3><p><i>(sin filas)</i></p>'
    pdf = pdf.copy()

    if cmap_columns is None:
        cmap_columns = [c for c in pdf.columns if _is_numeric_dtype(pdf[c])]

    # rangos por columna (evitar división por cero)
    col_minmax = {}
    for c in cmap_columns:
        ser = pd.to_numeric(pdf[c], errors="coerce")
        vmin = float(pd.Series(ser).min(skipna=True)) if ser.notna().any() else 0.0
        vmax = float(pd.Series(ser).max(skipna=True)) if ser.notna().any() else 1.0
        if math.isfinite(vmin) and math.isfinite(vmax) and vmax - vmin > 1e-12:
            col_minmax[c] = (vmin, vmax)
        else:
            col_minmax[c] = (0.0, 1.0)

    def color_for(val, vmin, vmax):
        try:
            x = float(val)
            if not math.isfinite(x):
                return ""
        except Exception:
            return ""
        t = (x - vmin) / (vmax - vmin + 1e-12)
        t = 0.0 if t < 0 else 1.0 if t > 1 else t
        # 0=rojo (h=0), 1=verde (h=120)
        hue = 120.0 * t
        return f"background-color:hsl({hue:.0f},70%,65%);"

    # construir tabla
    html = []
    html.append("<table>")
    if caption:
        html.append(f"<caption style='caption-side:top;font-weight:bold;font-size:14px'>{caption}</caption>")
    # header
    html.append("<thead><tr>")
    for c in pdf.columns:
        html.append(f"<th style='text-align:center;font-size:12px'>{c}</th>")
    html.append("</tr></thead>")
    # body
    html.append("<tbody>")
    for _, row in pdf.iterrows():
        html.append("<tr>")
        for c in pdf.columns:
            val = row[c]
            styles = ""
            if c in col_minmax and _is_numeric_dtype(pdf[c]):
                vmin, vmax = col_minmax[c]
                styles = color_for(val, vmin, vmax)
            # formateo
            if isinstance(val, float):
                sval = f"{val:.{precision}f}"
            else:
                sval = str(val)
            html.append(f"<td style='border:1px solid #ddd;padding:4px 6px;{styles}'>{sval}</td>")
        html.append("</tr>")
    html.append("</tbody></table>")
    return "".join(html)

def _df_to_html(df: pl.DataFrame, caption: str = "", cmap_columns=None, precision=6):
    """Usa pandas Styler si hay jinja2, si no usa motor manual con degradado inline."""
    if df.is_empty():
        return f'<h3>{caption}</h3><p><i>(sin filas)</i></p>'
    pdf = df.to_pandas()
    # columnas numéricas por defecto
    if cmap_columns is None:
        cmap_columns = [c for c in pdf.columns if _is_numeric_dtype(pdf[c])]

    if HAS_J2:
        try:
            sty = (pdf.style
                   .format(precision=precision)
                   .background_gradient(subset=cmap_columns, cmap="RdYlGn")
                   .set_table_styles([
                       {'selector': 'th', 'props': [('font-size', '12px'), ('text-align', 'center')]},
                       {'selector': 'td', 'props': [('font-size', '12px')]},
                       {'selector': 'caption', 'props': [('caption-side', 'top'), ('font-weight', 'bold'), ('font-size', '14px')]}
                   ])
                   .set_caption(caption)
                   .hide(axis="index"))
            return sty.to_html()
        except Exception:
            # fallback manual si falla Styler por cualquier razón
            pass
    return _df_to_html_manual(pdf, caption=caption, cmap_columns=cmap_columns, precision=precision)

def _matrix_to_html(path_parq: Path, title: str, value_label: str = "valor", precision=6):
    """Carga un wide y lo convierte en HTML con degradado; fallback manual sin jinja2."""
    if not path_parq.exists():
        return ""
    w = pl.read_parquet(path_parq)
    if w.is_empty():
        return ""
    pdf = w.to_pandas().set_index("entry_col")
    if HAS_J2:
        try:
            sty = (pdf.style
                   .format(precision=precision)
                   .background_gradient(cmap="RdYlGn", axis=None)
                   .set_table_styles([
                       {'selector': 'th', 'props': [('font-size', '12px'), ('text-align', 'center')]},
                       {'selector': 'td', 'props': [('font-size', '12px')]},
                       {'selector': 'caption', 'props': [('caption-side', 'top'), ('font-weight', 'bold'), ('font-size', '14px')]}
                   ])
                   .set_caption(f"{title} — {value_label}"))
            return sty.to_html()
        except Exception:
            pass
    # manual
    pdf = pdf.reset_index()
    return _df_to_html_manual(pdf, caption=f"{title} — {value_label}", cmap_columns=[c for c in pdf.columns if c != "entry_col"], precision=precision)

# -------------------- A) Config y rutas --------------------
print("📖 Leyendo configuración de reportes ...")
need_env = ("TWF_SYMBOL","TWF_TF","TWF_START_DATE","TWF_END_DATE","TWF_SEED_BASE")
assert all(k in os.environ for k in need_env), "Faltan TWF_*; ejecuta celdas previas."
SYMBOL = os.environ["TWF_SYMBOL"]; TF = os.environ["TWF_TF"]

PROJ_ROOT   = Path(os.environ.get("TWF_BASE", Path.cwd())).resolve()
CFG_PATH    = PROJ_ROOT / "outputs" / SYMBOL / TF / "logs" / f"config_{SYMBOL}_{TF}.json"
CFG         = json.loads(CFG_PATH.read_text(encoding="utf-8"))

PATHS       = {k: Path(v) for k, v in CFG["paths"].items()}
REPORTS_DIR = PATHS["reports"]; REPORTS_DIR.mkdir(parents=True, exist_ok=True)

# Entradas esperadas
KPI_PARQ        = REPORTS_DIR / f"kpi_{SYMBOL}_{TF}.parquet"
WF_EVAL_PARQ    = REPORTS_DIR / f"wf_select_eval_{SYMBOL}_{TF}.parquet"
WF_DECAY_PARQ   = REPORTS_DIR / f"wf_decay_{SYMBOL}_{TF}.parquet"
WF_DECAY_SUM    = REPORTS_DIR / f"wf_decay_summary_{SYMBOL}_{TF}.json"
WF_CHAMPS_PARQ  = REPORTS_DIR / f"wf_champions_{SYMBOL}_{TF}.parquet"
PBO_JSON        = REPORTS_DIR / f"wf_pbo_{SYMBOL}_{TF}.json"
QC_JSON         = REPORTS_DIR / f"surfaces_qc_summary_{SYMBOL}_{TF}.json"
STATES_PARQ     = REPORTS_DIR / f"states_{SYMBOL}_{TF}.parquet"

# Salidas
HTML_GLOBAL     = REPORTS_DIR / f"report_global_{SYMBOL}_{TF}.html"
HTML_WINDOWS    = REPORTS_DIR / f"report_windows_{SYMBOL}_{TF}.html"

_print_kv("KPI_PARQ", KPI_PARQ)
_print_kv("WF_EVAL_PARQ", WF_EVAL_PARQ)
_print_kv("WF_DECAY_PARQ", WF_DECAY_PARQ)
_print_kv("WF_CHAMPS_PARQ", WF_CHAMPS_PARQ)
_print_kv("PBO_JSON", PBO_JSON)
_print_kv("QC_JSON", QC_JSON)
_print_kv("Styling (jinja2)", HAS_J2)

# -------------------- B) Cargar artefactos -----------------
kpi      = pl.read_parquet(KPI_PARQ)          if KPI_PARQ.exists() else pl.DataFrame()
eval_df  = pl.read_parquet(WF_EVAL_PARQ)      if WF_EVAL_PARQ.exists() else pl.DataFrame()
decay    = pl.read_parquet(WF_DECAY_PARQ)     if WF_DECAY_PARQ.exists() else pl.DataFrame()
decay_js = _read_json_safe(WF_DECAY_SUM, default={})
champs   = pl.read_parquet(WF_CHAMPS_PARQ)    if WF_CHAMPS_PARQ.exists() else pl.DataFrame()
pbo_js   = _read_json_safe(PBO_JSON, default={})
qc_js    = _read_json_safe(QC_JSON, default={"summary": []})

# Para detectar state_types presentes (para heatmaps)
state_types = []
if STATES_PARQ.exists():
    st = pl.read_parquet(STATES_PARQ).select("state_type").unique()
    state_types = [r[0] for r in st.iter_rows()]

# -------------------- C) Secciones HTML (Global) -----------
print("🧩 Construyendo HTML global ...")
sections = []

# Header
header_html = f"""
<h1>Reporte Estadístico — {SYMBOL} {TF}</h1>
<p>Generado UTC: {datetime.now(timezone.utc).isoformat(timespec="seconds")}</p>
<hr/>
"""
sections.append(header_html)

# PBO
if pbo_js:
    pbo_val = pbo_js.get("PBO", None)
    usable  = pbo_js.get("windows_usable", 0)
    if pbo_val is not None:
        pbo_html = f"""
        <h2>PBO (Overfit Probability)</h2>
        <p><b>PBO</b>: {float(pbo_val):.4f} &nbsp; | &nbsp; <b>Ventanas evaluadas</b>: {int(usable)}</p>
        """
    else:
        pbo_html = "<h2>PBO (Overfit Probability)</h2><p><i>No disponible</i></p>"
    # Detalle (limitado)
    details = pbo_js.get("details", [])[:50]
    if details:
        det_df = pl.DataFrame(details)
        pbo_html += _df_to_html(det_df, caption="Detalle PBO (primeras filas)")
    sections.append(pbo_html)

# Campeones
if not champs.is_empty():
    if all(c in champs.columns for c in ["test_ret_mean","test_hit_excl","test_pf","delta_ret_mean","ratio_hit_excl","ratio_pf","n_total_seen"]):
        champs_view = champs.select([
            "wf_id","entry_col","rank_in_window",
            "test_ret_mean","test_hit_excl","test_pf",
            "delta_ret_mean","ratio_hit_excl","ratio_pf","n_total_seen"
        ])
    else:
        champs_view = champs
    sections.append(_df_to_html(
        champs_view, caption="Campeones por ventana",
        cmap_columns=[c for c in champs_view.columns if c not in ("wf_id","entry_col","rank_in_window")]
    ))

# Decay
if not decay.is_empty():
    cols = [c for c in [
        "wf_id","entry_col","train_n","test_n",
        "train_ret_mean","test_ret_mean","delta_ret_mean","ratio_ret_mean_abs",
        "train_hit_excl","test_hit_excl","delta_hit_excl","ratio_hit_excl",
        "train_pf","test_pf","delta_pf","ratio_pf"
    ] if c in decay.columns]
    decay_view = decay.select(cols).sort(["wf_id","entry_col"])
    sections.append(_df_to_html(
        decay_view, caption="Decay TRAIN→TEST (seleccionados)",
        cmap_columns=[c for c in cols if c not in ("wf_id","entry_col","train_n","test_n")]
    ))
    if decay_js:
        top5 = pl.DataFrame(decay_js.get("top5_by_robustness", []))
        bot5 = pl.DataFrame(decay_js.get("bottom5_by_robustness", []))
        if not top5.is_empty():
            sections.append(_df_to_html(top5, caption="Top-5 robustez (según resumen)", cmap_columns=None))
        if not bot5.is_empty():
            sections.append(_df_to_html(bot5, caption="Bottom-5 robustez (según resumen)", cmap_columns=None))

# KPIs globales (Celda 5)
if not kpi.is_empty():
    cols = [c for c in [
        "entry_col","n_signals","tp","sl","timeout",
        "ret_mean","ret_median","ret_std",
        "hit_rate_tp_incl_to","hit_rate_tp_excl_to","profit_factor_stat",
        "ret_mean_ci95_lo","ret_mean_ci95_hi",
        "hit_rate_excl_to_ci95_lo","hit_rate_excl_to_ci95_hi"
    ] if c in kpi.columns]
    kpi_view = kpi.select(cols).sort("entry_col")
    sections.append(_df_to_html(
        kpi_view, caption="KPIs globales por entry_col (forward por-señal)",
        cmap_columns=[c for c in cols if c not in ("entry_col","tp","sl","timeout","n_signals")]
    ))

# QC de surfaces (resumen)
if qc_js and qc_js.get("summary"):
    qc_df = pl.DataFrame(qc_js["summary"])
    sections.append(_df_to_html(
        qc_df, caption="QC de surfaces por estado (resumen)",
        cmap_columns=[c for c in qc_df.columns if c not in ("state_type","flag_name")]
    ))

# Heatmaps de surfaces (ret_mean)
heatmap_blocks = []
for s in state_types:
    w_ret = REPORTS_DIR / f"surfaces_wide_ret_mean_{SYMBOL}_{TF}_{s}.parquet"
    if w_ret.exists():
        heatmap_blocks.append(_matrix_to_html(w_ret, title=f"Surface — {s}", value_label="ret_mean", precision=6))
if heatmap_blocks:
    sections.append("<h2>Surfaces (ret_mean)</h2>" + "".join(heatmap_blocks))

# Ensamblar y guardar GLOBAL
GLOBAL_HTML = f"""
<html>
<head>
<meta charset="utf-8"/>
<title>Reporte Estadístico — {SYMBOL} {TF}</title>
<style>
body {{ font-family: Arial, sans-serif; margin: 16px; }}
h1, h2, h3 {{ margin: 6px 0; }}
table {{ border-collapse: collapse; margin: 8px 0; }}
th, td {{ border: 1px solid #ddd; padding: 4px 6px; font-size: 12px; }}
caption {{ caption-side: top; font-weight: bold; font-size: 14px; }}
</style>
</head>
<body>
{''.join(sections)}
</body>
</html>
"""
HTML_GLOBAL.write_text(GLOBAL_HTML, encoding="utf-8")
_print_kv("HTML_GLOBAL", HTML_GLOBAL)

# -------------------- D) Reporte por ventana ----------------
print("🧩 Construyendo HTML por ventana ...")
win_sections = [f"""
<h1>Reporte por Ventana — {SYMBOL} {TF}</h1>
<p>Generado UTC: {datetime.now(timezone.utc).isoformat(timespec="seconds")}</p>
<hr/>"""]

if not eval_df.is_empty():
    # Lista de ventanas en eval
    win_ids = eval_df.select("wf_id").unique().to_series().to_list()
    for wid in win_ids:
        sub = eval_df.filter(pl.col("wf_id")==wid).sort(["rank_in_window","entry_col"])
        title = f"Ventana {wid} — Seleccionados (TEST)"
        cols = [c for c in [
            "wf_id","entry_col","rank_in_window","cluster_id","cluster_size","n_signals",
            "ret_mean","hit_rate_tp_excl_to","profit_factor_stat",
            "ret_mean_ci95_lo","ret_mean_ci95_hi",
            "hit_rate_excl_to_ci95_lo","hit_rate_excl_to_ci95_hi"
        ] if c in sub.columns]
        win_sections.append(_df_to_html(
            sub.select(cols), caption=title,
            cmap_columns=[c for c in cols if c not in ("wf_id","entry_col","rank_in_window","cluster_id","cluster_size","n_signals")]
        ))
else:
    win_sections.append("<p><i>No hay evaluación TEST para mostrar.</i></p>")

WINDOWS_HTML = f"""
<html>
<head>
<meta charset="utf-8"/>
<title>Reporte por Ventana — {SYMBOL} {TF}</title>
<style>
body {{ font-family: Arial, sans-serif; margin: 16px; }}
h1, h2, h3 {{ margin: 6px 0; }}
table {{ border-collapse: collapse; margin: 8px 0; }}
th, td {{ border: 1px solid #ddd; padding: 4px 6px; font-size: 12px; }}
caption {{ caption-side: top; font-weight: bold; font-size: 14px; }}
</style>
</head>
<body>
{''.join(win_sections)}
</body>
</html>
"""
HTML_WINDOWS.write_text(WINDOWS_HTML, encoding="utf-8")
_print_kv("HTML_WINDOWS", HTML_WINDOWS)

print("✅ Celda 11 finalizada sin errores (con fallback de estilo si falta jinja2).\n")


📖 Leyendo configuración de reportes ...
   - KPI_PARQ                      : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\kpi_BTCUSDT_15m.parquet
   - WF_EVAL_PARQ                  : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_select_eval_BTCUSDT_15m.parquet
   - WF_DECAY_PARQ                 : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_decay_BTCUSDT_15m.parquet
   - WF_CHAMPS_PARQ                : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_champions_BTCUSDT_15m.parquet
   - PBO_JSON                      : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\wf_pbo_BTCUSDT_15m.json
   - QC_JSON                       : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\surfaces_qc_summary_BTCUSDT_15m.json
   - Styling (jinja2)              : False
🧩 Construyendo HTML global ...
   - HTML_GLOBAL                   : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\report_global_BTCUSDT_15m.html
🧩 Construyendo HTML por ventana ...
   - HTML_WINDOWS                  : C:\Quant\TWF\outputs\BTCUSDT\15m\reports\report_window